In [280]:
import requests
from bs4 import BeautifulSoup
from bs4.element import NavigableString
from selenium import webdriver
from datetime import datetime
import json

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from konlpy.tag import Kkma, Okt

kkma = Kkma()
okt = Okt()
from sklearn.feature_extraction.text import TfidfVectorizer, CounterVetorizer
tfidf = TfidfVectorizer()
cnt_vec = CountVectorizer()

from sklearn.preprocessing import normalize
import re
from gensim.summarization.summarizer import summarize

In [2]:
def text2sentences(text) : 
    sentences = kkma.sentences(text)
    for idx in range(0, len(sentences)) :
        if len(sentences[idx]) <= 10 :
            sentences[idx-1] += (' '+sentences[idx])
            sentences[idx] = ''
        return sentences

def get_nouns(sentences) :
    nouns = []
    for sentence in sentences :
        sentence = text_clean(sentence)
        if sentence is not '' :
            nouns.append(' '.join([noun for noun in okt.nouns(str(sentence))
                                  if noun not in stopwords and len(noun) > 1]))
    return nouns

def build_sent_graph(sentence) :
    tfidf_mat = tfidf.fit_transform(sentence).toarray()
    graph_sentence = np.dot(tfidf_mat, tfidf_mat.T)
    return graph_sentence

def bulid_word_graph(sentence) : 
    cnt_vec_mat = normalize(cnt_vec.fit_transform(sentence).toarray().astype(float), axis=1)
    vocab = cnt_vec.vocabulary_
    return np.dot(cnt_vec_mat, cnt_vec_mat.T), {vocab[word] : word for word in vocab}

###########################

def get_rank(graph, d=0.85) :
    A = graph
    matrix_size = A.shape[0]
    for id in range(matrix_size) : 
        A[id,id] = 0
        link_sum = np.sum(A[:,id])
        if link_sum != 0 :
            A[:,id] /= link_sum
        A[:,id] *= -d
        A[id,id] = 1
    B = (1-d) * np.ones((matrix_size, 1))
    ranks = np.linalg.solve(A,B)
    return {idx : r[0] for idx, r in enumerate(ranks)}

###########################

def summarize_no(sent_num=3) :
    summary = []
    index = []
    for idx in sorted_sent_rank_idx[:sent_num] :
        index.append(idx)
    index.sort()
    
    for idx in index :
        summary.append(sentences[idx])
    
    for text in summary :
        print(text)

def key_word(word_num=10) :
    keywords = []
    index = []
    for idx in sorted_word_rank_idx[:word_num] :
        index.append(idx)
    for idx in index :
        keywords.append(idx2word[idx])
    print(keywords)

    
###########################

def text_clean(doc):
    doc =re.sub("[^a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣]", "", doc)
    return doc

def stopwords_del(x) :
    if any(stopwords['word'] == x) :
        return False
    return True

stopwords = pd.read_csv('stopwords_kr.txt',header=None)
stopwords.columns = ['word']

tfidf = TfidfVectorizer()
cnt_vec = CountVectorizer()

<>:13: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:13: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<ipython-input-2-b537c3e6115e>:13: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if sentence is not '' :


In [3]:
def text2sentences(text) : 
    sentences = kkma.sentences(text)
    for idx in range(0, len(sentences)) :
        if len(sentences[idx]) <= 10 :
            sentences[idx-1] += (' '+sentences[idx])
            sentences[idx] = ''
        return sentences

def get_nouns(sentences) :
    nouns = []
    for sentence in sentences :
        sentence = text_clean(sentence)
        if sentence is not '' :
            nouns.append(' '.join([noun for noun in okt.nouns(str(sentence))
                                  if noun not in stopwords and len(noun) > 1]))
    return nouns

def build_sent_graph(sentence) :
    tfidf_mat = tfidf.fit_transform(sentence).toarray()
    graph_sentence = np.dot(tfidf_mat, tfidf_mat.T)
    return graph_sentence

def get_rank(graph, d=0.85) :
    A = graph
    matrix_size = A.shape[0]
    for id in range(matrix_size) : 
        A[id,id] = 0
        link_sum = np.sum(A[:,id])
        if link_sum != 0 :
            A[:,id] /= link_sum
        A[:,id] *= -d
        A[id,id] = 1
    B = (1-d) * np.ones((matrix_size, 1))
    ranks = np.linalg.solve(A,B)
    return {idx : r[0] for idx, r in enumerate(ranks)}

###########################

def summarize_no(sent_num=3) :
    summary = []
    index = []
    for idx in sorted_sent_rank_idx[:sent_num] :
        index.append(idx)
    index.sort()
    
    for idx in index :
        summary.append(sentences[idx])
    
    for text in summary :
        print(text)

<>:13: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:13: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<ipython-input-3-fe9c5c175971>:13: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if sentence is not '' :


In [5]:
def summarize_last(doc) :
    sentences = text2sentences(doc)
    nouns = get_nouns(sentences)
    graph_sentence =[]
    sent_graph = build_sent_graph(nouns)
    sent_rank_idx = get_rank(sent_graph)
    sorted_sent_rank_idx = sorted(sent_rank_idx, key=lambda k : sent_rank_idx[k], reverse=True)
    summarize_no()

In [242]:
summarize_last(test_df['contents'][5])

상업시설은 엘 시티 지상 1~3 층에 8만 3790㎡ 규모로 조성되며, 개별 분양으로 진행된다.
‘ 엘시 티 더 몰 (the MALL)’ 은 해운대 방문객 수요와 함께 엘 시티 내 마련되는 ‘ 해 운대
분양관계자는 “ 사전에 분양된 엘 시티 내 주거시설이 모두 완 판을 기록하며, 이번에 개별 분양되는 엘 시티 더 몰 (the MALL)에도 투자자들의 관심이 끊이질 않고 있다 ”며 “ 차 후 상업시설이 개장하면 엘 시티 내 여러 부대시설과 함께 해운대를 넘어 부산을 대표하는 랜드 마크 시설로 자리잡을 것” 이라고 전했다.


In [230]:
sentences = text2sentences(test_df['contents'][0])
nouns = get_nouns(sentences)
graph_sentence =[]
sent_graph = build_sent_graph(nouns)
sent_rank_idx = get_rank(sent_graph)
sorted_sent_rank_idx = sorted(sent_rank_idx, key=lambda k : sent_rank_idx[k], reverse=True)
summarize_no()

상업시설은 엘 시티 지상 1~3 층에 8만 3790㎡ 규모로 조성되며, 개별 분양으로 진행된다.
‘ 엘시 티 더 몰 (the MALL)’ 은 해운대 방문객 수요와 함께 엘 시티 내 마련되는 ‘ 해 운대
분양관계자는 “ 사전에 분양된 엘 시티 내 주거시설이 모두 완 판을 기록하며, 이번에 개별 분양되는 엘 시티 더 몰 (the MALL)에도 투자자들의 관심이 끊이질 않고 있다 ”며 “ 차 후 상업시설이 개장하면 엘 시티 내 여러 부대시설과 함께 해운대를 넘어 부산을 대표하는 랜드 마크 시설로 자리잡을 것” 이라고 전했다.


In [229]:
summarize_no()

상업시설은 엘 시티 지상 1~3 층에 8만 3790㎡ 규모로 조성되며, 개별 분양으로 진행된다.
‘ 엘시 티 더 몰 (the MALL)’ 은 해운대 방문객 수요와 함께 엘 시티 내 마련되는 ‘ 해 운대
분양관계자는 “ 사전에 분양된 엘 시티 내 주거시설이 모두 완 판을 기록하며, 이번에 개별 분양되는 엘 시티 더 몰 (the MALL)에도 투자자들의 관심이 끊이질 않고 있다 ”며 “ 차 후 상업시설이 개장하면 엘 시티 내 여러 부대시설과 함께 해운대를 넘어 부산을 대표하는 랜드 마크 시설로 자리잡을 것” 이라고 전했다.


In [273]:
df

,0,1
0,"[101, 100, 113, 153, 11565, 11779, 114, 8843, ...","[101, 100, 9562, 14040, 45725, 9074, 118937, 1..."
1,"[101, 164, 9638, 28911, 18392, 12692, 8917, 22...","[101, 164, 9638, 28911, 18392, 12692, 8917, 22..."
2,"[101, 9706, 33305, 46079, 8909, 26444, 25242, ...","[101, 100, 10021, 14523, 9425, 49515, 12605, 2..."
3,"[101, 9644, 14423, 12030, 8900, 12692, 12638, ...","[101, 9644, 14423, 12030, 8900, 12692, 12638, ..."
4,"[101, 164, 48253, 31720, 17730, 166, 15361, 88...","[101, 164, 48253, 31720, 17730, 166, 15361, 88..."
...,...,...
694,"[101, 164, 9973, 118865, 15001, 31720, 17730, ...","[101, 12314, 24982, 9752, 119230, 27506, 10892..."
695,"[101, 164, 9638, 28911, 18392, 12692, 9638, 18...","[101, 164, 9638, 28911, 18392, 12692, 9638, 18..."
696,"[101, 43494, 8935, 9612, 63671, 9487, 10954, 3...","[101, 8935, 9612, 63671, 100, 8928, 119184, 21..."
697,"[101, 164, 9519, 46861, 31720, 17730, 8935, 42...","[101, 164, 9519, 46861, 31720, 17730, 8935, 42..."


In [219]:
sent_rank_idx = get_rank(sent_graph)
sorted_sent_rank_idx = sorted(sent_rank_idx, key=lambda k : sent_rank_idx[k], reverse=True)

NameError: name 'sent_graph' is not defined

In [279]:
article_df['contents']

0         ㈜엘시티피에프브이(PFV)가 개별분양에 돌입한 상업시설 ‘엘시티 더몰(the MAL...
1         시티건설은 경기도 화성시 남양뉴타운 B-14BL에 공급하는 '화성 남양 시티프라디움...
2         [이데일리 권소현 기자] 올해 글로벌 리츠 변동성이 확대될 것으로 예상되는 가운데 ...
3            지난 17일 국토교통부가 부산 9곳, 대구 7곳, 광주 5곳, 울산 2곳, 파...
4            임대인 관리와 안정적이고 꾸준한 수익을 기대할 수 있는 지식산업센터 투자에 투...
                                ...                        
323112    내년부터 상속세를 미술품이나 문화재로 대신 납부할 수 있는 길이 열린다. 또 상속세...
323113    난소암 수술 도중 암 병변을 정확하게 포착하는 데 도움을 주는 새로운 조영치료제가 ...
323114      [파이낸셜뉴스] 한국타이어가 미래 모빌리티 핵심 분야인 자율주행 부품 기술을 보...
323115    '대한민국 소비자만족지수 1위'는 국내 소비자들의 만족도가 높고 미래성장 가능성을 ...
323116    [경향신문] 굴이나 조개 껍데기(패각)는 국내에서 연간 30만~35만t 정도 발생하...
Name: contents, Length: 718681, dtype: object

In [218]:
summarize_no()

NameError: name 'sorted_sent_rank_idx' is not defined

In [54]:
article_df = pd.DataFrame(columns=['date','title','contents','press','good','well','sad','mad','want'])
scan_day = pd.datetime(2021,1,10)
page_count = True
page = 1

for a in range(0, 10) : 

    search_day = str(scan_day.year) + str(scan_day.month).zfill(2) + str(scan_day.day).zfill(2)
    print(search_day)
    while page_count :
        main_url = 'https://news.naver.com/main/list.naver?mode=LS2D&sid2=259&page2=&sid1=101&mid=shm&date={}&page={}'.format(search_day, page)
        headers = {'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36'}
        main_res = requests.get(main_url, headers = headers)
        main_soup = BeautifulSoup(main_res.text, 'html.parser')
        path = '#main_content > div.list_body.newsflash_body > ul > li'
        page_path1 = '#main_content > div.paging > a'
        page_path2 = '#main_content > div.paging > strong'
        main_select = main_soup.select(path)
        page_select = main_soup.select(page_path1)
        present = main_soup.select(page_path2)
        if page_select[-1].text != '다음' :
            if 20 != len(main_select) :
                page_count = False
            elif int(present[0].text) == int(page_select[-1].text)+1  :
                page_count = False

        page += 1            
        print(main_url)
        try : 
            for i in range(0, len(main_select)) :
                news_cont_url = main_select[i].select('a')[0]['href']

                cont_res = requests.get(news_cont_url, headers = headers)
                cont_soup = BeautifulSoup(cont_res.text, 'html.parser')
                cont_select_head = cont_soup.select('#main_content > div.article_header')
                cont_select_body = cont_soup.select('#articleBodyContents')

                target_text_without_child_tags = [bs_object for bs_object in cont_select_body[0]
                    if isinstance(bs_object, NavigableString)]
                text = "".join(target_text_without_child_tags[8:-3])

                date = cont_select_head[0].select('span.t11')[0].text
                title = cont_select_head[0].select('div.article_info #articleTitle')[0].text
                contents = text
                press = cont_select_head[0].select('a')[0]['href']
                good = cont_select_head[0].select('span.u_likeit_list_count._count')[0].text
                well = cont_select_head[0].select('span.u_likeit_list_count._count')[1].text
                sad = cont_select_head[0].select('span.u_likeit_list_count._count')[2].text
                mad = cont_select_head[0].select('span.u_likeit_list_count._count')[3].text
                want = cont_select_head[0].select('span.u_likeit_list_count._count')[4].text

                data = {'date':date, 'title':title, 'contents':contents, 'press':press, 'good':good,
                       'well':well, 'sad':sad, 'mad':mad, 'want':want}
                article_df = article_df.append(data, ignore_index=True)
        except :
            continue

    scan_day = scan_day - pd.Timedelta("1day")
    page_count = True
    page = 1

# article_df.to_csv('article_info.csv', encoding='utf-8-sig')

<ipython-input-54-2f844ebf4c0d>:2: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  scan_day = pd.datetime(2021,1,10)


20210110
https://news.naver.com/main/list.naver?mode=LS2D&sid2=259&page2=&sid1=101&mid=shm&date=20210110&page=1
https://news.naver.com/main/list.naver?mode=LS2D&sid2=259&page2=&sid1=101&mid=shm&date=20210110&page=2
https://news.naver.com/main/list.naver?mode=LS2D&sid2=259&page2=&sid1=101&mid=shm&date=20210110&page=3
https://news.naver.com/main/list.naver?mode=LS2D&sid2=259&page2=&sid1=101&mid=shm&date=20210110&page=4
https://news.naver.com/main/list.naver?mode=LS2D&sid2=259&page2=&sid1=101&mid=shm&date=20210110&page=5
https://news.naver.com/main/list.naver?mode=LS2D&sid2=259&page2=&sid1=101&mid=shm&date=20210110&page=6
https://news.naver.com/main/list.naver?mode=LS2D&sid2=259&page2=&sid1=101&mid=shm&date=20210110&page=7
20210109
https://news.naver.com/main/list.naver?mode=LS2D&sid2=259&page2=&sid1=101&mid=shm&date=20210109&page=1
https://news.naver.com/main/list.naver?mode=LS2D&sid2=259&page2=&sid1=101&mid=shm&date=20210109&page=2
20210108
https://news.naver.com/main/list.naver?mode=LS

https://news.naver.com/main/list.naver?mode=LS2D&sid2=259&page2=&sid1=101&mid=shm&date=20210105&page=9
https://news.naver.com/main/list.naver?mode=LS2D&sid2=259&page2=&sid1=101&mid=shm&date=20210105&page=10
https://news.naver.com/main/list.naver?mode=LS2D&sid2=259&page2=&sid1=101&mid=shm&date=20210105&page=11
https://news.naver.com/main/list.naver?mode=LS2D&sid2=259&page2=&sid1=101&mid=shm&date=20210105&page=12
https://news.naver.com/main/list.naver?mode=LS2D&sid2=259&page2=&sid1=101&mid=shm&date=20210105&page=13
https://news.naver.com/main/list.naver?mode=LS2D&sid2=259&page2=&sid1=101&mid=shm&date=20210105&page=14
https://news.naver.com/main/list.naver?mode=LS2D&sid2=259&page2=&sid1=101&mid=shm&date=20210105&page=15
https://news.naver.com/main/list.naver?mode=LS2D&sid2=259&page2=&sid1=101&mid=shm&date=20210105&page=16
https://news.naver.com/main/list.naver?mode=LS2D&sid2=259&page2=&sid1=101&mid=shm&date=20210105&page=17
https://news.naver.com/main/list.naver?mode=LS2D&sid2=259&page2=&

In [2]:
df = pd.read_csv('2021news_R1.csv', index_col=0)

test_df = df.sample(n=1000, random_state = 2)
test_df

test_df=df[:5]

In [5]:
summa = []
for contents in df['contents'] :
    try : 
        sentence = kkma.sentences(contents)        
        if len(sentence) > 10 :
            summary = ''
            for row in summarize(contents).split('\n') :
                summary += row
            if summary.strip() == '' :
                summa.append(np.nan)
            else : 
                summa.append(summary)
        else :
            summa.append(np.nan)
    except :
        summa.append(np.nan)

df['summary'] = summa
df

,date,title,contents,press,good,well,sad,mad,want,summary
0,2021.01.01. 오전 10:00,해운대 랜드마크 상업시설 ‘엘시티 더몰(the MALL)’ 분양돌입… 조기 완판 기대,㈜엘시티피에프브이(PFV)가 개별분양에 돌입한 상업시설 ‘엘시티 더몰(the MAL...,https://www.asiae.co.kr/,0,0,0,0,0,‘엘시티 더몰(the MALL)’은 해운대 방문객 수요와 함께 엘시티 내 마련되는 ...
1,2021.01.01. 오전 10:00,"합리적 가격 갖춘, 시티건설 `화성 남양 시티프라디움 4차`, 1월 4일부터 정당계...",시티건설은 경기도 화성시 남양뉴타운 B-14BL에 공급하는 '화성 남양 시티프라디움...,http://www.dt.co.kr/,0,0,0,0,0,NaN
2,2021.01.01. 오전 10:00,재산세 일부 임차인이 부담…넷리스 리츠 사볼까,[이데일리 권소현 기자] 올해 글로벌 리츠 변동성이 확대될 것으로 예상되는 가운데 ...,http://www.edaily.co.kr/,0,0,0,0,0,[이데일리 권소현 기자] 올해 글로벌 리츠 변동성이 확대될 것으로 예상되는 가운데 ...
3,2021.01.01. 오전 10:01,조정대상지역 제외된 포항 ‘흥해 서희스타힐스 더 캐슬’ 오는 6일(수) 1순위 청...,"지난 17일 국토교통부가 부산 9곳, 대구 7곳, 광주 5곳, 울산 2곳, 파...",http://www.fnnews.com/,0,0,0,0,0,‘흥해 서희스타힐스 더 캐슬’은 포항시 북구 흥해읍 남성리 남옥지구 B1블록에 지하...
4,2021.01.01. 오전 10:01,지식산업센터 임대 관리 편해 투자자 사이에서 인기 높아… 장기 임대에 임대료 안 밀린다,임대인 관리와 안정적이고 꾸준한 수익을 기대할 수 있는 지식산업센터 투자에 투...,http://www.fnnews.com/,0,0,0,0,0,임대인 관리와 안정적이고 꾸준한 수익을 기대할 수 있는 지식산업센터 투자에 투자자가...
...,...,...,...,...,...,...,...,...,...,...
323112,2021.11.30. 오후 9:31,내년부터 문화재·미술품 상속세 물납 허용된다…최대 10년 분할 납부,내년부터 상속세를 미술품이나 문화재로 대신 납부할 수 있는 길이 열린다. 또 상속세...,http://www.hankookilbo.com/,0,0,0,0,0,또 상속세를 최대 10년에 걸쳐 나눠 낼 수 있게 된다.이에 따라 역사적·학술적·예...
323113,2021.11.30. 오후 9:32,"난소암 위치 정확히 포착…새 조영치료제, FDA 승인",난소암 수술 도중 암 병변을 정확하게 포착하는 데 도움을 주는 새로운 조영치료제가 ...,https://www.kormedi.com/,0,0,0,0,0,난소암 수술 도중 암 병변을 정확하게 포착하는 데 도움을 주는 새로운 조영치료제가 ...
323114,2021.11.30. 오후 9:33,"한국타이어, 캐나다 프리사이슬리 인수한다",[파이낸셜뉴스] 한국타이어가 미래 모빌리티 핵심 분야인 자율주행 부품 기술을 보...,http://www.fnnews.com/,0,0,0,0,0,NaN
323115,2021.11.30. 오후 9:33,"비츄인, 2021 대한민국 소비자만족지수1위 손소독제 부문 2년 연속 수상",'대한민국 소비자만족지수 1위'는 국내 소비자들의 만족도가 높고 미래성장 가능성을 ...,http://www.mt.co.kr/,0,0,0,0,0,NaN


In [4]:
article_df = pd.read_csv('data_last2.csv',index_col=0)
article_df.drop(['press', 'good','well','sad','mad','want'], axis=1)
article_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 718681 entries, 0 to 323116
Data columns (total 14 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   date             718681 non-null  object 
 1   title            718681 non-null  object 
 2   contents         718681 non-null  object 
 3   press            718681 non-null  object 
 4   good             718681 non-null  int64  
 5   well             718681 non-null  int64  
 6   sad              718681 non-null  int64  
 7   mad              718681 non-null  int64  
 8   want             718681 non-null  int64  
 9   summary          362494 non-null  object 
 10  LDA_topic_value  718681 non-null  object 
 11  cluster_LDA      718681 non-null  float64
 12  LDA_topic_score  718681 non-null  float64
 13  sentence_len     717893 non-null  float64
dtypes: float64(3), int64(5), object(6)
memory usage: 82.2+ MB


In [7]:
test_df = article_df.drop(['press', 'good','well','sad','mad','want'], axis=1)

In [6]:
tfidf = TfidfVectorizer()

In [8]:
test_df = article_df[:500]
test_df.dropna(inplace=True)
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 374 entries, 0 to 497
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   date      374 non-null    object
 1   title     374 non-null    object
 2   contents  374 non-null    object
 3   press     374 non-null    object
 4   good      374 non-null    int64 
 5   well      374 non-null    int64 
 6   sad       374 non-null    int64 
 7   mad       374 non-null    int64 
 8   want      374 non-null    int64 
 9   summary   374 non-null    object
dtypes: int64(5), object(5)
memory usage: 32.1+ KB


<ipython-input-8-957b8c57bac3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df.dropna(inplace=True)


In [63]:
x_graph_list = []
y_graph_list = []

for x, y in zip(test_df['contents'], test_df['summary']) :
    x_mat = tfidf.fit_transform(kkma.sentences(x)).toarray()
    x_graph = np.dot(x_mat, x_mat.T)
    x_graph_list.append(x_graph)

    y_mat = tfidf.fit_transform(kkma.sentences(y)).toarray()
    y_graph = np.dot(y_mat, y_mat.T)
    y_graph_list.append(y_graph)

In [43]:
y_graph_list

[array([[1.        , 0.07285002, 0.10881427, 0.14710059, 0.12488226],
        [0.07285002, 1.        , 0.09310181, 0.03313896, 0.07522582],
        [0.10881427, 0.09310181, 1.        , 0.11262907, 0.10896312],
        [0.14710059, 0.03313896, 0.11262907, 1.        , 0.03787203],
        [0.12488226, 0.07522582, 0.10896312, 0.03787203, 1.        ]]),
 array([[1.        , 0.02458508, 0.04628014, 0.        ],
        [0.02458508, 1.        , 0.09268131, 0.        ],
        [0.04628014, 0.09268131, 1.        , 0.        ],
        [0.        , 0.        , 0.        , 1.        ]]),
 array([[1.        , 0.13507446, 0.14059482, 0.072794  , 0.09154838],
        [0.13507446, 1.        , 0.12777462, 0.01409795, 0.11938551],
        [0.14059482, 0.12777462, 1.        , 0.01022649, 0.08660082],
        [0.072794  , 0.01409795, 0.01022649, 1.        , 0.00955506],
        [0.09154838, 0.11938551, 0.08660082, 0.00955506, 1.        ]]),
 array([[1.        , 0.05353736, 0.0935424 , 0.        , 0.027

In [44]:
a = pd.DataFrame()

In [45]:
a['x'] = x_graph_list
a['y'] = y_graph_list

In [31]:
kkma.sentences(test_df['summary'][0])

['‘ 엘시 티 더 몰 (the MALL)’ 은 해운대 방문객 수요와 함께 엘 시티 내 마련되는 ‘ 해 운대',
 '엘시 티 더 샵 (882 가구)’, ‘ 엘시 티 레지 던스 (561 실)’ 와 ‘ 시 그니 엘 부산 (260 실)’ 등을 고정 수요로 누릴 예정이며, 가까이에 위치한 마린 시티 약 51,500 세대의 배후 수요도 품을 것으로 보인다.',
 '해 운 대의 ‘ 엘시 티 더 몰 (the MALL)’ 역시 부산 초고층 마천루 엘 시티 내에 조성되며, 개 장과 동시에 지역경제 활성화를 리딩하는 랜드 마크 상업시설의 역할을 수행할 것으로 보여 진다.',
 '분양관계자는 “ 사전에 분양된 엘 시티 내 주거시설이 모두 완 판을 기록하며, 이번에 개별 분양되는 엘 시티 더 몰 (the MALL)에도 투자자들의 관심이 끊이질 않고 있다 ”며 “ 차 후 상업시설이 개장하면 엘 시티 내 여러 부대시설과 함께 해운대를 넘어 부산을 대표하는 랜드 마크 시설로 자리잡을 것” 이라고 전했다.',
 '한편, 프리미엄 상업시설 ‘ 엘시 티 더 몰 (the MALL)’ 의 홍보관은 부산 광역시 해운대구에 위치한다.']

In [48]:
model = Word2Vec(sentences=test_df['summary'][0], size=100, window=5, min_count=5, workers=4, sg=0)

C:\anaconda3\lib\site-packages\gensim\models\base_any2vec.py:322: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  warnings.warn(


In [64]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
tokenized_data = []
for sentence in test_df['contents']:
    tokenized_sentence = kkma.morphs(sentence) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    tokenized_data.append(stopwords_removed_sentence)

In [55]:
tokenized_data

[['㈜',
  '엘시',
  '티',
  '피',
  '에프',
  '브이',
  '(',
  'PFV',
  ')',
  '아',
  '개별',
  '분양',
  '돌입',
  '하',
  'ㄴ',
  '상업',
  '시설',
  '‘',
  '엘시',
  '티',
  '더',
  '몰',
  'ㄹ',
  '(',
  'the',
  'MALL',
  ')',
  '’',
  '분양',
  '순풍',
  '불',
  '고',
  '있',
  '다',
  '.',
  '지나',
  'ㄴ',
  '21',
  '일',
  '부터',
  '24',
  '일',
  '까지',
  '공개',
  '청약',
  '을',
  '진행',
  '하',
  'ㄴ',
  '부산',
  '해운대',
  '초대형',
  '상업',
  '시설',
  '‘',
  '엘시',
  '티',
  '더',
  '몰',
  'ㄹ',
  '(',
  'the',
  'MALL',
  ')',
  '’',
  '경우',
  '최고',
  '청약',
  '경쟁률',
  '15',
  '대',
  '1',
  '기록',
  '하',
  '등',
  '분양',
  '선전',
  '하',
  '고',
  '있',
  '어',
  '주목',
  '을',
  '받',
  '고',
  '있',
  '다',
  '.',
  '특히',
  '해운대',
  '해수욕장',
  '을',
  '조망',
  '하',
  'ㄹ',
  '수',
  '있',
  '거나',
  '출입구',
  '주변',
  '위치',
  '하',
  'ㄴ',
  '인기',
  '점포',
  '경우',
  '대부분',
  '분양',
  '완료',
  '되',
  '었',
  '다',
  '.',
  '상업',
  '시설',
  '엘',
  '시티',
  '지상',
  '1',
  '~',
  '3',
  '층',
  '8',
  '만',
  '3790',
  '㎡',
  '규모',
  '로',
  '조성',
  '되',
  '며',
  ',',

In [84]:
kkma.pos(test_df['contents'][0])

[('㈜', 'SW'),
 ('엘시', 'NNG'),
 ('티', 'NNG'),
 ('피', 'NNG'),
 ('에프', 'NNG'),
 ('브이', 'NNG'),
 ('(', 'SS'),
 ('PFV', 'OL'),
 (')', 'SS'),
 ('가', 'VV'),
 ('아', 'ECS'),
 ('개별', 'NNG'),
 ('분양', 'NNG'),
 ('에', 'JKM'),
 ('돌입', 'NNG'),
 ('하', 'XSV'),
 ('ㄴ', 'ETD'),
 ('상업', 'NNG'),
 ('시설', 'NNG'),
 ('‘', 'SS'),
 ('엘시', 'NNG'),
 ('티', 'NNG'),
 ('더', 'MAG'),
 ('몰', 'VV'),
 ('ㄹ', 'ETD'),
 ('(', 'SS'),
 ('the', 'OL'),
 ('MALL', 'OL'),
 (')', 'SS'),
 ('’', 'SS'),
 ('의', 'NNG'),
 ('분양', 'NNG'),
 ('에', 'JKM'),
 ('순풍', 'NNG'),
 ('이', 'JKS'),
 ('불', 'VV'),
 ('고', 'ECE'),
 ('있', 'VXV'),
 ('다', 'EFN'),
 ('.', 'SF'),
 ('지나', 'VV'),
 ('ㄴ', 'ETD'),
 ('21', 'NR'),
 ('일', 'NNM'),
 ('부터', 'JX'),
 ('24', 'NR'),
 ('일', 'NNM'),
 ('까지', 'JX'),
 ('공개', 'NNG'),
 ('청약', 'NNG'),
 ('을', 'JKO'),
 ('진행', 'NNG'),
 ('하', 'XSV'),
 ('ㄴ', 'ETD'),
 ('부산', 'NNG'),
 ('해운대', 'NNG'),
 ('의', 'JKG'),
 ('초대형', 'NNG'),
 ('상업', 'NNG'),
 ('시설', 'NNG'),
 ('‘', 'SS'),
 ('엘시', 'NNG'),
 ('티', 'NNG'),
 ('더', 'MAG'),
 ('몰', 'VV'),
 ('ㄹ', 'ETD'

In [98]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 718681 entries, 0 to 323116
Data columns (total 10 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   date      718681 non-null  object
 1   title     718681 non-null  object
 2   contents  718681 non-null  object
 3   press     718681 non-null  object
 4   good      718681 non-null  int64 
 5   well      718681 non-null  int64 
 6   sad       718681 non-null  int64 
 7   mad       718681 non-null  int64 
 8   want      718681 non-null  int64 
 9   summary   362494 non-null  object
dtypes: int64(5), object(5)
memory usage: 60.3+ MB


In [108]:
test_df = df[:10]
test_df.dropna(inplace=True)
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8 entries, 0 to 8
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   date      8 non-null      object
 1   title     8 non-null      object
 2   contents  8 non-null      object
 3   press     8 non-null      object
 4   good      8 non-null      int64 
 5   well      8 non-null      int64 
 6   sad       8 non-null      int64 
 7   mad       8 non-null      int64 
 8   want      8 non-null      int64 
 9   summary   8 non-null      object
dtypes: int64(5), object(5)
memory usage: 704.0+ bytes


<ipython-input-108-007608ba1a62>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df.dropna(inplace=True)


In [122]:
def vect_tfidf(docs) :
    doc_vect_list = []
    
    for doc in docs :
        doc_pattern = text_clean(doc)
        doc_sent = kkma.sentences(doc_pattern)
        doc_mat = tfidf.fit_transform(doc_sent).toarray()
        doc_graph = np.dot(doc_mat.T,doc_mat)
        doc_graph = np.resize(doc_graph,(500,500))
        doc_vect_list.append(doc_graph)
        
    return doc_vect_list

In [189]:
doc_mat.shape

(2, 377)

In [123]:
x = vect_tfidf(test_df['contents'][:10])
y = vect_tfidf(test_df['summary'][:10])

In [165]:
from gensim.models import word2vec

C:\anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [187]:
doc_graph.shape

(377, 377)

In [158]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.models import *
from tensorflow.keras.callbacks import *

In [164]:
model = Sequential([
    Conv2D(32,3, activation='softmax', input_shape=(10,10,)),
    MaxPooling2D(2,2),
    Conv2D(10,3, activation='softmax'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(32, activation='softmax')
])
# model.add(LSTM(128))
# # model.add(LSTM(128, input_shape=(5,1,)))
# model.add(Dense(10))
# model.add(Dense(1, activation='softmax'))  # 연속값 데이터 - 회귀분석
# model.compile(loss='mse', optimizer='adam')

ValueError: Input 0 of layer conv2d_2 is incompatible with the layer: : expected min_ndim=4, found ndim=3. Full shape received: [None, 10, 10]

In [154]:
x[0].shape

(500, 500)

In [156]:
model.fit(x, y, epochs=10, batch_size=10)

Epoch 1/10


ValueError: in user code:

    C:\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    C:\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:789 run_step  **
        outputs = model.train_step(data)
    C:\anaconda3\lib\site-packag

In [176]:
kkma.pos(text_clean(test_df['summary'][0]))

[('엘', 'NNG'),
 ('시티', 'NNG'),
 ('더', 'MAG'),
 ('몰', 'VV'),
 ('ㄹ', 'ETD'),
 ('theMALL', 'OL'),
 ('은', 'JX'),
 ('해운대', 'NNG'),
 ('방문객', 'NNG'),
 ('수요', 'NNG'),
 ('와', 'JKM'),
 ('함께', 'MAG'),
 ('엘', 'NNG'),
 ('시티', 'NNG'),
 ('내', 'NNG'),
 ('마련', 'NNG'),
 ('되', 'XSV'),
 ('는', 'ETD'),
 ('해운대', 'NNG'),
 ('엘', 'NNG'),
 ('시티', 'NNG'),
 ('더', 'MAG'),
 ('샵', 'NNG'),
 ('가구', 'NNG'),
 ('엘', 'NNG'),
 ('시티', 'NNG'),
 ('레지', 'NNG'),
 ('던', 'NNP'),
 ('슬', 'VV'),
 ('시', 'EPH'),
 ('ㄹ', 'ETD'),
 ('오', 'VV'),
 ('아', 'ECS'),
 ('시', 'NNG'),
 ('그', 'VV'),
 ('니', 'ECD'),
 ('엘', 'NNG'),
 ('부산', 'NNG'),
 ('실', 'NNG'),
 ('등', 'NNB'),
 ('을', 'JKO'),
 ('고정', 'NNG'),
 ('수요', 'NNG'),
 ('로', 'JKM'),
 ('누리', 'VV'),
 ('ㄹ', 'ETD'),
 ('예정', 'NNG'),
 ('이', 'VCP'),
 ('며', 'ECE'),
 ('가까이', 'NNG'),
 ('에', 'JKM'),
 ('위치', 'NNG'),
 ('하', 'XSV'),
 ('ㄴ', 'ETD'),
 ('마린', 'NNG'),
 ('시티', 'NNG'),
 ('약', 'NNG'),
 ('세대', 'NNG'),
 ('의', 'JKG'),
 ('배후', 'NNG'),
 ('수요', 'NNG'),
 ('도', 'JX'),
 ('품', 'VV'),
 ('을', 'ETD'),
 ('것', 'NNB'),


In [178]:
from nltk.tokenize import word_tokenize
from nltk.tokenize import WordPunctTokenizer
from tensorflow.keras.preprocessing.text import text_to_word_sequence

In [183]:
from transformers import BertTokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [180]:
tokenizer = BertWordPieceTokenizer(
    vocab_file=None,
    clean_text=True,
    handle_chinese_chars=True,
    strip_accents=False, # Must be False if cased model
    lowercase=False,
    wordpieces_prefix="##"
)

TypeError: __init__() got an unexpected keyword argument 'vocab_file'

In [ ]:
max_word = 3000
max_len = 300
tok = word_tokenize()

In [177]:
Tokenizer()

NameError: name 'Tokenizer' is not defined

In [214]:
doc_vect_list = []
for doc in test_df['contents'][:3] :
#     doc_pattern = text_clean(doc)
    doc_sent = kkma.morphs(doc)
    doc_mat = tfidf.fit_transform(doc_sent).toarray()
    doc_mat = pad_sequences(doc_mat, padding='post', dtype='float')
    
    doc_graph = np.dot(doc_mat.T,doc_mat)
    print(doc_mat.shape)
    doc_vect_list.append(doc_graph)

(1105, 260)
(612, 186)
(1090, 320)


In [210]:
doc_vect_list

[array([[0.05212095, 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.02931968, 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.02636085, ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ]]),
 array([[0.03752841, 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.04023937, 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.10962073, ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0. 

In [206]:
# doc_pattern = text_clean(test_df['contents'][0])
doc_sent = kkma.sentences(test_df['contents'][0])
doc_sent

['㈜ 엘시 티피에프 브이 (PFV) 가 개별 분양에 돌입한 상업시설 ‘ 엘시 티 더 몰 (the MALL)’ 의 분양에 순풍이 불고 있다.',
 '지난 21일부터 24일까지 공개 청약을 진행한 부산 해운대의 초대형 상업시설 ‘ 엘시 티 더 몰 (the MALL)’ 의 경우 최고 청약 경쟁률이 15대 1를 기록하는 등 분양에 선전하고 있어 주목을 받고 있다.',
 '특히 해운대 해수욕장을 조망할 수 있거나 출입구 주변에 위치한 인기 점포의 경우 대부분 분양이 완료됐다.',
 '상업시설은 엘 시티 지상 1~3 층에 8만 3790㎡ 규모로 조성되며, 개별 분양으로 진행된다.',
 '청약 전 일주일간 분양의 향서를 접수한 결과 700 여 명이 몰리면서 조기 완 판에 대한 기대감을 높이기도 했다.',
 '‘ 엘시 티 더 몰 (the MALL)’ 은 부산 광역시 해운대구 일원에 지상 1 층~ 지상 3 층, 연면적 8만 3790㎡ 규모로 조성되는 상업시설로, 국내에서 두 번째로 높은 초고층 주상 복합단지 엘 시티에 들어서 일대 랜드 마크로 자리잡을 것으로 보여 진다.',
 '상업시설은 해운대 해수욕장과 인접해 있으며, 이러한 강점을 적극 활용한 매장 구성과 설계를 선보인다.',
 '먼저, 해운대를 바라보는 상업시설 전면 부에는 오션 뷰가 가능한 통창이 적용돼 해운대 방문객들의 접근성을 높였다.',
 '1 층에는 오션 뷰 데크를, 2~3 층에 위치한 점포의 경우에는 오션 뷰 테라스를 적용해 해운대 오션 뷰를 극대화했다.',
 '또 한, 상업시설 앞에 위치한 광장 및 해수욕장과의 연결을 고려한 출입구 배치로 일대 유동인구를 흡수할 수 있을 것으로 보인다.',
 '상업시설 내에는 층별 색다른 경험을 위한 리딩 테넌트 유치가 예정돼 있음은 물론 집 객력이 큰 앵커 테넌트들도 계획돼 있다.',
 '현재 엘 시티 98 층 ~101 층에는 부산 최고층 전망대 ‘ 부산 엑스 더 스카이 (BUSAN X the SKY)’ 가 운영 중이며, 4 층 ~6 층에는 국내 최대 규모의 인 피니 티

In [202]:
pad_sequences(doc_mat,maxlen=500, padding='post', dtype='float').shape

(2, 500)

In [201]:
doc_mat.shape

(2, 377)

In [95]:
pad_sequences(doc_vect_list[2], padding='post', dtype='float').shape

(377, 377)

In [116]:
print(doc_vect_list[0].shape)
print(doc_vect_list[1].shape)
print(doc_vect_list[2].shape)

(331, 331)
(228, 228)
(377, 377)


In [121]:
doc_vect_list[0] = np.resize(doc_vect_list[0],(500,500))
doc_vect_list[1] = np.resize(doc_vect_list[1],(500,500))
doc_vect_list[2] = np.resize(doc_vect_list[2],(500,500))

(500, 500)

In [58]:
doc_pattern = text_clean(test_df['contents'][2])
doc_sent = kkma.sentences(doc_pattern)
doc_mat = tfidf.fit_transform(doc_sent).toarray()
doc_mat.shape


(3, 228)

In [ ]:
doc_pattern = text_clean(test_df['contents'][2])
doc_sent = kkma.sentences(doc_pattern)
doc_mat = tfidf.fit_transform(doc_sent).toarray()
doc_mat.shape

In [57]:
test_df['contents']

0      ㈜엘시티피에프브이(PFV)가 개별분양에 돌입한 상업시설 ‘엘시티 더몰(the MAL...
2      [이데일리 권소현 기자] 올해 글로벌 리츠 변동성이 확대될 것으로 예상되는 가운데 ...
3         지난 17일 국토교통부가 부산 9곳, 대구 7곳, 광주 5곳, 울산 2곳, 파...
4         임대인 관리와 안정적이고 꾸준한 수익을 기대할 수 있는 지식산업센터 투자에 투...
5      [서울경제] 3월 공매도 재개에 따라 금융당국이 불법(무차입) 공매도에 대한 처벌과...
                             ...                        
492    [아시아경제 이춘희 기자] 국토교통부와 한국토지주택공사(LH)는 에너지와 생태환경이...
493    [이데일리 김나리 기자] 3기 신도시 등 사전청약이 올해 7월 시작된다. 인천계양을...
494    [이데일리 김나리 기자] 국토교통부, 교육부, 중소벤처기업부는 지난해 12월 31일...
496    [헤럴드경제=이현정 기자]유진투자증권은 3일 SK바이오팜의 목표주가를 11만원에서 ...
497    은퇴한 이들에게 상장지수펀드(ETF)는 분산된 주식 포트폴리오를 통해 인컴 소득을 ...
Name: contents, Length: 374, dtype: object

In [10]:
x = vect_tfidf(test_df['contents'][:10])
y = vect_tfidf(test_df['summary'][:10])

In [18]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [21]:
a = pd.DataFrame()
a['x'] = x
a['y'] = y
a

,x,y
0,"[[0.030671383606587218, 0.030671383606587218, ...","[[0.07053456158585983, 0.07053456158585983, 0...."
1,"[[0.0, 0.07074690146658279, 0.1076096841975704...","[[0.11322770341445956, 0.11322770341445956, 0...."
2,"[[0.048059385566625194, 0.048059385566625194, ...","[[0.08362420100070908, 0.08362420100070908, 0...."
3,"[[0.0, 0.06167013790654738, 0.0486214450764837...","[[0.09901475429766744, 0.09901475429766744, 0...."
4,"[[0.0, 0.10397104766472066, 0.0, 0.07306383987...","[[0.3692744729379982, 0.12309149097933272, 0.1..."
...,...,...
369,"[[0.10206207261596582, 0.0, 0.0510310363079829...","[[0.0, 0.11867816581938533, 0.1186781658193853..."
370,"[[0.0, 0.06923368579695134, 0.0692336857969513...","[[0.1414213562373095, 0.282842712474619, 0.141..."
371,"[[0.05198752449100364, 0.05198752449100364, 0....","[[0.1849000654084097, 0.09245003270420485, 0.0..."
372,"[[0.0, 0.0, 0.13182544500430626, 0.10326501669...","[[0.0, 0.0, 0.0, 0.15811388300841894, 0.158113..."


In [44]:
a['x'][1]

array([[0.        , 0.0707469 , 0.10760968, 0.        , 0.        ,
        0.2122407 , 0.        , 0.0707469 , 0.        , 0.1414938 ,
        0.0707469 , 0.05380484, 0.0707469 , 0.0707469 , 0.0707469 ,
        0.0707469 , 0.0707469 , 0.        , 0.0707469 , 0.0707469 ,
        0.        , 0.0707469 , 0.0707469 , 0.28298761, 0.0707469 ,
        0.0707469 , 0.0707469 , 0.0707469 , 0.        , 0.0707469 ,
        0.0707469 , 0.        , 0.0707469 , 0.        , 0.        ,
        0.05380484, 0.0707469 , 0.        , 0.0707469 , 0.        ,
        0.        , 0.0707469 , 0.0707469 , 0.        , 0.0707469 ,
        0.        , 0.0707469 , 0.        , 0.1414938 , 0.0707469 ,
        0.0707469 , 0.        , 0.        , 0.        , 0.0707469 ,
        0.        , 0.        , 0.0707469 , 0.04178426, 0.0707469 ,
        0.0707469 , 0.        , 0.0707469 , 0.0707469 , 0.0707469 ,
        0.0707469 , 0.0707469 , 0.1414938 , 0.0707469 , 0.0707469 ,
        0.05380484, 0.        , 0.        , 0.07

In [43]:
for i in a['x'] :
    print(len(i))
#     max(len(i[0]))

1
3
2
4
2
5
2
4
2
2
3
2
5
3
5
2
6
1
3
6
55
78
57
3
5
9
11
2
5
7
2
115
3
5
2
5
1
15
3
2
6
3
2
7
2
2
3
2
1
6
3
7
1
3
4
7
4
2
7
5
5
6
9
4
8
6
2
2
3
4
8
6
4
2
3
1
8
3
2
7
5
8
11
73
6
11
9
9
6
6
5
4
5
4
2
7
11
16
16
17
4
5
4
45
3
4
3
4
4
7
4
18
2
3
5
2
2
3
77
4
3
3
6
3
7
6
27
2
2
33
2
2
7
3
4
6
6
3
5
7
5
5
5
5
5
2
9
4
3
5
4
1
4
4
1
1
2
2
6
9
2
17
15
1
3
6
1
6
3
4
6
2
6
6
3
5
4
4
4
4
1
4
4
60
3
74
1
2
19
2
5
4
2
5
1
2
1
2
4
2
2
3
5
3
1
89
3
5
2
8
2
4
1
7
5
8
2
3
3
2
5
4
4
1
4
5
4
7
6
1
2
5
2
3
21
2
5
7
1
1
5
6
3
1
6
1
4
3
5
2
4
2
28
3
2
12
2
4
5
5
5
2
9
5
1
2
3
2
4
5
2
3
2
19
3
3
1
5
7
4
5
7
2
3
3
3
4
3
22
31
1
30
2
4
10
2
41
4
4
2
18
8
34
6
6
4
3
3
6
29
7
2
3
10
2
3
3
4
2
2
8
2
2
1
5
1
4
4
5
4
2
1
1
1
6
4
2
5
8
2
3
1
15
1
8
5
2
3
7
28
4
2
3
4
4
3
3
3
11
3
13
6
4
3
5
2
2
1
4
2
2
1
4
1


In [41]:
pad_sequences(a['x'], padding='post',dtype='float')

ValueError: Shape of sample (228,) of sequence at position 1 is different from expected shape (331,)

In [29]:
x[0][1]

IndexError: index 1 is out of bounds for axis 0 with size 1

In [249]:
len(x[4])

3

In [138]:
model = Sequential()
lstm.
rfc.
fit(x,y)
predict(x_test)
y_test

8 8


In [ ]:
y_test.vec2sent

In [167]:
x_vect_list[0]

array([[0.03016485, 0.03016485, 0.03016485, 0.03016485, 0.03016485,
        0.03016485, 0.03016485, 0.03016485, 0.03016485, 0.03016485,
        0.03016485, 0.03016485, 0.03016485, 0.03016485, 0.0603297 ,
        0.03016485, 0.03016485, 0.03016485, 0.03016485, 0.03016485,
        0.03016485, 0.0603297 , 0.03016485, 0.03016485, 0.03016485,
        0.03016485, 0.03016485, 0.0603297 , 0.09049455, 0.03016485,
        0.03016485, 0.03016485, 0.0603297 , 0.03016485, 0.03016485,
        0.03016485, 0.03016485, 0.09049455, 0.0603297 , 0.03016485,
        0.03016485, 0.03016485, 0.1206594 , 0.03016485, 0.03016485,
        0.0603297 , 0.03016485, 0.03016485, 0.03016485, 0.0603297 ,
        0.03016485, 0.03016485, 0.03016485, 0.03016485, 0.03016485,
        0.03016485, 0.03016485, 0.03016485, 0.0603297 , 0.03016485,
        0.0603297 , 0.03016485, 0.03016485, 0.03016485, 0.03016485,
        0.03016485, 0.03016485, 0.0603297 , 0.0603297 , 0.0603297 ,
        0.03016485, 0.03016485, 0.03016485, 0.03

In [140]:
!pip install transformers

In [157]:
!pip install torch

In [158]:
import torch

In [39]:
from transformers import BertTokenizer, BertModel
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [6]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

In [148]:
marked_text = "[CLS] " + test_df['contents'][0] + " [SEP]"

# Tokenize our sentence with the BERT tokenizer.
tokenized_text = tokenizer.tokenize(marked_text)

In [149]:
tokenized_text

['[CLS]',
 '[UNK]',
 '(',
 'P',
 '##F',
 '##V',
 ')',
 '가',
 '개',
 '##별',
 '##분',
 '##양',
 '##에',
 '돌',
 '##입',
 '##한',
 '상',
 '##업',
 '##시',
 '##설',
 '[UNK]',
 '엘',
 '##시',
 '##티',
 '더',
 '##몰',
 '(',
 'the',
 'MA',
 '##LL',
 ')',
 '[UNK]',
 '의',
 '분',
 '##양',
 '##에',
 '순',
 '##풍',
 '##이',
 '불',
 '##고',
 '있다',
 '.',
 '지',
 '##난',
 '21일',
 '##부터',
 '24일',
 '##까지',
 '공',
 '##개',
 '##청',
 '##약을',
 '진',
 '##행',
 '##한',
 '부',
 '##산',
 '해',
 '##운',
 '##대의',
 '초대',
 '##형',
 '상',
 '##업',
 '##시',
 '##설',
 '[UNK]',
 '엘',
 '##시',
 '##티',
 '더',
 '##몰',
 '(',
 'the',
 'MA',
 '##LL',
 ')',
 '[UNK]',
 '의',
 '경우',
 '최고',
 '청',
 '##약',
 '##경',
 '##쟁',
 '##률',
 '##이',
 '15',
 '##대',
 '1',
 '##를',
 '기',
 '##록',
 '##하는',
 '등',
 '분',
 '##양',
 '##에',
 '선',
 '##전',
 '##하고',
 '있어',
 '주',
 '##목',
 '##을',
 '받고',
 '있다',
 '.',
 '특히',
 '해',
 '##운',
 '##대',
 '##해',
 '##수',
 '##욕',
 '##장을',
 '조',
 '##망',
 '##할',
 '수',
 '있',
 '##거나',
 '출',
 '##입',
 '##구',
 '주',
 '##변',
 '##에',
 '위치한',
 '인',
 '##기',
 '점',
 '##포',
 '#

In [152]:
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

# Display the words with their indeces.
for tup in zip(tokenized_text, indexed_tokens):
    print('{:<12} {:>6,}'.format(tup[0], tup[1]))

[CLS]           101
[UNK]           100
(               113
P               153
##F          11,565
##V          11,779
)               114
가             8,843
개             8,857
##별          61,844
##분          37,712
##양          37,114
##에          10,530
돌             9,091
##입          58,303
##한          11,102
상             9,414
##업          26,784
##시          14,040
##설          31,928
[UNK]           100
엘             9,562
##시          14,040
##티          45,725
더             9,074
##몰          118,937
(               113
the          10,105
MA           27,277
##LL         82,834
)               114
[UNK]           100
의             9,637
분             9,367
##양          37,114
##에          10,530
순             9,462
##풍          119,409
##이          10,739
불             9,368
##고          11,664
있다           11,506
.               119
지             9,706
##난          33,305
21일          45,966
##부터         17,655
24일          47,251
##까지         18,382
공             8,89

In [155]:
segments_ids = [1] * len(tokenized_text)

print(segments_ids)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [170]:
k = tokenizer.encode(test_df['contents'][0])

tokenizer.decode(k)

'[CLS] [UNK] ( PFV ) 가 개별분양에 돌입한 상업시설 [UNK] 엘시티 더몰 ( the MALL ) [UNK] 의 분양에 순풍이 불고 있다. 지난 21일부터 24일까지 공개청약을 진행한 부산 해운대의 초대형 상업시설 [UNK] 엘시티 더몰 ( the MALL ) [UNK] 의 경우 최고 청약경쟁률이 15대 1를 기록하는 등 분양에 선전하고 있어 주목을 받고 있다. 특히 해운대해수욕장을 조망할 수 있거나 출입구 주변에 위치한 인기 점포의 경우 대부분 분양이 완료됐다. 상업시설은 엘시티 지상 1 ~ 3층에 [UNK] 규모로 조성되며, 개별 분양으로 진행된다. 청약 전 일주일간 분양의향서를 접수한 결과 700여명이 몰리면서 조기 완판에 대한 기대감을 높이기도 했다. [UNK] 엘시티 더몰 ( the MALL ) [UNK] 은 부산광역시 해운대구 일원에 지상 1층 ~ 지상 3층, 연면적 8만 [UNK] 규모로 조성되는 상업시설로, 국내에서 두번째로 높은 초고층 주상복합단지 엘시티에 들어서 일대 랜드마크로 자리잡을 것으로 보여진다. 상업시설은 해운대 해수욕장과 인접해 있으며, 이러한 강점을 적극 활용한 매장 구성과 설계를 선보인다. 먼저, 해운대를 바라보는 상업시설 전면부에는 오션뷰가 가능한 통창이 적용돼 해운대 방문객들의 접근성을 높였다. 1층에는 오션뷰 데크를, 2 ~ 3층에 위치한 점포의 경우에는 오션뷰 테라스를 적용해 해운대 오션뷰를 극대화했다. 또한, 상업시설 앞에 위치한 광장 및 해수욕장과의 연결을 고려한 출입구 배치로 일대 유동인구를 흡수할 수 있을 것으로 보인다. 상업시설 내에는 층별 색다른 경험을 위한 리딩테넌트 유치가 예정돼 있음은 물론 집객력이 큰 앵커테넌트들도 계획돼 있다. 현재 엘시티 98층 ~ 101층에는 부산 최고층 전망대 [UNK] 부산 엑스 더 스카이 ( BUSAN X the SKY ) [UNK] 가 운영 중이며, 4층 ~ 6층에는 국내 최대 규모의 인피니티풀과 온천수를 활용해 사계절 내내 이용 가능한 도심형 워터파크가 내년

In [78]:
test_df1 = test_df[:1000]
test_df1

,date,title,contents,summary,LDA_topic_value,cluster_LDA,LDA_topic_score,sentence_len
0,2021.01.01. 오전 10:00,해운대 랜드마크 상업시설 ‘엘시티 더몰(the MALL)’ 분양돌입… 조기 완판 기대,㈜엘시티피에프브이(PFV)가 개별분양에 돌입한 상업시설 ‘엘시티 더몰(the MAL...,‘엘시티 더몰(the MALL)’은 해운대 방문객 수요와 함께 엘시티 내 마련되는 ...,"[(0, 0.6916478), (1, 0.015565783), (6, 0.06934...",0.0,0.6916,24.0
1,2021.01.01. 오전 10:00,"합리적 가격 갖춘, 시티건설 `화성 남양 시티프라디움 4차`, 1월 4일부터 정당계...",시티건설은 경기도 화성시 남양뉴타운 B-14BL에 공급하는 '화성 남양 시티프라디움...,NaN,"[(0, 0.9159487), (1, 0.01692329), (4, 0.022365...",0.0,0.9160,18.0
2,2021.01.01. 오전 10:00,재산세 일부 임차인이 부담…넷리스 리츠 사볼까,[이데일리 권소현 기자] 올해 글로벌 리츠 변동성이 확대될 것으로 예상되는 가운데 ...,[이데일리 권소현 기자] 올해 글로벌 리츠 변동성이 확대될 것으로 예상되는 가운데 ...,"[(0, 0.021519404), (1, 0.20194383), (2, 0.0701...",7.0,0.4127,17.0
3,2021.01.01. 오전 10:01,조정대상지역 제외된 포항 ‘흥해 서희스타힐스 더 캐슬’ 오는 6일(수) 1순위 청...,"지난 17일 국토교통부가 부산 9곳, 대구 7곳, 광주 5곳, 울산 2곳, 파...",‘흥해 서희스타힐스 더 캐슬’은 포항시 북구 흥해읍 남성리 남옥지구 B1블록에 지하...,"[(0, 0.7659112), (1, 0.08600859), (2, 0.018735...",0.0,0.7659,29.0
4,2021.01.01. 오전 10:01,지식산업센터 임대 관리 편해 투자자 사이에서 인기 높아… 장기 임대에 임대료 안 밀린다,임대인 관리와 안정적이고 꾸준한 수익을 기대할 수 있는 지식산업센터 투자에 투...,임대인 관리와 안정적이고 꾸준한 수익을 기대할 수 있는 지식산업센터 투자에 투자자가...,"[(0, 0.57049745), (1, 0.13001157), (2, 0.04496...",0.0,0.5705,28.0
...,...,...,...,...,...,...,...,...
997,2021.01.04. 오전 10:32,김제 요촌동‧신풍동 2곳 '마을정비형 공공주택 공모' 선정,(김제=뉴스1) 김재수 기자 = 전북 김제시는 국토교통부 '마을정비형 공공주택사업'...,NaN,"[(0, 0.8717346), (5, 0.07839203), (8, 0.036954...",5.0,0.5150,10.0
998,2021.01.04. 오전 10:32,"[신년사]윤종규 KB금융 회장 ""빅블러 시대 대변화 오히려 호기""",[이데일리 이진철 기자] 윤종규 KB금융그룹 회장은 4일 “가속화되는 변화와 업종간...,[이데일리 이진철 기자] 윤종규 KB금융그룹 회장은 4일 “가속화되는 변화와 업종간...,"[(1, 0.011547657), (2, 0.025092348), (3, 0.011...",4.0,0.4393,86.0
999,2021.01.04. 오전 10:32,"[신년사]김학수 금결원장 ""금융결제·인증·데이터 분야 최고될 것""",4일 김 원장은 신년사를 통해 “금결원이 금융권 디지털 뉴딜의 선봉장으로 역할을 계...,김 원장은 “금융·공공에만 쓸 수 있었던 인증서 용도 제한이 해소되면서 금결원의 인...,"[(1, 0.03282939), (2, 0.074339665), (5, 0.5149...",9.0,0.4759,15.0
1000,2021.01.04. 오전 10:34,"SH공사, 이산화탄소·미세먼지 저감 '스마트 모스월' 특허…마곡에 적용",[아시아경제 김유리 기자] 서울주택도시공사(SH공사)가 이끼를 활용해 도시 내 이산...,[아시아경제 김유리 기자] 서울주택도시공사(SH공사)가 이끼를 활용해 도시 내 이산...,"[(0, 0.28889814), (3, 0.0591748), (4, 0.439339...",5.0,0.6075,14.0


In [79]:
test_df1.dropna(inplace=True)
test_df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 699 entries, 0 to 1001
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   date             699 non-null    object 
 1   title            699 non-null    object 
 2   contents         699 non-null    object 
 3   summary          699 non-null    object 
 4   LDA_topic_value  699 non-null    object 
 5   cluster_LDA      699 non-null    float64
 6   LDA_topic_score  699 non-null    float64
 7   sentence_len     699 non-null    float64
dtypes: float64(3), object(5)
memory usage: 49.1+ KB


<ipython-input-79-0d7a6052dd36>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df1.dropna(inplace=True)


In [76]:
len(test_df1)

8

In [80]:
tok_encode_con_li = []
for i in test_df1['contents'] :
    tok_encode_con = tokenizer.encode(i)
    tok_encode_con_li.append(tok_encode_con)
test_df1['tok_en_con'] = tok_encode_con_li

tok_encode_sum_li = []
for i in test_df1['summary'] :
    tok_encode_sum = tokenizer.encode(i)
    tok_encode_sum_li.append(tok_encode_sum)
test_df1['tok_en_sum'] = tok_encode_sum_li

tok_encode_con_pad = pad_sequences(test_df1['tok_en_con'], padding='post',value=-1,)
tok_encode_con_pad_li = []
for i in tok_encode_con_pad :
    tok_encode_con_pad_li.append(i)
test_df1['tok_encode_con_pad'] = tok_encode_con_pad_li

tok_encode_sum_pad = pad_sequences(test_df1['tok_en_sum'], padding='post',value=-1,)
tok_encode_sum_pad_li = []
for i in tok_encode_sum_pad :
    tok_encode_sum_pad_li.append(i)
test_df1['tok_encode_sum_pad'] = tok_encode_sum_pad_li

<ipython-input-80-930bbdb5a1a5>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df1['tok_en_con'] = tok_encode_con_li
<ipython-input-80-930bbdb5a1a5>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df1['tok_en_sum'] = tok_encode_sum_li
<ipython-input-80-930bbdb5a1a5>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

In [86]:
len(test_df1['tok_encode_con_pad'][0])

5027

In [84]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding

In [135]:
model = Sequential()
model.add(Embedding(5027, 1000))
model.add(LSTM(200))
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
x = test_df1['tok_encode_con_pad']

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
history = model.fit(x_train, y_train, batch_size=128, epochs=30, callbacks=[es, mc], validation_data=(x_test, y_test))

In [165]:
# x = np.array(test_df1['tok_encode_con_pad'])
# y = np.array(test_df1['tok_encode_sum_pad'])
# np.array(your_array).astype("float32")
x = np.array(test_df1['tok_encode_con_pad_toli'])#.astype('int')
y = np.array(test_df1['tok_encode_sum_pad_toli'])#.astype('int')

In [151]:
x = test_df1['tok_encode_con_pad'].values.tolist()
x = test_df1['tok_encode_sum_pad'].values.tolist()

In [271]:
# (test_con, test_sum)
model.fit(test_con, test_sum, epochs=10)

Epoch 1/10


ValueError: in user code:

    C:\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    C:\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2945 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:789 run_step  **
        outputs = model.train_step(data)
    C:\anaconda3\lib\site-packag

In [122]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state=0)

In [164]:
test_df1['tok_encode_con_pad_toli'] = test_df1['tok_encode_con_pad'].apply(lambda x : x.tolist())
test_df1['tok_encode_sum_pad_toli'] = test_df1['tok_encode_sum_pad'].apply(lambda x : x.tolist())
test_df1

<ipython-input-164-fe2cc71feb8a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df1['tok_encode_con_pad_toli'] = test_df1['tok_encode_con_pad'].apply(lambda x : x.tolist())
<ipython-input-164-fe2cc71feb8a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df1['tok_encode_sum_pad_toli'] = test_df1['tok_encode_sum_pad'].apply(lambda x : x.tolist())


,date,title,contents,summary,LDA_topic_value,cluster_LDA,LDA_topic_score,sentence_len,tok_en_con,tok_en_sum,tok_encode_con_pad,tok_encode_sum_pad,tok_encode_con_pad_toli,tok_encode_sum_pad_toli
0,2021.01.01. 오전 10:00,해운대 랜드마크 상업시설 ‘엘시티 더몰(the MALL)’ 분양돌입… 조기 완판 기대,㈜엘시티피에프브이(PFV)가 개별분양에 돌입한 상업시설 ‘엘시티 더몰(the MAL...,‘엘시티 더몰(the MALL)’은 해운대 방문객 수요와 함께 엘시티 내 마련되는 ...,"[(0, 0.6916478), (1, 0.015565783), (6, 0.06934...",0.0,0.6916,24.0,"[101, 100, 113, 153, 11565, 11779, 114, 8843, ...","[101, 100, 9562, 14040, 45725, 9074, 118937, 1...","[101, 100, 113, 153, 11565, 11779, 114, 8843, ...","[101, 100, 9562, 14040, 45725, 9074, 118937, 1...","[101, 100, 113, 153, 11565, 11779, 114, 8843, ...","[101, 100, 9562, 14040, 45725, 9074, 118937, 1..."
2,2021.01.01. 오전 10:00,재산세 일부 임차인이 부담…넷리스 리츠 사볼까,[이데일리 권소현 기자] 올해 글로벌 리츠 변동성이 확대될 것으로 예상되는 가운데 ...,[이데일리 권소현 기자] 올해 글로벌 리츠 변동성이 확대될 것으로 예상되는 가운데 ...,"[(0, 0.021519404), (1, 0.20194383), (2, 0.0701...",7.0,0.4127,17.0,"[101, 164, 9638, 28911, 18392, 12692, 8917, 22...","[101, 164, 9638, 28911, 18392, 12692, 8917, 22...","[101, 164, 9638, 28911, 18392, 12692, 8917, 22...","[101, 164, 9638, 28911, 18392, 12692, 8917, 22...","[101, 164, 9638, 28911, 18392, 12692, 8917, 22...","[101, 164, 9638, 28911, 18392, 12692, 8917, 22..."
3,2021.01.01. 오전 10:01,조정대상지역 제외된 포항 ‘흥해 서희스타힐스 더 캐슬’ 오는 6일(수) 1순위 청...,"지난 17일 국토교통부가 부산 9곳, 대구 7곳, 광주 5곳, 울산 2곳, 파...",‘흥해 서희스타힐스 더 캐슬’은 포항시 북구 흥해읍 남성리 남옥지구 B1블록에 지하...,"[(0, 0.7659112), (1, 0.08600859), (2, 0.018735...",0.0,0.7659,29.0,"[101, 9706, 33305, 46079, 8909, 26444, 25242, ...","[101, 100, 10021, 14523, 9425, 49515, 12605, 2...","[101, 9706, 33305, 46079, 8909, 26444, 25242, ...","[101, 100, 10021, 14523, 9425, 49515, 12605, 2...","[101, 9706, 33305, 46079, 8909, 26444, 25242, ...","[101, 100, 10021, 14523, 9425, 49515, 12605, 2..."
4,2021.01.01. 오전 10:01,지식산업센터 임대 관리 편해 투자자 사이에서 인기 높아… 장기 임대에 임대료 안 밀린다,임대인 관리와 안정적이고 꾸준한 수익을 기대할 수 있는 지식산업센터 투자에 투...,임대인 관리와 안정적이고 꾸준한 수익을 기대할 수 있는 지식산업센터 투자에 투자자가...,"[(0, 0.57049745), (1, 0.13001157), (2, 0.04496...",0.0,0.5705,28.0,"[101, 9644, 14423, 12030, 8900, 12692, 12638, ...","[101, 9644, 14423, 12030, 8900, 12692, 12638, ...","[101, 9644, 14423, 12030, 8900, 12692, 12638, ...","[101, 9644, 14423, 12030, 8900, 12692, 12638, ...","[101, 9644, 14423, 12030, 8900, 12692, 12638, ...","[101, 9644, 14423, 12030, 8900, 12692, 12638, ..."
5,2021.01.01. 오전 10:01,4월부터 불법 공매도 감시·처벌 강화,[서울경제] 3월 공매도 재개에 따라 금융당국이 불법(무차입) 공매도에 대한 처벌과...,[서울경제] 3월 공매도 재개에 따라 금융당국이 불법(무차입) 공매도에 대한 처벌과...,"[(1, 0.022685068), (2, 0.08446691), (5, 0.0102...",8.0,0.5257,16.0,"[101, 164, 48253, 31720, 17730, 166, 15361, 88...","[101, 164, 48253, 31720, 17730, 166, 15361, 88...","[101, 164, 48253, 31720, 17730, 166, 15361, 88...","[101, 164, 48253, 31720, 17730, 166, 15361, 88...","[101, 164, 48253, 31720, 17730, 166, 15361, 88...","[101, 164, 48253, 31720, 17730, 166, 15361, 88..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2021.01.04. 오전 10:31,"교보생명, 보장금액 점차 늘어나는 종신보험 출시",[헤럴드경제=정경수 기자] 교보생명은 보험료 부담을 대폭 낮춘 ‘(무)교보실속있는체...,"61세 체증형은 61세부터, 조기체증형은 가입 3년 후부터 최대 20년간 매년 보험...","[(2, 0.8073188), (4, 0.068389356), (9, 0.12062...",0.0,0.8717,17.0,"[101, 164, 9973, 118865, 15001, 31720, 17730, ...","[101, 12314, 24982, 9752, 119230, 27506, 10892...","[101, 164, 9973, 118865, 15001, 31720, 17730, ...","[101, 12314, 24982, 9752, 119230, 27506, 10892...","[101, 164, 9973, 118865, 15001, 31720, 17730, ...","[101, 12314, 24982, 9752, 119230, 27506, 10892..."
998,2021.01.04. 오전 10:32,"[신년사]윤종규 KB금융 회장 ""빅블러 시대 대변화 오히려 호기""",[이데일리 이진철 기자] 윤종규 KB금융그룹 회장은 4일 “가속화되는 변화와 업종간...,[이데일리 이진철 기자] 윤종규 KB금융그룹 회장은 4일 “가속화되는 변화와 업종간...,"[(1, 0.011547657), (2, 0.025092348), (3, 0.011...",4.0,0.4393,86.0,"[101, 164, 9638, 28911, 18392, 12692, 9638, 18...","[101, 164, 9638, 28911, 18392, 12692, 9638, 18...","[101, 164, 9638, 28911, 18392, 12692, 9638, 18...","[101, 164, 9638, 28911, 18392, 12692, 9638, 18...","[101, 164, 9638, 28911, 18392, 12692, 9638, 18...","[101, 164, 

In [221]:
test_df1

,date,title,contents,summary,LDA_topic_value,cluster_LDA,LDA_topic_score,sentence_len,tok_en_con,tok_en_sum,tok_encode_con_pad,tok_encode_sum_pad,tok_encode_con_pad_toli,tok_encode_sum_pad_toli
0,2021.01.01. 오전 10:00,해운대 랜드마크 상업시설 ‘엘시티 더몰(the MALL)’ 분양돌입… 조기 완판 기대,㈜엘시티피에프브이(PFV)가 개별분양에 돌입한 상업시설 ‘엘시티 더몰(the MAL...,‘엘시티 더몰(the MALL)’은 해운대 방문객 수요와 함께 엘시티 내 마련되는 ...,"[(0, 0.6916478), (1, 0.015565783), (6, 0.06934...",0.0,0.6916,24.0,"[101, 100, 113, 153, 11565, 11779, 114, 8843, ...","[101, 100, 9562, 14040, 45725, 9074, 118937, 1...","[101, 100, 113, 153, 11565, 11779, 114, 8843, ...","[101, 100, 9562, 14040, 45725, 9074, 118937, 1...","[101, 100, 113, 153, 11565, 11779, 114, 8843, ...","[101, 100, 9562, 14040, 45725, 9074, 118937, 1..."
2,2021.01.01. 오전 10:00,재산세 일부 임차인이 부담…넷리스 리츠 사볼까,[이데일리 권소현 기자] 올해 글로벌 리츠 변동성이 확대될 것으로 예상되는 가운데 ...,[이데일리 권소현 기자] 올해 글로벌 리츠 변동성이 확대될 것으로 예상되는 가운데 ...,"[(0, 0.021519404), (1, 0.20194383), (2, 0.0701...",7.0,0.4127,17.0,"[101, 164, 9638, 28911, 18392, 12692, 8917, 22...","[101, 164, 9638, 28911, 18392, 12692, 8917, 22...","[101, 164, 9638, 28911, 18392, 12692, 8917, 22...","[101, 164, 9638, 28911, 18392, 12692, 8917, 22...","[101, 164, 9638, 28911, 18392, 12692, 8917, 22...","[101, 164, 9638, 28911, 18392, 12692, 8917, 22..."
3,2021.01.01. 오전 10:01,조정대상지역 제외된 포항 ‘흥해 서희스타힐스 더 캐슬’ 오는 6일(수) 1순위 청...,"지난 17일 국토교통부가 부산 9곳, 대구 7곳, 광주 5곳, 울산 2곳, 파...",‘흥해 서희스타힐스 더 캐슬’은 포항시 북구 흥해읍 남성리 남옥지구 B1블록에 지하...,"[(0, 0.7659112), (1, 0.08600859), (2, 0.018735...",0.0,0.7659,29.0,"[101, 9706, 33305, 46079, 8909, 26444, 25242, ...","[101, 100, 10021, 14523, 9425, 49515, 12605, 2...","[101, 9706, 33305, 46079, 8909, 26444, 25242, ...","[101, 100, 10021, 14523, 9425, 49515, 12605, 2...","[101, 9706, 33305, 46079, 8909, 26444, 25242, ...","[101, 100, 10021, 14523, 9425, 49515, 12605, 2..."
4,2021.01.01. 오전 10:01,지식산업센터 임대 관리 편해 투자자 사이에서 인기 높아… 장기 임대에 임대료 안 밀린다,임대인 관리와 안정적이고 꾸준한 수익을 기대할 수 있는 지식산업센터 투자에 투...,임대인 관리와 안정적이고 꾸준한 수익을 기대할 수 있는 지식산업센터 투자에 투자자가...,"[(0, 0.57049745), (1, 0.13001157), (2, 0.04496...",0.0,0.5705,28.0,"[101, 9644, 14423, 12030, 8900, 12692, 12638, ...","[101, 9644, 14423, 12030, 8900, 12692, 12638, ...","[101, 9644, 14423, 12030, 8900, 12692, 12638, ...","[101, 9644, 14423, 12030, 8900, 12692, 12638, ...","[101, 9644, 14423, 12030, 8900, 12692, 12638, ...","[101, 9644, 14423, 12030, 8900, 12692, 12638, ..."
5,2021.01.01. 오전 10:01,4월부터 불법 공매도 감시·처벌 강화,[서울경제] 3월 공매도 재개에 따라 금융당국이 불법(무차입) 공매도에 대한 처벌과...,[서울경제] 3월 공매도 재개에 따라 금융당국이 불법(무차입) 공매도에 대한 처벌과...,"[(1, 0.022685068), (2, 0.08446691), (5, 0.0102...",8.0,0.5257,16.0,"[101, 164, 48253, 31720, 17730, 166, 15361, 88...","[101, 164, 48253, 31720, 17730, 166, 15361, 88...","[101, 164, 48253, 31720, 17730, 166, 15361, 88...","[101, 164, 48253, 31720, 17730, 166, 15361, 88...","[101, 164, 48253, 31720, 17730, 166, 15361, 88...","[101, 164, 48253, 31720, 17730, 166, 15361, 88..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2021.01.04. 오전 10:31,"교보생명, 보장금액 점차 늘어나는 종신보험 출시",[헤럴드경제=정경수 기자] 교보생명은 보험료 부담을 대폭 낮춘 ‘(무)교보실속있는체...,"61세 체증형은 61세부터, 조기체증형은 가입 3년 후부터 최대 20년간 매년 보험...","[(2, 0.8073188), (4, 0.068389356), (9, 0.12062...",0.0,0.8717,17.0,"[101, 164, 9973, 118865, 15001, 31720, 17730, ...","[101, 12314, 24982, 9752, 119230, 27506, 10892...","[101, 164, 9973, 118865, 15001, 31720, 17730, ...","[101, 12314, 24982, 9752, 119230, 27506, 10892...","[101, 164, 9973, 118865, 15001, 31720, 17730, ...","[101, 12314, 24982, 9752, 119230, 27506, 10892..."
998,2021.01.04. 오전 10:32,"[신년사]윤종규 KB금융 회장 ""빅블러 시대 대변화 오히려 호기""",[이데일리 이진철 기자] 윤종규 KB금융그룹 회장은 4일 “가속화되는 변화와 업종간...,[이데일리 이진철 기자] 윤종규 KB금융그룹 회장은 4일 “가속화되는 변화와 업종간...,"[(1, 0.011547657), (2, 0.025092348), (3, 0.011...",4.0,0.4393,86.0,"[101, 164, 9638, 28911, 18392, 12692, 9638, 18...","[101, 164, 9638, 28911, 18392, 12692, 9638, 18...","[101, 164, 9638, 28911, 18392, 12692, 9638, 18...","[101, 164, 9638, 28911, 18392, 12692, 9638, 18...","[101, 164, 9638, 28911, 18392, 12692, 9638, 18...","[101, 164, 

In [170]:
rfc.fit(test_df1['tok_encode_con_pad_toli'], test_df1['tok_encode_sum_pad_toli'])

ValueError: setting an array element with a sequence.

In [242]:
count = 0
for i in test_df1['tok_encode_con_pad_toli'] :
#     print(i[0])
#     count+=1
    for k in range(len(i)) :
        test.append({k:i[k]}, ignore_index=True)

KeyboardInterrupt: 

In [243]:
test

""


In [266]:
test_con = pd.DataFrame()
for i in range(len(test_df1['tok_encode_con_pad_toli'])) : 
    test_con2 = pd.DataFrame(test_df1['tok_encode_con_pad_toli'][i])
    test_con = pd.concat([test_con,test_con2], axis=1)
    
test_sum = pd.DataFrame()
for i in range(len(test_df1['tok_encode_sum_pad_toli'])) : 
    test_sum2 = pd.DataFrame(test_df1['tok_encode_sum_pad_toli'][i])
    test_sum = pd.concat([test_sum,test_sum2], axis=1)

In [268]:
test_con = test_con.T
test_sum = test_sum.T

merge = pd.concat([test_sum,test_sum2], axis=1)
merge

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [265]:
test1.T

,0,1,2,3,4,5,6,7,8,9,...,5017,5018,5019,5020,5021,5022,5023,5024,5025,5026
0,101,100,113,153,11565,11779,114,8843,8857,61844,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
0,101,164,9638,28911,18392,12692,8917,22333,30842,60886,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
0,101,9706,33305,46079,8909,26444,25242,43022,81896,9365,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
0,101,9644,14423,12030,8900,12692,12638,9521,16605,14801,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
0,101,164,48253,31720,17730,166,15361,8896,100372,12092,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,101,164,9973,118865,15001,31720,17730,134,9670,31720,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
0,101,164,9638,28911,18392,12692,9638,18623,47465,60886,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
0,101,43494,8935,9612,63671,9487,10954,32159,25605,100,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
0,101,164,9519,46861,31720,17730,8935,42815,12692,60886,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [260]:
test_df1=test_df1.reset_index()

In [269]:
rfc.fit(test_con, test_sum)

MemoryError: could not allocate 7434821632 bytes

In [175]:
tokenizer.decode(test_df1['tok_encode_con_pad'][0])

'[CLS] [UNK] ( PFV ) 가 개별분양에 돌입한 상업시설 [UNK] 엘시티 더몰 ( the MALL ) [UNK] 의 분양에 순풍이 불고 있다. 지난 21일부터 24일까지 공개청약을 진행한 부산 해운대의 초대형 상업시설 [UNK] 엘시티 더몰 ( the MALL ) [UNK] 의 경우 최고 청약경쟁률이 15대 1를 기록하는 등 분양에 선전하고 있어 주목을 받고 있다. 특히 해운대해수욕장을 조망할 수 있거나 출입구 주변에 위치한 인기 점포의 경우 대부분 분양이 완료됐다. 상업시설은 엘시티 지상 1 ~ 3층에 [UNK] 규모로 조성되며, 개별 분양으로 진행된다. 청약 전 일주일간 분양의향서를 접수한 결과 700여명이 몰리면서 조기 완판에 대한 기대감을 높이기도 했다. [UNK] 엘시티 더몰 ( the MALL ) [UNK] 은 부산광역시 해운대구 일원에 지상 1층 ~ 지상 3층, 연면적 8만 [UNK] 규모로 조성되는 상업시설로, 국내에서 두번째로 높은 초고층 주상복합단지 엘시티에 들어서 일대 랜드마크로 자리잡을 것으로 보여진다. 상업시설은 해운대 해수욕장과 인접해 있으며, 이러한 강점을 적극 활용한 매장 구성과 설계를 선보인다. 먼저, 해운대를 바라보는 상업시설 전면부에는 오션뷰가 가능한 통창이 적용돼 해운대 방문객들의 접근성을 높였다. 1층에는 오션뷰 데크를, 2 ~ 3층에 위치한 점포의 경우에는 오션뷰 테라스를 적용해 해운대 오션뷰를 극대화했다. 또한, 상업시설 앞에 위치한 광장 및 해수욕장과의 연결을 고려한 출입구 배치로 일대 유동인구를 흡수할 수 있을 것으로 보인다. 상업시설 내에는 층별 색다른 경험을 위한 리딩테넌트 유치가 예정돼 있음은 물론 집객력이 큰 앵커테넌트들도 계획돼 있다. 현재 엘시티 98층 ~ 101층에는 부산 최고층 전망대 [UNK] 부산 엑스 더 스카이 ( BUSAN X the SKY ) [UNK] 가 운영 중이며, 4층 ~ 6층에는 국내 최대 규모의 인피니티풀과 온천수를 활용해 사계절 내내 이용 가능한 도심형 워터파크가 내년

In [ ]:
tokenizer.encode(i)

In [ ]:
# test = pd.DataFrame([test_df1['tok_encode_con_pad_toli'], test_df1['tok_encode_sum_pad_toli']]).T
test = pd.DataFrame()

In [215]:
for i in test_df1['tok_encode_con_pad_toli'] :
    count = 1
    for k in range(len(i)) :
        test['{}-{}'.format(count,k)] = i[k]
    count += 1

In [220]:
test['1-0']

101

In [188]:
from tensorflow.keras.np_utils import np_utils

ModuleNotFoundError: No module named 'tensorflow.keras.np_utils'

In [186]:
np.to_categorical(test_df1['tok_encode_con_pad'])

AttributeError: module 'numpy' has no attribute 'to_categorical'

In [68]:
tok_encode_con_pad = pad_sequences(test_df1['tok_en_con'], padding='post',value=-1,)
tok_encode_con_pad_li = []
for i in tok_encode_con_pad :
    tok_encode_con_pad_li.append(i)
test_df1['tok_encode_con_pad'] = tok_encode_con_pad_li

tok_encode_sum_pad = pad_sequences(test_df1['tok_en_sum'], padding='post',value=-1,)
tok_encode_sum_pad_li = []
for i in tok_encode_sum_pad :
    tok_encode_sum_pad_li.append(i)
test_df1['tok_encode_sum_pad'] = tok_encode_sum_pad_li

<ipython-input-68-5cd3637386ab>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df1['tok_encode_con_pad'] = tok_encode_con_pad_li


In [70]:
tok_encode_con_pad[0]

array([101, 100, 113, ...,  -1,  -1,  -1])

In [69]:
tok_encode_con_pad_li[0]

array([101, 100, 113, ...,  -1,  -1,  -1])

In [67]:
len(test_df1['tok_encode_con_pad'][2])

1432

In [71]:
test_df1

,date,title,contents,summary,LDA_topic_value,cluster_LDA,LDA_topic_score,sentence_len,tok_en_con,tok_en_sum,tok_encode_con_pad
0,2021.01.01. 오전 10:00,해운대 랜드마크 상업시설 ‘엘시티 더몰(the MALL)’ 분양돌입… 조기 완판 기대,㈜엘시티피에프브이(PFV)가 개별분양에 돌입한 상업시설 ‘엘시티 더몰(the MAL...,‘엘시티 더몰(the MALL)’은 해운대 방문객 수요와 함께 엘시티 내 마련되는 ...,"[(0, 0.6916478), (1, 0.015565783), (6, 0.06934...",0.0,0.6916,24.0,"[101, 100, 113, 153, 11565, 11779, 114, 8843, ...","[101, 100, 9562, 14040, 45725, 9074, 118937, 1...","[101, 100, 113, 153, 11565, 11779, 114, 8843, ..."
2,2021.01.01. 오전 10:00,재산세 일부 임차인이 부담…넷리스 리츠 사볼까,[이데일리 권소현 기자] 올해 글로벌 리츠 변동성이 확대될 것으로 예상되는 가운데 ...,[이데일리 권소현 기자] 올해 글로벌 리츠 변동성이 확대될 것으로 예상되는 가운데 ...,"[(0, 0.021519404), (1, 0.20194383), (2, 0.0701...",7.0,0.4127,17.0,"[101, 164, 9638, 28911, 18392, 12692, 8917, 22...","[101, 164, 9638, 28911, 18392, 12692, 8917, 22...","[101, 164, 9638, 28911, 18392, 12692, 8917, 22..."
3,2021.01.01. 오전 10:01,조정대상지역 제외된 포항 ‘흥해 서희스타힐스 더 캐슬’ 오는 6일(수) 1순위 청...,"지난 17일 국토교통부가 부산 9곳, 대구 7곳, 광주 5곳, 울산 2곳, 파...",‘흥해 서희스타힐스 더 캐슬’은 포항시 북구 흥해읍 남성리 남옥지구 B1블록에 지하...,"[(0, 0.7659112), (1, 0.08600859), (2, 0.018735...",0.0,0.7659,29.0,"[101, 9706, 33305, 46079, 8909, 26444, 25242, ...","[101, 100, 10021, 14523, 9425, 49515, 12605, 2...","[101, 9706, 33305, 46079, 8909, 26444, 25242, ..."
4,2021.01.01. 오전 10:01,지식산업센터 임대 관리 편해 투자자 사이에서 인기 높아… 장기 임대에 임대료 안 밀린다,임대인 관리와 안정적이고 꾸준한 수익을 기대할 수 있는 지식산업센터 투자에 투...,임대인 관리와 안정적이고 꾸준한 수익을 기대할 수 있는 지식산업센터 투자에 투자자가...,"[(0, 0.57049745), (1, 0.13001157), (2, 0.04496...",0.0,0.5705,28.0,"[101, 9644, 14423, 12030, 8900, 12692, 12638, ...","[101, 9644, 14423, 12030, 8900, 12692, 12638, ...","[101, 9644, 14423, 12030, 8900, 12692, 12638, ..."
5,2021.01.01. 오전 10:01,4월부터 불법 공매도 감시·처벌 강화,[서울경제] 3월 공매도 재개에 따라 금융당국이 불법(무차입) 공매도에 대한 처벌과...,[서울경제] 3월 공매도 재개에 따라 금융당국이 불법(무차입) 공매도에 대한 처벌과...,"[(1, 0.022685068), (2, 0.08446691), (5, 0.0102...",8.0,0.5257,16.0,"[101, 164, 48253, 31720, 17730, 166, 15361, 88...","[101, 164, 48253, 31720, 17730, 166, 15361, 88...","[101, 164, 48253, 31720, 17730, 166, 15361, 88..."
6,2021.01.01. 오전 10:01,2021년 IPO 대박 이어진다...카뱅·크래프톤 등 출격 준비,[서울경제] 지난해 기업공개(IPO)에는 300조 원 가량의 청약자금이 몰리는 등 ...,"앞서 SK증권은 LG에너지솔루션(40조~50조 원), 크래프톤(20조~30조 원),...","[(1, 0.03350094), (4, 0.17119534), (6, 0.19393...",7.0,0.5986,22.0,"[101, 164, 48253, 31720, 17730, 166, 9706, 333...","[101, 9531, 12424, 21275, 119230, 25347, 10892...","[101, 164, 48253, 31720, 17730, 166, 9706, 333..."
7,2021.01.01. 오전 10:14,연말 막혔던 은행 신용대출 다시 열린다,[이데일리 김유성 기자] 시중은행들의 신용대출 상품 판매가 새해부터 속속 재개된다....,[이데일리 김유성 기자] 시중은행들의 신용대출 상품 판매가 새해부터 속속 재개된다....,"[(1, 0.0970983), (2, 0.78306836), (6, 0.060629...",2.0,0.7828,25.0,"[101, 164, 9638, 28911, 18392, 12692, 8935, 42...","[101, 164, 9638, 28911, 18392, 12692, 8935, 42...","[101, 164, 9638, 28911, 18392, 12692, 8935, 42..."
8,2021.01.01. 오전 10:18,"2020년 뉴욕증시, 신고가로 마감…""내년엔 모든 것이 좋을 것""",2020년 뉴욕증시가 신고점을 경신하면서 마감했다. 나스닥 지수는 올해 약 44% ...,나스닥 지수는 올해 약 44% 급등했다.31일(현지시각) 뉴욕증권거래소(NYSE)에...,"[(1, 0.20098624), (4, 0.014861636), (6, 0.5922...",6.0,0.5922,16.0,"[101, 23607, 10954, 9039, 109971, 119230, 1404...","[101, 8982, 12605, 118770, 9706, 93835, 9583, ...","[101, 23607, 10954, 9039, 109971, 119230, 1404..."


In [47]:
test_df1['tok_en_con']

0    101
2    101
3    101
4    101
5    101
6    101
7    101
8    101
Name: tok_en_con, dtype: int32

In [38]:
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git
from pykospacing import spacing


  Cloning https://github.com/haven-jeon/PyKoSpacing.git to c:\users\dmaqp\appdata\local\temp\pip-req-build-bgmorsif
  Resolved https://github.com/haven-jeon/PyKoSpacing.git to commit f2febf985f256f4ebf169f4f78a765fef715d2cb
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for pykospacing: filename=pykospacing-0.5-py3-none-any.whl size=2268594 sha256=a879370bb2343d58efe0cce28c52c01004b99a94dd067f5270d4dbc9d01d5c23
  Stored in directory: C:\Users\dmaqp\AppData\Local\Temp\pip-ephem-wheel-cache-cm9ac3lp\wheels\79\a0\33\16f2cd03d21f76a663f5d69a0b96f0351335385349136fbd03
Successfully built pykospacing
  Attempting uninstall: numpy
    Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18.5:
      Successfully uninstalled numpy-1.18.5


  Running command git clone --filter=blob:none -q https://github.com/haven-jeon/PyKoSpacing.git 'C:\Users\dmaqp\AppData\Local\Temp\pip-req-build-bgmorsif'
ERROR: Could not install packages due to an OSError: [WinError 5] 액세스가 거부되었습니다: 'C:\\anaconda3\\Lib\\site-packages\\~umpy\\.libs\\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll'
Consider using the `--user` option or check the permissions.



ModuleNotFoundError: No module named 'pykospacing'

In [22]:
tokenizer.encode(test_df1['contents'][0])

list

In [13]:
len(tokenizer.encode(test_df1['contents'][1]))

946

In [250]:
test_df['contents'][0]

'㈜엘시티피에프브이(PFV)가 개별분양에 돌입한 상업시설 ‘엘시티 더몰(the MALL)’의 분양에 순풍이 불고 있다.지난 21일부터 24일까지 공개청약을 진행한 부산 해운대의 초대형 상업시설 ‘엘시티 더몰(the MALL)’의 경우 최고 청약경쟁률이 15대 1를 기록하는 등 분양에 선전하고 있어 주목을 받고 있다. 특히 해운대해수욕장을 조망할 수 있거나 출입구 주변에 위치한 인기 점포의 경우 대부분 분양이 완료됐다.상업시설은 엘시티 지상 1~3층에 8만3790㎡ 규모로 조성되며, 개별 분양으로 진행된다. 청약 전 일주일간 분양의향서를 접수한 결과 700여명이 몰리면서 조기 완판에 대한 기대감을 높이기도 했다.‘엘시티 더몰(the MALL)’은 부산광역시 해운대구 일원에 지상 1층~지상 3층, 연면적 8만 3790㎡ 규모로 조성되는 상업시설로, 국내에서 두번째로 높은 초고층 주상복합단지 엘시티에 들어서 일대 랜드마크로 자리잡을 것으로 보여진다.상업시설은 해운대 해수욕장과 인접해 있으며, 이러한 강점을 적극 활용한 매장 구성과 설계를 선보인다. 먼저, 해운대를 바라보는 상업시설 전면부에는 오션뷰가 가능한 통창이 적용돼 해운대 방문객들의 접근성을 높였다. 1층에는 오션뷰 데크를, 2~3층에 위치한 점포의 경우에는 오션뷰 테라스를 적용해 해운대 오션뷰를 극대화했다. 또한, 상업시설 앞에 위치한 광장 및 해수욕장과의 연결을 고려한 출입구 배치로 일대 유동인구를 흡수할 수 있을 것으로 보인다.상업시설 내에는 층별 색다른 경험을 위한 리딩테넌트 유치가 예정돼 있음은 물론 집객력이 큰 앵커테넌트들도 계획돼 있다. 현재 엘시티 98층~101층에는 부산 최고층 전망대 ‘부산 엑스 더 스카이(BUSAN X the SKY)’가 운영 중이며, 4층~6층에는 국내 최대 규모의 인피니티풀과 온천수를 활용해 사계절 내내 이용 가능한 도심형 워터파크가 내년 중 상업시설과 함께 개장 예정이다. 롯데호텔이 부산 최초로 오픈한 6성급 ‘시그니엘 부산’ 역시 호텔웨딩, 연회장 등이 마련돼, 엘시

In [173]:
from sklearn.ensemble import RandomForestClassifier

In [174]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=5, random_state=0)

In [196]:
tfidf.fit_transform(test_df2['contents'][0])

KeyError: 'contents'

In [198]:
x_list = []
y_list = []

for x,y in zip(test_df['contents'], test_df['summary']) :
    x_token = tokenizer.encode(x)
    y_token = tokenizer.encode(y)
    
    x_list.append(x_token)
    y_list.append(y_token)
    
test_df2 = pd.DataFrame()
test_df2['x'] = x_list
test_df2['y'] = y_list

In [240]:
len(test_df2['x'][3])

1160

In [226]:
x_train = np.array(test_df2['x'].values, dtype='float')
# y_train = np.array(list(test_df2['y']))

ValueError: setting an array element with a sequence.

In [213]:
print(x_train.shape)
print(y_train.shape)

(8,)
(8,)


In [221]:
type(x[0])

str

In [212]:
rfc.fit(x_train,y_train)

ValueError: setting an array element with a sequence.

In [191]:
rfc.fit(np.array(list(test_df2['x'])),np.array(list(test_df2['y'])))

ValueError: setting an array element with a sequence.

In [186]:
type(test_df2['x'])

pandas.core.series.Series

In [172]:
type(k)

list

In [8]:
test_df['contents'][0]

NameError: name 'test_df' is not defined

In [162]:
segments_ids

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [159]:
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

In [160]:
with torch.no_grad():
    outputs = model(tokens_tensor, segments_tensors)
    hidden_states = outputs[2]

TypeError: 'Word2Vec' object is not callable

In [147]:
tokenizer

PreTrainedTokenizer(name_or_path='bert-base-multilingual-cased', vocab_size=119547, model_max_len=512, is_fast=False, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [146]:
how_to_tokenize = BertTokenizer()

TypeError: __init__() missing 1 required positional argument: 'vocab_file'

In [ ]:
list(tokenizer.vocab.keys())[5000:5020]

In [117]:
x_vect_list[0][1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [107]:
y_mat

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [104]:
test_df

,date,title,contents,press,good,well,sad,mad,want,summary
0,2021.01.01. 오전 10:00,해운대 랜드마크 상업시설 ‘엘시티 더몰(the MALL)’ 분양돌입… 조기 완판 기대,㈜엘시티피에프브이(PFV)가 개별분양에 돌입한 상업시설 ‘엘시티 더몰(the MAL...,https://www.asiae.co.kr/,0,0,0,0,0,‘엘시티 더몰(the MALL)’은 해운대 방문객 수요와 함께 엘시티 내 마련되는 ...
2,2021.01.01. 오전 10:00,재산세 일부 임차인이 부담…넷리스 리츠 사볼까,[이데일리 권소현 기자] 올해 글로벌 리츠 변동성이 확대될 것으로 예상되는 가운데 ...,http://www.edaily.co.kr/,0,0,0,0,0,[이데일리 권소현 기자] 올해 글로벌 리츠 변동성이 확대될 것으로 예상되는 가운데 ...
3,2021.01.01. 오전 10:01,조정대상지역 제외된 포항 ‘흥해 서희스타힐스 더 캐슬’ 오는 6일(수) 1순위 청...,"지난 17일 국토교통부가 부산 9곳, 대구 7곳, 광주 5곳, 울산 2곳, 파...",http://www.fnnews.com/,0,0,0,0,0,‘흥해 서희스타힐스 더 캐슬’은 포항시 북구 흥해읍 남성리 남옥지구 B1블록에 지하...
4,2021.01.01. 오전 10:01,지식산업센터 임대 관리 편해 투자자 사이에서 인기 높아… 장기 임대에 임대료 안 밀린다,임대인 관리와 안정적이고 꾸준한 수익을 기대할 수 있는 지식산업센터 투자에 투...,http://www.fnnews.com/,0,0,0,0,0,임대인 관리와 안정적이고 꾸준한 수익을 기대할 수 있는 지식산업센터 투자에 투자자가...
5,2021.01.01. 오전 10:01,4월부터 불법 공매도 감시·처벌 강화,[서울경제] 3월 공매도 재개에 따라 금융당국이 불법(무차입) 공매도에 대한 처벌과...,http://www.sedaily.com,0,0,0,0,0,[서울경제] 3월 공매도 재개에 따라 금융당국이 불법(무차입) 공매도에 대한 처벌과...
...,...,...,...,...,...,...,...,...,...,...
492,2021.01.03. 오전 11:01,수원당수 지구에 '제로에너지 특화도시' 들어선다,[아시아경제 이춘희 기자] 국토교통부와 한국토지주택공사(LH)는 에너지와 생태환경이...,https://www.asiae.co.kr/,0,0,0,0,0,홍보관도 건립해 제로에너지 도시의 홍보·견학·시민참여 공간도 마련한다.한편 특화도시...
493,2021.01.03. 오전 11:01,"3기신도시, 7월 사전청약 돌입…""3만가구 먼저 쏜다""",[이데일리 김나리 기자] 3기 신도시 등 사전청약이 올해 7월 시작된다. 인천계양을...,http://www.edaily.co.kr/,0,0,0,0,0,국토교통부는 본 청약 대비 1~2년 조기 공급하는 사전청약제를 올해 7월부터 3기 ...
494,2021.01.03. 오전 11:01,"강원대 캠퍼스, 혁신파크로 조성…뭐가 달라지나","[이데일리 김나리 기자] 국토교통부, 교육부, 중소벤처기업부는 지난해 12월 31일...",http://www.edaily.co.kr/,0,0,0,0,0,"[이데일리 김나리 기자] 국토교통부, 교육부, 중소벤처기업부는 지난해 12월 31일..."
496,2021.01.03. 오전 11:07,"""SK바이오팜, 투자의견 '중립' 하향…검증 시간 필요""",[헤럴드경제=이현정 기자]유진투자증권은 3일 SK바이오팜의 목표주가를 11만원에서 ...,http://www.heraldbiz.com/,0,0,0,0,0,파이프라인에 대한 투자 때문에 2023년까지는 적자가 불가피하다는 판단에서다.한 연...


In [103]:
x_mat.shape

(353, 305)

In [ ]:
x_mat = tfidf.fit_transform(nouns).toarray()

In [95]:
def text_clean(doc):
    doc.strip()
    doc = re.sub("[^0-9a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣]", "", doc)
    return doc

In [94]:
pattern

'엘시티피에프브이PFV가개별분양에돌입한상업시설엘시티더몰theMALL의분양에순풍이불고있다지난일부터일까지공개청약을진행한부산해운대의초대형상업시설엘시티더몰theMALL의경우최고청약경쟁률이대를기록하는등분양에선전하고있어주목을받고있다특히해운대해수욕장을조망할수있거나출입구주변에위치한인기점포의경우대부분분양이완료됐다상업시설은엘시티지상층에만규모로조성되며개별분양으로진행된다청약전일주일간분양의향서를접수한결과여명이몰리면서조기완판에대한기대감을높이기도했다엘시티더몰theMALL은부산광역시해운대구일원에지상층지상층연면적만규모로조성되는상업시설로국내에서두번째로높은초고층주상복합단지엘시티에들어서일대랜드마크로자리잡을것으로보여진다상업시설은해운대해수욕장과인접해있으며이러한강점을적극활용한매장구성과설계를선보인다먼저해운대를바라보는상업시설전면부에는오션뷰가가능한통창이적용돼해운대방문객들의접근성을높였다층에는오션뷰데크를층에위치한점포의경우에는오션뷰테라스를적용해해운대오션뷰를극대화했다또한상업시설앞에위치한광장및해수욕장과의연결을고려한출입구배치로일대유동인구를흡수할수있을것으로보인다상업시설내에는층별색다른경험을위한리딩테넌트유치가예정돼있음은물론집객력이큰앵커테넌트들도계획돼있다현재엘시티층층에는부산최고층전망대부산엑스더스카이BUSANXtheSKY가운영중이며층층에는국내최대규모의인피니티풀과온천수를활용해사계절내내이용가능한도심형워터파크가내년중상업시설과함께개장예정이다롯데호텔이부산최초로오픈한성급시그니엘부산역시호텔웨딩연회장등이마련돼엘시티단지내여러시설을방문한고객들의연계효과가기대된다또한엘시티더몰theMALL이들어서는관광특구해운대의매출스케일도주목할만하다부산광역시관광정책과가제공한자료에따르면연간만명규모년기준의방문객이다녀가는해운대해수욕장은관광객을대상으로조사된신용카드지출액규모년기준에서조억원을넘었다또한부산국제영화제월를필두로북극곰축제월모레축제월등다양한축제가계획돼있어사시사철수요가마르지않는상권으로자리잡을예정이다상업시설인근에위치한풍부한배후수요도눈길을끈다엘시티더몰theMALL은해운대방문객수요와함께엘시티내마련되는해운대엘시티더샵가구엘시티레지던스실와시그니엘부산실등을고정수요로누릴예정이며가까이

In [59]:
model_result = model.wv.most_similar("롯데")
print(model_result)

KeyError: "word '롯데' not in vocabulary"

In [65]:
from gensim.models import Word2Vec
model = Word2Vec(sentences=tokenized_data, size=100, window=5, min_count=5, workers=4, sg=0)

C:\anaconda3\lib\site-packages\gensim\models\base_any2vec.py:322: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  warnings.warn(


In [66]:
model.wv.vectors.shape

(4296, 100)

In [76]:
model.wv.

array([[ 0.79967815,  0.27416223,  0.29902717, ..., -0.40457007,
         1.0508871 , -0.16384436],
       [-0.3548724 ,  0.1073368 ,  0.35061374, ...,  0.2290531 ,
         1.4256465 , -0.647022  ],
       [-0.05606882, -0.2296918 , -0.00509479, ..., -0.3687787 ,
         0.9018941 , -0.32515296],
       ...,
       [-0.01812896,  0.0158402 ,  0.04240381, ..., -0.05798826,
         0.02382107,  0.02517957],
       [-0.00686397,  0.01951384,  0.0722761 , ..., -0.0591094 ,
         0.07943931,  0.04546796],
       [-0.02065172,  0.00301191,  0.04357777, ..., -0.04455767,
         0.02395046,  0.03852733]], dtype=float32)

In [73]:
print(model.wv.most_similar("기사"))

[('신문', 0.9871894121170044), ('파이낸셜', 0.9861264228820801), ('>', 0.986055850982666), ('이민', 0.9812132120132446), ('김유성', 0.9807685017585754), ('양지', 0.9767648577690125), ('아이', 0.9738174676895142), ('경', 0.9735647439956665), ('택', 0.9731738567352295), ('규', 0.9728302359580994)]


In [4]:
summa = []
for contents in test_df['contents'] :
    try : 
        sentence = kkma.sentences(contents)        
        if len(sentence) > 10 :
            summary = ''
            for row in summarize(contents).split('\n') :
                summary += row
            if summary.strip() == '' :
                summa.append(np.nan)
            else : 
                summa.append(summary)
        else :
            summa.append(np.nan)
    except :
        summa.append(np.nan)

test_df['summary'] = summa
test_df

<ipython-input-4-704859be3ec2>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['summary'] = summa


,date,title,contents,press,good,well,sad,mad,want,summary
0,2021.01.01. 오전 10:00,해운대 랜드마크 상업시설 ‘엘시티 더몰(the MALL)’ 분양돌입… 조기 완판 기대,㈜엘시티피에프브이(PFV)가 개별분양에 돌입한 상업시설 ‘엘시티 더몰(the MAL...,https://www.asiae.co.kr/,0,0,0,0,0,‘엘시티 더몰(the MALL)’은 해운대 방문객 수요와 함께 엘시티 내 마련되는 ...
1,2021.01.01. 오전 10:00,"합리적 가격 갖춘, 시티건설 `화성 남양 시티프라디움 4차`, 1월 4일부터 정당계...",시티건설은 경기도 화성시 남양뉴타운 B-14BL에 공급하는 '화성 남양 시티프라디움...,http://www.dt.co.kr/,0,0,0,0,0,NaN
2,2021.01.01. 오전 10:00,재산세 일부 임차인이 부담…넷리스 리츠 사볼까,[이데일리 권소현 기자] 올해 글로벌 리츠 변동성이 확대될 것으로 예상되는 가운데 ...,http://www.edaily.co.kr/,0,0,0,0,0,[이데일리 권소현 기자] 올해 글로벌 리츠 변동성이 확대될 것으로 예상되는 가운데 ...
3,2021.01.01. 오전 10:01,조정대상지역 제외된 포항 ‘흥해 서희스타힐스 더 캐슬’ 오는 6일(수) 1순위 청...,"지난 17일 국토교통부가 부산 9곳, 대구 7곳, 광주 5곳, 울산 2곳, 파...",http://www.fnnews.com/,0,0,0,0,0,‘흥해 서희스타힐스 더 캐슬’은 포항시 북구 흥해읍 남성리 남옥지구 B1블록에 지하...
4,2021.01.01. 오전 10:01,지식산업센터 임대 관리 편해 투자자 사이에서 인기 높아… 장기 임대에 임대료 안 밀린다,임대인 관리와 안정적이고 꾸준한 수익을 기대할 수 있는 지식산업센터 투자에 투...,http://www.fnnews.com/,0,0,0,0,0,임대인 관리와 안정적이고 꾸준한 수익을 기대할 수 있는 지식산업센터 투자에 투자자가...


In [ ]:
test_df = 

In [96]:
test_df

,date,title,contents,press,good,well,sad,mad,want
10027,2021.01.13. 오전 8:04,음식료 '곳곳' 好실적…최선호주는 CJ제일제당-신함금융투자,"박희진 신한금융투자 연구위원은 ""CJ제일제당 영업이익은 전년대비 12.8% 증가한 ...",http://www.inews24.com/,0,0,0,0,0
10028,2021.01.13. 오전 8:05,"삼성생명, 비대면 디지털진단 서비스 오픈",삼성생명은 비대면으로 5분 내에 간편하게 보험 가입심사를 할 수 있는 디지털진단 서...,http://www.dailian.co.kr/,0,0,0,0,0
10029,2021.01.13. 오전 8:06,"코스피 방향전환 시작될까…""영향제한적, '숨고르기 장세'""",이틀 동안 코스피가 거친 '숨고르기' 장세를 보였다. 단기 조정은 바닥을 다지는 ...,http://www.mt.co.kr/,0,0,0,0,0
10030,2021.01.13. 오전 8:08,[빅데이터MSI]13일 오전 주식시장 심리 6단계 '좋음',[서울=뉴시스] 류병화 기자 = 13일 오전 빅데이터로 분석한 주식시장 코스피 20...,http://www.newsis.com,0,0,0,0,0
10031,2021.01.13. 오전 8:08,[빅데이터MSI]13일 오전 주식시장 심리 6단계 '좋음',[서울=뉴시스] 류병화 기자 = 13일 오전 빅데이터로 분석한 주식시장 코스피 20...,http://www.newsis.com,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
15028,2021.01.18. 오전 9:02,"[시황]코스피 3079.90(▼0.19%), 코스닥 963.18(▼0.13%), 원...",코스닥지수는 전 거래일(964.44)보다 1.26포인트(0.13%) 내린 963.1...,http://www.newsis.com,0,0,0,0,0
15029,2021.01.18. 오전 9:02,일본 닛케이지수 0.98% 하락 출발,한경닷컴▶ ▶ ▶ ⓒ 한국경제 &,http://www.hankyung.com/,0,0,0,0,0
15030,2021.01.18. 오전 9:02,"LH, 청년·신혼부부 대상 공공임대 525가구 입주자 모집",주택도시기금 출자로 설립한 청년희망임대주택리츠가 매입한 아파트 등 공동주택이다. L...,http://www.edaily.co.kr/,0,0,0,0,0
15031,2021.01.18. 오전 9:02,[출발]코스피 기관 18억 순매도,09:01 현재 코스피 기관 18억 순매도마켓포인트 (stockind@edaily....,http://www.edaily.co.kr/,0,0,0,0,0


In [9]:
for contents in test_df['contents'] :  
    sentence = text2sentences(contents)
    nouns = get_nouns(sentence)
    sent_graph = build_sent_graph(sentence)
    word_graph, idx2word = bulid_word_graph(nouns)

    sent_rank_idx = get_rank(sent_graph)
    sorted_sent_rank_idx = sorted(sent_rank_idx, key=lambda k : sent_rank_idx[k], reverse=True)

    word_rank_idx = get_rank(word_graph)
    sorted_word_rank_idx = sorted(word_rank_idx, key=lambda k : word_rank_idx[k], reverse=True)

    print(summarize(punctuation_text, word_count=40))
#     key_word()
    print()

NameError: name 'text2sentences' is not defined

In [149]:
test_df['summary'] = np.nan
# test_df.columns=['date','title','contents','press','good','well','sad','mad','want','summary']
# test_df


<ipython-input-149-b7a5e3e614da>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['summary'] = np.nan


In [42]:
test_df

,date,title,contents,press,good,well,sad,mad,want,summary
369713,2021.10.29. 오후 3:59,"코스피, 외인·기관 동반 매도에 2970선까지 밀려",코스피가 외국인과 기관의 '팔자'에 2970선까지 주저 앉았다. 개인이 홀로 물량을...,http://www.inews24.com/,0,0,0,0,0,NaN
202779,2021.06.15. 오전 10:29,"롯데손보, 주력 상품 경쟁력 높인다…""수술비 담보 특약 확대""",[아시아경제 기하영 기자]롯데손해보험이 주력 보험서비스인 장기보험의 경쟁력을 높이기...,https://www.asiae.co.kr/,0,0,0,0,0,NaN
276029,2021.08.11. 오전 7:58,"제이시스메디칼, 접속초음파·고주파로 고성장 기대…목표가 ↑-대신","대신증권에 따르면 제이시스메디칼은 2분기 매출액 200억원, 영업이익 62억원을 기...",http://www.edaily.co.kr/,0,0,0,0,0,NaN
224019,2021.08.24. 오후 5:20,"금호석유화학, 폐플라스틱 재활용 사업 진출",금호석화는 기존의 물리적 재활용 방식에서 벗어나 열분해 등 화학적 재활용 방식을 활...,http://www.hankyung.com/,0,0,0,0,0,NaN
128245,2021.04.19. 오후 12:03,"새희망홀씨 대출, 올해 3.5조원 공급",[헤럴드경제=김성훈 기자] 서민들을 위한 신용대출상품인 새희망홀씨 대출이 올해 3조...,http://www.heraldbiz.com/,0,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...
169851,2021.05.20. 오전 9:09,[인사] 한국예탁결제원,▲전략기획본부장 최정철,http://www.newsis.com,0,0,0,0,0,NaN
145372,2021.06.04. 오후 3:04,"셀트리온-이셀, 국산화 바이오 부품 장기 공급계약 체결",셀트리온은 바이오 의약품 소부장(소재·부품·장비) 전문 업체 이셀과 본사 대회의실에...,http://www.dailian.co.kr/,0,0,0,0,0,NaN
156332,2021.05.10. 오후 7:57,"[스톡톡스]“코스피 최고치, 美부양책·외국인 유입…차익실현 가능성도”",[이데일리 이은정 기자] “예상치를 하회한 미국 고용지표에 따른 테이퍼링(자산매입 ...,http://www.edaily.co.kr/,0,0,0,0,0,NaN
104015,2021.04.01. 오전 10:28,"은성수 금융위원장, 금소법 관련 은행장 간담회",(서울=뉴스1) 김진환 기자 = 은성수 금융위원장이 1일 서울 중구 은행연합회 회의...,http://news1.kr/,0,0,0,0,0,NaN


In [161]:
summa = []
for contents in test_df['contents'] :
    try : 
        sentence = kkma.sentences(contents)        
        if len(sentence) > 10 :
            summary = ''
            for row in summarize(contents).split('\n') :
                summary += row
            if summary.strip() == '' :
                summa.append(np.nan)
            else : 
                summa.append(summary)
        else :
            summa.append(np.nan)
    except :
        summa.append(np.nan)

test_df['summary'] = summa
test_df

<ipython-input-161-ea114be3d130>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['summary'] = summa


,date,title,contents,press,good,well,sad,mad,want,summary
0,2021.01.01. 오전 10:00,해운대 랜드마크 상업시설 ‘엘시티 더몰(the MALL)’ 분양돌입… 조기 완판 기대,㈜엘시티피에프브이(PFV)가 개별분양에 돌입한 상업시설 ‘엘시티 더몰(the MAL...,https://www.asiae.co.kr/,0,0,0,0,0,‘엘시티 더몰(the MALL)’은 해운대 방문객 수요와 함께 엘시티 내 마련되는 ...
1,2021.01.01. 오전 10:00,"합리적 가격 갖춘, 시티건설 `화성 남양 시티프라디움 4차`, 1월 4일부터 정당계...",시티건설은 경기도 화성시 남양뉴타운 B-14BL에 공급하는 '화성 남양 시티프라디움...,http://www.dt.co.kr/,0,0,0,0,0,NaN
2,2021.01.01. 오전 10:00,재산세 일부 임차인이 부담…넷리스 리츠 사볼까,[이데일리 권소현 기자] 올해 글로벌 리츠 변동성이 확대될 것으로 예상되는 가운데 ...,http://www.edaily.co.kr/,0,0,0,0,0,[이데일리 권소현 기자] 올해 글로벌 리츠 변동성이 확대될 것으로 예상되는 가운데 ...
3,2021.01.01. 오전 10:01,조정대상지역 제외된 포항 ‘흥해 서희스타힐스 더 캐슬’ 오는 6일(수) 1순위 청...,"지난 17일 국토교통부가 부산 9곳, 대구 7곳, 광주 5곳, 울산 2곳, 파...",http://www.fnnews.com/,0,0,0,0,0,‘흥해 서희스타힐스 더 캐슬’은 포항시 북구 흥해읍 남성리 남옥지구 B1블록에 지하...
4,2021.01.01. 오전 10:01,지식산업센터 임대 관리 편해 투자자 사이에서 인기 높아… 장기 임대에 임대료 안 밀린다,임대인 관리와 안정적이고 꾸준한 수익을 기대할 수 있는 지식산업센터 투자에 투...,http://www.fnnews.com/,0,0,0,0,0,임대인 관리와 안정적이고 꾸준한 수익을 기대할 수 있는 지식산업센터 투자에 투자자가...


In [143]:
article

'임대인 관리와 안정적이고 꾸준한 수익을 기대할 수 있는 지식산업센터 투자에 투자자가 몰리고 있다.주택 시장이 정부 규제로 얼어붙어 대체 투자 처로 꼽힌 것도 지식산업센터 투자 열기에 한몫하고 있다.지식산업센터의 주된 임차인은 기업이다.기업은 주로 장기 임대를 하는 경우가 많고, 임대료가 늦는 경우가 낮아 투자자로서 안정적인 임대 수익을 기대할 수 있어 인기다.반면 개인이 입주하는 주택 투자의 경우 개인의 사정으로 인해 계약이 변경되는 경우가 잦고, 임대료가 연체되는 일도 있으며 사적인 공간으로 인해 관리가 어려운 점도 많다.또 한, 최근 정부의 임대차 3 법 등으로 인해 매년 임대료 상승을 기대할 수 없어 투자자 입장에서는 투자가치가 낮아 졌다.이 외에도 기업들이 주로 입주하는 지식산업센터는 주거지가 아닌 업무시설로 분류된다.따라서 투자자 입장에서는 주택 수에 포함되지 않아 다주택자 규제에서 피할 수 있는 등 다양한 장점이 부각되고 있다.이에 임대수익이 중요한 투자자 입장에서는 지식산업센터가 다시금 투자자에게 선호되고 있는데, 각종 지표나 정부 정책 또한 지식산업센터 투자자에게 긍정적이다.먼저 투자 진입 장벽도 낮다.지식산업센터는 각종 세제 혜택과 함께 규제에서 상대적으로 자유로 워 전매가 가능하고 담보대출 한도 또한 최대 80%까지 가능한 점도 투자자 입장에서 투자 부담이 주거시설보다 낮다.또 한, 분양 가상한 제 등 정부의 주거 시장에 대한 규제 영향도 없어 실수요는 물론 투자자들의 관심이 크다.지식산업센터의 경우 중소기업이 분양 받을 경우 취득세의 50%, 재산세의 37.5%를 감면 받을 수 있어 입주 기업에도 혜택이 크다.2019년이었던 감면 기한이 2022년 12월 31일까지 연장되면서 지식산업센터 분양 시장이 더욱 탄력을 받았다.이런 상황에 가산 디지털 단지 양지사 부지에 새롭게 들어서는 ‘ 가 산 어 반 워크’ 가 분양을 알려 주목받고 있다.서울 금천구 가산동 451-1, 452-1에 들어서는 ‘ 가 산 어 반 워크’ 는 지하 5 층~ 지상 20

In [157]:
sentence = kkma.sentences(test_df.iloc[2]['contents'])
sentence
summarize(test_df.iloc[2]['contents']).

'[이데일리 권소현 기자] 올해 글로벌 리츠 변동성이 확대될 것으로 예상되는 가운데 넷리스 리츠에 주목할 필요가 있다는 분석이 나왔다.\n이 리츠는 미국과 유럽에 산업용, 오피스, 리테일 부동산 자산을 혼합 보유한 미국혼합형 리츠인 동시에 순임대 계약 구조를 사용하는 넷리스 리츠다.\n다만 리테일 자산 비중이 높다는 점이 단점으로 꼽히지만 WPC US는 산업용 부동산 비중이 46% 이상이기 때문에 영화관, 레스토랑, 체육관 등의 비중이 높은 넷리스 리츠 대비 안정적인 성과를 낼 수 있을 것으로 전망했다.권소현 (juddie@edaily.co.kr)'

In [160]:
for a in summarize(test_df.iloc[2]['contents']).split('\n') :
    

['[이데일리 권소현 기자] 올해 글로벌 리츠 변동성이 확대될 것으로 예상되는 가운데 넷리스 리츠에 주목할 필요가 있다는 분석이 나왔다.',
 '이 리츠는 미국과 유럽에 산업용, 오피스, 리테일 부동산 자산을 혼합 보유한 미국혼합형 리츠인 동시에 순임대 계약 구조를 사용하는 넷리스 리츠다.',
 '다만 리테일 자산 비중이 높다는 점이 단점으로 꼽히지만 WPC US는 산업용 부동산 비중이 46% 이상이기 때문에 영화관, 레스토랑, 체육관 등의 비중이 높은 넷리스 리츠 대비 안정적인 성과를 낼 수 있을 것으로 전망했다.권소현 (juddie@edaily.co.kr)']

In [152]:
sentence
for i in sentence : 
    article += i.strip()
article

'임대인 관리와 안정적이고 꾸준한 수익을 기대할 수 있는 지식산업센터 투자에 투자자가 몰리고 있다.주택 시장이 정부 규제로 얼어붙어 대체 투자 처로 꼽힌 것도 지식산업센터 투자 열기에 한몫하고 있다.지식산업센터의 주된 임차인은 기업이다.기업은 주로 장기 임대를 하는 경우가 많고, 임대료가 늦는 경우가 낮아 투자자로서 안정적인 임대 수익을 기대할 수 있어 인기다.반면 개인이 입주하는 주택 투자의 경우 개인의 사정으로 인해 계약이 변경되는 경우가 잦고, 임대료가 연체되는 일도 있으며 사적인 공간으로 인해 관리가 어려운 점도 많다.또 한, 최근 정부의 임대차 3 법 등으로 인해 매년 임대료 상승을 기대할 수 없어 투자자 입장에서는 투자가치가 낮아 졌다.이 외에도 기업들이 주로 입주하는 지식산업센터는 주거지가 아닌 업무시설로 분류된다.따라서 투자자 입장에서는 주택 수에 포함되지 않아 다주택자 규제에서 피할 수 있는 등 다양한 장점이 부각되고 있다.이에 임대수익이 중요한 투자자 입장에서는 지식산업센터가 다시금 투자자에게 선호되고 있는데, 각종 지표나 정부 정책 또한 지식산업센터 투자자에게 긍정적이다.먼저 투자 진입 장벽도 낮다.지식산업센터는 각종 세제 혜택과 함께 규제에서 상대적으로 자유로 워 전매가 가능하고 담보대출 한도 또한 최대 80%까지 가능한 점도 투자자 입장에서 투자 부담이 주거시설보다 낮다.또 한, 분양 가상한 제 등 정부의 주거 시장에 대한 규제 영향도 없어 실수요는 물론 투자자들의 관심이 크다.지식산업센터의 경우 중소기업이 분양 받을 경우 취득세의 50%, 재산세의 37.5%를 감면 받을 수 있어 입주 기업에도 혜택이 크다.2019년이었던 감면 기한이 2022년 12월 31일까지 연장되면서 지식산업센터 분양 시장이 더욱 탄력을 받았다.이런 상황에 가산 디지털 단지 양지사 부지에 새롭게 들어서는 ‘ 가 산 어 반 워크’ 가 분양을 알려 주목받고 있다.서울 금천구 가산동 451-1, 452-1에 들어서는 ‘ 가 산 어 반 워크’ 는 지하 5 층~ 지상 20

In [154]:
summarize(test_df.iloc[4]['contents'])

'임대인 관리와 안정적이고 꾸준한 수익을 기대할 수 있는 지식산업센터 투자에 투자자가 몰리고 있다.\n또한, 최근 정부의 임대차 3법 등으로 인해 매년 임대료 상승을 기대할 수 없어 투자자 입장에서는 투자가치가 낮아졌다.\n따라서 투자자 입장에서는 주택 수에 포함되지 않아 다주택자 규제에서 피할 수 있는 등 다양한 장점이 부각되고 있다.\n지식산업센터는 각종 세제 혜택과 함께 규제에서 상대적으로 자유로워 전매가 가능하고 담보대출 한도 또한 최대 80%까지 가능한 점도 투자자 입장에서 투자 부담이 주거시설보다 낮다.\n해당 단지가 위치한 가산 디지털 3단지의 경우 1·2단지와 비교해 강남순환도로와 서부간선도로, 남부순환로, 시흥대로 등 도로 교통 또한 원활해 도로 접근성에서도 강점을 지니고 있다.'

In [153]:
summarize(article)

''

In [138]:
article

'㈜ 엘시 티피에프 브이 (PFV) 가 개별 분양에 돌입한 상업시설 ‘ 엘시 티 더 몰 (the MALL)’ 의 분양에 순풍이 불고 있다.지난 21일부터 24일까지 공개 청약을 진행한 부산 해운대의 초대형 상업시설 ‘ 엘시 티 더 몰 (the MALL)’ 의 경우 최고 청약 경쟁률이 15대 1를 기록하는 등 분양에 선전하고 있어 주목을 받고 있다.특히 해운대 해수욕장을 조망할 수 있거나 출입구 주변에 위치한 인기 점포의 경우 대부분 분양이 완료됐다.상업시설은 엘 시티 지상 1~3 층에 8만 3790㎡ 규모로 조성되며, 개별 분양으로 진행된다.청약 전 일주일간 분양의 향서를 접수한 결과 700 여 명이 몰리면서 조기 완 판에 대한 기대감을 높이기도 했다.‘ 엘시 티 더 몰 (the MALL)’ 은 부산 광역시 해운대구 일원에 지상 1 층~ 지상 3 층, 연면적 8만 3790㎡ 규모로 조성되는 상업시설로, 국내에서 두 번째로 높은 초고층 주상 복합단지 엘 시티에 들어서 일대 랜드 마크로 자리잡을 것으로 보여 진다.상업시설은 해운대 해수욕장과 인접해 있으며, 이러한 강점을 적극 활용한 매장 구성과 설계를 선보인다.먼저, 해운대를 바라보는 상업시설 전면 부에는 오션 뷰가 가능한 통창이 적용돼 해운대 방문객들의 접근성을 높였다.1 층에는 오션 뷰 데크를, 2~3 층에 위치한 점포의 경우에는 오션 뷰 테라스를 적용해 해운대 오션 뷰를 극대화했다.또 한, 상업시설 앞에 위치한 광장 및 해수욕장과의 연결을 고려한 출입구 배치로 일대 유동인구를 흡수할 수 있을 것으로 보인다.상업시설 내에는 층별 색다른 경험을 위한 리딩 테넌트 유치가 예정돼 있음은 물론 집 객력이 큰 앵커 테넌트들도 계획돼 있다.현재 엘 시티 98 층 ~101 층에는 부산 최고층 전망대 ‘ 부산 엑스 더 스카이 (BUSAN X the SKY)’ 가 운영 중이며, 4 층 ~6 층에는 국내 최대 규모의 인 피니 티 풀과 온천수를 활용해 사계절 내내 이용 가능한 도심형 워터 파크가 내년 중 상업시설과 함께 개장 예

In [113]:
summarize(article, words=20)

''

In [98]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 10027 to 15032
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   date      5000 non-null   object
 1   title     5000 non-null   object
 2   contents  5000 non-null   object
 3   press     5000 non-null   object
 4   good      5000 non-null   int64 
 5   well      5000 non-null   int64 
 6   sad       5000 non-null   int64 
 7   mad       5000 non-null   int64 
 8   want      5000 non-null   int64 
 9   summary   2567 non-null   object
dtypes: int64(5), object(5)
memory usage: 429.7+ KB


In [69]:
# test_df[test_df['summary'].isna()]
test_df.iloc[0]['summary'].split('\n')

['‘엘시티 더몰(the MALL)’은 해운대 방문객 수요와 함께 엘시티 내 마련되는 ‘해운대 엘시티 더샵(882가구)’, ‘엘시티 레지던스(561실)’와 ‘시그니엘 부산(260실)’ 등을 고정 수요로 누릴 예정이며, 가까이에 위치한 마린시티 약 51,500세대의 배후수요도 품을 것으로 보인다.',
 '해운대의 ‘엘시티 더몰(the MALL)’ 역시 부산 초고층 마천루 엘시티 내에 조성되며, 개장과 동시에 지역경제 활성화를 리딩하는 랜드마크 상업시설의 역할을 수행할 것으로 보여진다.분양관계자는 “사전에 분양된 엘시티 내 주거시설이 모두 완판을 기록하며, 이번에 개별 분양되는 엘시티 더몰(the MALL)에도 투자자들의 관심이 끊이질 않고 있다”며 “차후 상업시설이 개장하면 엘시티 내 여러 부대시설과 함께 해운대를 넘어 부산을 대표하는 랜드마크 시설로 자리잡을 것”이라고 전했다.한편, 프리미엄 상업시설 ‘엘시티 더몰(the MALL)’의 홍보관은 부산광역시 해운대구에 위치한다.']

In [87]:
# sentence = text2sentences(test_df['contents'][0])
# kkma.sentences(test_df['contents'][0])
summary = summarize(test_df['contents'][1])
summary

''

In [86]:
test_df['contents'][0]

'㈜엘시티피에프브이(PFV)가 개별분양에 돌입한 상업시설 ‘엘시티 더몰(the MALL)’의 분양에 순풍이 불고 있다.지난 21일부터 24일까지 공개청약을 진행한 부산 해운대의 초대형 상업시설 ‘엘시티 더몰(the MALL)’의 경우 최고 청약경쟁률이 15대 1를 기록하는 등 분양에 선전하고 있어 주목을 받고 있다. 특히 해운대해수욕장을 조망할 수 있거나 출입구 주변에 위치한 인기 점포의 경우 대부분 분양이 완료됐다.상업시설은 엘시티 지상 1~3층에 8만3790㎡ 규모로 조성되며, 개별 분양으로 진행된다. 청약 전 일주일간 분양의향서를 접수한 결과 700여명이 몰리면서 조기 완판에 대한 기대감을 높이기도 했다.‘엘시티 더몰(the MALL)’은 부산광역시 해운대구 일원에 지상 1층~지상 3층, 연면적 8만 3790㎡ 규모로 조성되는 상업시설로, 국내에서 두번째로 높은 초고층 주상복합단지 엘시티에 들어서 일대 랜드마크로 자리잡을 것으로 보여진다.상업시설은 해운대 해수욕장과 인접해 있으며, 이러한 강점을 적극 활용한 매장 구성과 설계를 선보인다. 먼저, 해운대를 바라보는 상업시설 전면부에는 오션뷰가 가능한 통창이 적용돼 해운대 방문객들의 접근성을 높였다. 1층에는 오션뷰 데크를, 2~3층에 위치한 점포의 경우에는 오션뷰 테라스를 적용해 해운대 오션뷰를 극대화했다. 또한, 상업시설 앞에 위치한 광장 및 해수욕장과의 연결을 고려한 출입구 배치로 일대 유동인구를 흡수할 수 있을 것으로 보인다.상업시설 내에는 층별 색다른 경험을 위한 리딩테넌트 유치가 예정돼 있음은 물론 집객력이 큰 앵커테넌트들도 계획돼 있다. 현재 엘시티 98층~101층에는 부산 최고층 전망대 ‘부산 엑스 더 스카이(BUSAN X the SKY)’가 운영 중이며, 4층~6층에는 국내 최대 규모의 인피니티풀과 온천수를 활용해 사계절 내내 이용 가능한 도심형 워터파크가 내년 중 상업시설과 함께 개장 예정이다. 롯데호텔이 부산 최초로 오픈한 6성급 ‘시그니엘 부산’ 역시 호텔웨딩, 연회장 등이 마련돼, 엘시

In [38]:
test_df['contents'][2]

'[이데일리 권소현 기자] 올해 글로벌 리츠 변동성이 확대될 것으로 예상되는 가운데 넷리스 리츠에 주목할 필요가 있다는 분석이 나왔다. 낙폭이 컸던 만큼 가격매력이 있는데다 배당을 늘릴 것이란 예상에서다. 넷리스(Net Lease) 리츠는 임차인이 유지보수비, 재산세, 보험료 중 일부를 부담하는 순임대 구조로 단일 임차인에게 자산을 임대하는 리츠를 말한다. 혼합된 자산을 보유하는 경우가 많으며 자산 매입 후에 매도자에게 재임대하는 것이 일반적이다. 1일 NH투자증권에 따르면 지난해 12월 한 달간 글로벌 리츠 지수는 3.58% 상승했다. 겨울을 맞아 전 세계 코로나19 확산속도가 빨라지고 있지만 미국 재정부양책 협상과 코로나19 백신 공급에 대한 기대로 글로벌 리츠도 상승세를 보인 것이다. 그러나 영국을 중심으로 기존보다 전염력이 70% 이상 강한 변이 코로나19가 빠르게 확산되면서 봉쇄조치가 시행되자 지수 상승폭이 일부 축소된 채 마감했다. 홍지환 NH투자증권 연구원은 저가매력 부각, 배당 회복 가시화, 리플레이션 진입이라는 글로벌 리츠 비중 확대 근거는 여전히 유효하다고 분석했다. 특히 올해 경기회복과 코로나19 사태 정상화가 예상되는 가운데 리츠의 상대적인 투자 매력도가 더 높다고 판단했다. 11월 이후 호텔, 리테일 리츠의 임대수익 정상화에 대한 기대감이 선반영되면서 가격 급등이 나타났지만 올해 상반기까지 실적 및 배당공백, 높아진 밸류에이션으로 인해 당분간 가격 변동성이 확대될 수 있다고 내다봤다. 이에 따라 낙폭과대 섹터 내에서도 배당금 증액과 회복이 가시화된 주거용, 헬스케어, 카지노, 넷리스 리츠에 우선 접근하는 전략이 유효하다고 설명했다. 특히 NH투자증권은 미국 혼합형 리츠 WP Carey(WPC US)를 추천했다. 이 리츠는 미국과 유럽에 산업용, 오피스, 리테일 부동산 자산을 혼합 보유한 미국혼합형 리츠인 동시에 순임대 계약 구조를 사용하는 넷리스 리츠다. 넷리스 리츠는 임대수익을 높이기 위해 자산 인수를 적극 활용하기 때문에 현재의 저금

In [88]:
summa

['역시 상승 출발 이후 하락 전환해 낙폭을 키워 마감했다.코스닥에서는 개인과 기관이 각각 111억원, 127억원을 순매도한 반면 외국인만 332억원을 순매수했다.코스닥 시가총액 상위 10개 종목 가운데 셀트리온헬스가 7%대로 낙폭이 가장 컸다.',
 "수술비 담보 특약을 업계 최대수준인 140대 질병으로 확대하는 한편, 대리청구인제도를 개선해 보험금 청구의 불편을 해소한다.롯데손해보험은 '렛:스마일 종합건강보험(더끌림 프리미엄)'과 '렛:플레이 자녀보험(도담도담)' 보험서비스(상품)를 개정하고 140대 특정질병 수술비 담보를 신설했다고 15일 밝혔다.",
 '소모품에 대해서는 “상반기 소모품 매출액 80% 수준이 고마진 HIFU 카트리지에서 발생했다”며 “3분기부터 RF 팁 매출 본격화 및 시술 확대 효과가 더해질 전망”이라고 했다.',
 nan,
 '',
 nan,
 nan,
 'KB국민은행은 부동산 정보 플랫폼 리브부동산(Liiv부동산) 앱 다운로드 수가 200만 건을 돌파했다고 28일 밝혔다.',
 nan,
 '시중은행의 대출 금리가 1년 새 1%포인트 가까이 뛴 가운데 한국은행의 기준금리 인상이 초읽기에 들어가면서 가계의 이자 부담이 더 커질 것으로 전망된다.',
 nan,
 nan,
 nan,
 nan,
 nan,
 '이는 2차전지 관련 기업이 정부의 뒷받침 아래 투자와 수주가 늘어날 것이란 기대감이 크게 작용한 것으로 보인다.투자 자금이 이처럼 친환경주로 몰리는 데는 최근 플랫폼주에 대한 정부 규제의 반사이익 효과라는 해석이 지배적이다.',
 '',
 '',
 nan,
 '',
 '금융당국은 또 개인별 총부채원리금상환비율(DSR) 규제가 전면 시행되는 2023년 7월부터는 이들 비주거용 부동산에 대해서도 개인 단위 DSR을 적용한다는 계획이다.그동안 금융당국은 비주택 담보대출 증가율이 비교적 높은 상호금융에 대해서만 행정지도로 LTV 70% 규제를 시행해왔다.',
 nan,
 nan,
 '[서울=뉴시스] 옥승욱 기자 = 삼성물산 합병의혹 관련 이재

In [105]:
!pip install summa

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for summa: filename=summa-1.2.0-py3-none-any.whl size=54411 sha256=8786f8cfa710806a9c0173e9e3d41354bb9ed3e11e699e8235322d0d2818594a
  Stored in directory: c:\users\dmaqp\appdata\local\pip\cache\wheels\fd\6a\dd\209eb19d5f2266b9cfd06827539bf70435b0ad5fe8244e52d3
Successfully built summa


In [112]:

summarize(test_df['contents'][202779])

'대리청구인 지정을 원활하게 하기 위해 자격·제출서류 요건을 완화하고 제도안내와 알림서비스를 강화하기로 했다.롯데손해보험 관계자는 "고객이 보험서비스를 통해 얻는 가치를 향상시키기 위해 지속적인 보험서비스 개선과 고객 편의 강화 조치들을 시행해 나갈 것"이라며 "앞으로도 보험업 본연의 경쟁력을 강화해나가기 위해 끊임없이 노력하겠다"고 말했다.기하영 기자 hykii@asiae.co.kr'

In [113]:
from gensim.summarization.summarizer import summarize
summarize(test_df['contents'][202779])

"수술비 담보 특약을 업계 최대수준인 140대 질병으로 확대하는 한편, 대리청구인제도를 개선해 보험금 청구의 불편을 해소한다.롯데손해보험은 '렛:스마일 종합건강보험(더끌림 프리미엄)'과 '렛:플레이 자녀보험(도담도담)' 보험서비스(상품)를 개정하고 140대 특정질병 수술비 담보를 신설했다고 15일 밝혔다."

In [114]:
test_df['contents'][202779]

'[아시아경제 기하영 기자]롯데손해보험이 주력 보험서비스인 장기보험의 경쟁력을 높이기 위해 보험서비스별 담보 특약을 강화한다. 수술비 담보 특약을 업계 최대수준인 140대 질병으로 확대하는 한편, 대리청구인제도를 개선해 보험금 청구의 불편을 해소한다.롯데손해보험은 \'렛:스마일 종합건강보험(더끌림 프리미엄)\'과 \'렛:플레이 자녀보험(도담도담)\' 보험서비스(상품)를 개정하고 140대 특정질병 수술비 담보를 신설했다고 15일 밝혔다. 기존 건강보험 보험서비스가 보장하던 64대 질병 수술비에 더해, 76개 질환에 대한 수술비에도 보장을 제공한다. 렛:플레이 자녀보험(도담도담)의 경우 갑상선암·기타피부암·유사암진단비에 대한 납입면제 담보도 추가했다.롯데손보는 \'렛:드라이브 운전자보험\' 에도 가사도우미를 지원하는 현물급부(특약)를 신설했다. 자동차사고 등으로 부상과 후유장해를 입은 고객이 보험을 통해 가사에 도움을 받을 수 있도록 한 것이다. 대리청구인제도 역시 개선했다. 롯데손보는 치매보험 가입자의 보험금 청구불능을 막기 위해 대리청구인 지정을 의무화하고, 지정가능 인원을 1명에서 복수로 확대한다. 대리청구인 지정을 원활하게 하기 위해 자격·제출서류 요건을 완화하고 제도안내와 알림서비스를 강화하기로 했다.롯데손해보험 관계자는 "고객이 보험서비스를 통해 얻는 가치를 향상시키기 위해 지속적인 보험서비스 개선과 고객 편의 강화 조치들을 시행해 나갈 것"이라며 "앞으로도 보험업 본연의 경쟁력을 강화해나가기 위해 끊임없이 노력하겠다"고 말했다.기하영 기자 hykii@asiae.co.kr'

In [35]:
test_df['summary']

KeyError: 'summary'

In [47]:
test_df['summary'] = np.nan

builtin_function_or_method

In [66]:
sentence = text2sentences(test_df['contents'][369713])
if len(sentence) > 10 :
    test_df[test_df['contents'] == test_df['contents'][369713]]['summary'] = summarize(test_df['contents'][369713], word_count=40)
test_df

<ipython-input-66-1610a03a6f99>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df[test_df['contents'] == test_df['contents'][369713]]['summary'] = summarize(test_df['contents'][369713], word_count=40)


,date,title,contents,press,good,well,sad,mad,want,summary
369713,2021.10.29. 오후 3:59,"코스피, 외인·기관 동반 매도에 2970선까지 밀려",코스피가 외국인과 기관의 '팔자'에 2970선까지 주저 앉았다. 개인이 홀로 물량을...,http://www.inews24.com/,0,0,0,0,0,NaN
202779,2021.06.15. 오전 10:29,"롯데손보, 주력 상품 경쟁력 높인다…""수술비 담보 특약 확대""",[아시아경제 기하영 기자]롯데손해보험이 주력 보험서비스인 장기보험의 경쟁력을 높이기...,https://www.asiae.co.kr/,0,0,0,0,0,NaN
276029,2021.08.11. 오전 7:58,"제이시스메디칼, 접속초음파·고주파로 고성장 기대…목표가 ↑-대신","대신증권에 따르면 제이시스메디칼은 2분기 매출액 200억원, 영업이익 62억원을 기...",http://www.edaily.co.kr/,0,0,0,0,0,NaN
224019,2021.08.24. 오후 5:20,"금호석유화학, 폐플라스틱 재활용 사업 진출",금호석화는 기존의 물리적 재활용 방식에서 벗어나 열분해 등 화학적 재활용 방식을 활...,http://www.hankyung.com/,0,0,0,0,0,NaN
128245,2021.04.19. 오후 12:03,"새희망홀씨 대출, 올해 3.5조원 공급",[헤럴드경제=김성훈 기자] 서민들을 위한 신용대출상품인 새희망홀씨 대출이 올해 3조...,http://www.heraldbiz.com/,0,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...
169851,2021.05.20. 오전 9:09,[인사] 한국예탁결제원,▲전략기획본부장 최정철,http://www.newsis.com,0,0,0,0,0,NaN
145372,2021.06.04. 오후 3:04,"셀트리온-이셀, 국산화 바이오 부품 장기 공급계약 체결",셀트리온은 바이오 의약품 소부장(소재·부품·장비) 전문 업체 이셀과 본사 대회의실에...,http://www.dailian.co.kr/,0,0,0,0,0,NaN
156332,2021.05.10. 오후 7:57,"[스톡톡스]“코스피 최고치, 美부양책·외국인 유입…차익실현 가능성도”",[이데일리 이은정 기자] “예상치를 하회한 미국 고용지표에 따른 테이퍼링(자산매입 ...,http://www.edaily.co.kr/,0,0,0,0,0,NaN
104015,2021.04.01. 오전 10:28,"은성수 금융위원장, 금소법 관련 은행장 간담회",(서울=뉴스1) 김진환 기자 = 은성수 금융위원장이 1일 서울 중구 은행연합회 회의...,http://news1.kr/,0,0,0,0,0,NaN


In [61]:
test_df[test_df['contents'] == test_df['contents'][369713]]['summary']

369713   NaN
Name: summary, dtype: float64

In [30]:
sentence = text2sentences(test_df['contents'][369713])

In [31]:
len(sentence)

23

In [18]:
sentence = text2sentences(test_df['contents'][20133])
nouns = get_nouns(sentence)
sent_graph = build_sent_graph(sentence)
word_graph, idx2word = bulid_word_graph(nouns)

sent_rank_idx = get_rank(sent_graph)
sorted_sent_rank_idx = sorted(sent_rank_idx, key=lambda k : sent_rank_idx[k], reverse=True)

word_rank_idx = get_rank(word_graph)
sorted_word_rank_idx = sorted(word_rank_idx, key=lambda k : word_rank_idx[k], reverse=True)

summarize()
key_word()
print()

TypeError: summarize() missing 1 required positional argument: 'text'

In [24]:
sentences = kkma.sentences(test_df['contents'][20133])
sentences

['[ 파이낸셜 뉴스] 조선업계가 지난해 코로나 19 위 기 속에서도 글로벌 수주 량 1위를 기록했지만 수익성은 그 만큼 회복되기 어려울 것으로 보인다.',
 '연말 수주 량을 늘렸지만 그만큼 싼 가격에 선박을 판매했기 때문이다.',
 '하지만 새해 들어 선박 가격이 오를 조짐이 보이며 올해는 제 가격을 받고 팔 수 있다는 전망도 나오고 있다.',
 '25일 관련 업계 및 영국의 조선· 해운 분석기관인 클락 슨 리 서치에 따르면 이날 기준으로 신조선 가 지수는 지난해 12월 125.60 포인트보다 소폭 오른 126.55 포인트를 기록했다.',
 '신조선 가지수란 전세계 신규 건조 선박 가격을 평균지수화 한 것으로 신조선 가지 수가 상승하면 선박가격이 올랐다는 의미다.',
 '신조선 가는 매년 꾸준하게 상승세를 보이다 지난해 코로나 19 여파로 확연하게 하락세로 바뀌었다.',
 '지난 2019년 까지만 해도 12월 기준 129.77 포인트를 기록하던 신조선 가 지수는 지난해 12월 125.6을 기록했다.',
 '다만 새해 들어 소폭 상승하며 반등 조짐을 보이고 있다.',
 '이 같은 가격 하락으로 국내 대형 조선 3 사( 한국 조선 해양· 대우 조선 해양· 삼성 중공업) 는 실제 지난해 12월 수주가 몰렸음에도 이 기간을 포함한 2020년 4·4 분기 실적이 크게 개선되지 않을 것으로 보인다.',
 '목표 수주 량을 채우기 급급했던 조선사들이 막판에 할 일을 해서 라도 수주 량을 늘리기 분주했기 때문이다.',
 '금융정보업체 에프 앤 가이드가 전망한 컨 센서스( 실적 예상치 )에 따르면 전년 동기와 비교해 지난해 4·4 분기 한국 조선 해양의 영업이익은 63.4% 가 줄어들 것으로 보인다.',
 '같은 기간을 비교해 대우 조선 해양이 영업이익도 69.8% 감소할 전망이며, 삼성 중공업은 적자의 폭은 줄어들었지만 여전히 적자가 축적될 것으로 예상된다.',
 '지난해 한국 조선 해양은 연간 목표 수주 량의 90.5%, 대우 조선 해양은 75%, 삼성 중공업은 65.

In [25]:
article = ''
for i in sentences : 
    a = i.strip()
    article += a
article

'[ 파이낸셜 뉴스] 조선업계가 지난해 코로나 19 위 기 속에서도 글로벌 수주 량 1위를 기록했지만 수익성은 그 만큼 회복되기 어려울 것으로 보인다.연말 수주 량을 늘렸지만 그만큼 싼 가격에 선박을 판매했기 때문이다.하지만 새해 들어 선박 가격이 오를 조짐이 보이며 올해는 제 가격을 받고 팔 수 있다는 전망도 나오고 있다.25일 관련 업계 및 영국의 조선· 해운 분석기관인 클락 슨 리 서치에 따르면 이날 기준으로 신조선 가 지수는 지난해 12월 125.60 포인트보다 소폭 오른 126.55 포인트를 기록했다.신조선 가지수란 전세계 신규 건조 선박 가격을 평균지수화 한 것으로 신조선 가지 수가 상승하면 선박가격이 올랐다는 의미다.신조선 가는 매년 꾸준하게 상승세를 보이다 지난해 코로나 19 여파로 확연하게 하락세로 바뀌었다.지난 2019년 까지만 해도 12월 기준 129.77 포인트를 기록하던 신조선 가 지수는 지난해 12월 125.6을 기록했다.다만 새해 들어 소폭 상승하며 반등 조짐을 보이고 있다.이 같은 가격 하락으로 국내 대형 조선 3 사( 한국 조선 해양· 대우 조선 해양· 삼성 중공업) 는 실제 지난해 12월 수주가 몰렸음에도 이 기간을 포함한 2020년 4·4 분기 실적이 크게 개선되지 않을 것으로 보인다.목표 수주 량을 채우기 급급했던 조선사들이 막판에 할 일을 해서 라도 수주 량을 늘리기 분주했기 때문이다.금융정보업체 에프 앤 가이드가 전망한 컨 센서스( 실적 예상치 )에 따르면 전년 동기와 비교해 지난해 4·4 분기 한국 조선 해양의 영업이익은 63.4% 가 줄어들 것으로 보인다.같은 기간을 비교해 대우 조선 해양이 영업이익도 69.8% 감소할 전망이며, 삼성 중공업은 적자의 폭은 줄어들었지만 여전히 적자가 축적될 것으로 예상된다.지난해 한국 조선 해양은 연간 목표 수주 량의 90.5%, 대우 조선 해양은 75%, 삼성 중공업은 65.5%를 달성했다.업계에선 올해부턴 서서히 신조선 가가 바닥을 치고 일어나 제 값을 받고 선박을 팔 수 있을 것

In [28]:
type(article)

str

In [30]:
print(summarize(article, word_count=30))

ValueError: input must have more than one sentence

In [23]:
test_df['contents'][20133].strip()

'[파이낸셜뉴스] 조선업계가 지난해 코로나19 위기 속에서도 글로벌 수주량 1위를 기록했지만 수익성은 그 만큼 회복되기 어려울 것으로 보인다. 연말 수주량을 늘렸지만 그만큼 싼 가격에 선박을 판매했기 때문이다. 하지만 새해 들어 선박 가격이 오를 조짐이 보이며 올해는 제 가격을 받고 팔 수 있다는 전망도 나오고 있다.   25일 관련 업계 및 영국의 조선·해운 분석기관인 클락슨리서치에 따르면 이날 기준으로 신조선가 지수는 지난해 12월 125.60포인트보다 소폭 오른 126.55포인트를 기록했다. 신조선가지수란 전세계 신규건조 선박 가격을 평균지수화한 것으로 신조선가지수가 상승하면 선박가격이 올랐다는 의미다.   신조선가는 매년 꾸준하게 상승세를 보이다 지난해 코로나19 여파로 확연하게 하락세로 바뀌었다. 지난 2019년 까지만 해도 12월 기준 129.77포인트를 기록하던 신조선가 지수는 지난해 12월 125.6을 기록했다. 다만 새해 들어 소폭 상승하며 반등 조짐을 보이고 있다.   이 같은 가격 하락으로 국내 대형 조선 3사(한국조선해양·대우조선해양·삼성중공업)는 실제 지난해 12월 수주가 몰렸음에도 이 기간을 포함한 2020년 4·4분기 실적이 크게 개선되지 않을 것으로 보인다. 목표수주량을 채우기 급급했던 조선사들이 막판에 할일을 해서라도 수주량을 늘리기 분주했기 때문이다.   금융정보업체 에프앤가이드가 전망한 컨센서스(실적 예상치)에 따르면 전년 동기와 비교해 지난해 4·4분기 한국조선해양의 영업이익은 63.4%가 줄어들 것으로 보인다. 같은 기간을 비교해 대우조선해양이 영업이익도 69.8% 감소할 전망이며, 삼성중공업은 적자의 폭은 줄어들었지만 여전히 적자가 축적될 것으로 예상된다. 지난해 한국조선해양은 연간 목표수주량의 90.5%, 대우조선해양은 75%, 삼성중공업은 65.5%를 달성했다.   업계에선 올해부턴 서서히 신조선가가 바닥을 치고 일어나 제 값을 받고 선박을 팔 수 있을 것으로 보고 있다. 코로나19로 멈췄던 발주물량이 지난해 하반기 

In [29]:
type(punctuation_text)

str

In [ ]:
def text_clean(doc):
    doc =re.sub("[^a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣]", "", doc)
    return doc

def stopwords_del(x) :

In [26]:
punctuation_text = test_df['contents'][369713].replace("니다", "니다.") #습니다. 입니다.
# punctuation_text = text_clean(punctuation_text)
# punctuation_text = stopwords_del(punctuation_text)
print(summarize(punctuation_text, word_count=40))

개인만 홀로 1조2천559억원을 순매수했다.시가총액 상위 10개 종목 가운데 카카오를 제외하고 전부 내림세를 보였다.
역시 상승 출발 이후 하락 전환해 낙폭을 키워 마감했다.코스닥에서는 개인과 기관이 각각 111억원, 127억원을 순매도한 반면 외국인만 332억원을 순매수했다.코스닥 시가총액 상위 10개 종목 가운데 셀트리온헬스가 7%대로 낙폭이 가장 컸다.


In [27]:
test_df['contents'][369713]

"코스피가 외국인과 기관의 '팔자'에 2970선까지 주저 앉았다. 개인이 홀로 물량을 담아냈지만 지수를 올리기에는 역부족이었다.29일 코스피 지수는 전 거래일 대비 38.87포인트(1.29%) 떨어진 2970.68로 마감했다. 이날 코스피는 상승 출발했지만 오전 9시30분을 기점으로 하락 반전한 뒤 낙폭을 키웠다.수급별로 보면 외국인과 기관은 각각 8천61억원, 4천759억원을 순매도했다. 개인만 홀로 1조2천559억원을 순매수했다.시가총액 상위 10개 종목 가운데 카카오를 제외하고 전부 내림세를 보였다. SK하이닉스가 3%대 하락으로 낙폭이 가장 컸다. 이외 삼성전자, 네이버, LG화학, 삼성바이오로직스, 삼성SDI, 기아 등이 1% 넘게 떨어졌다. 카카오는 보합을 유지했다.업종별로 보면 비금속광물이 1% 넘게 오른 것을 제외하고 전 업종이 하락했다. 의약품이 3% 넘게 하락했고 보험업과 은행이 2%대의 약세를 보였다. 또한 전기전자, 제조업, 음식료업, 금융업, 운수창고 등이 1% 넘게 하락했다.코스닥 지수도 전 거래일 대비 7.80포인트(0.78%) 하락한 992.33으로 장을 마쳤다. 역시 상승 출발 이후 하락 전환해 낙폭을 키워 마감했다.코스닥에서는 개인과 기관이 각각 111억원, 127억원을 순매도한 반면 외국인만 332억원을 순매수했다.코스닥 시가총액 상위 10개 종목 가운데 셀트리온헬스가 7%대로 낙폭이 가장 컸다. 셀트리온제약이 5% 넘게 떨어졌고 펄어비스, 에이치엘비, SK머티리얼즈가 1% 이상 떨어졌다. 반면 위메이드는 14% 이상 올랐고 에코프로비엠과 카카오게임즈도 1%대 약세를 보였다.업종별로 보면 유통이 3%대 하락세를 나타냈다. 제약, 신성장기업은 2% 넘게 내렸다. 섬유·의류, 정보기기, 금속, 운송·부품, 반도체, 제조 등은 1% 이상 떨어졌다.이날 서울 외환시장에서 원/달러 환율은 전 거래일 대비 1.10원(0.09%) 내린 1168.60원으로 마감했다."

In [36]:
punctuation_text

'파이낸셜뉴스조선업계가지난해코로나위기속에서도글로벌수주량위를기록했지만수익성은그만큼회복되기어려울것으로보인다연말수주량을늘렸지만그만큼싼가격에선박을판매했기때문이다하지만새해들어선박가격이오를조짐이보이며올해는제가격을받고팔수있다는전망도나오고있다일관련업계및영국의조선해운분석기관인클락슨리서치에따르면이날기준으로신조선가지수는지난해월포인트보다소폭오른포인트를기록했다신조선가지수란전세계신규건조선박가격을평균지수화한것으로신조선가지수가상승하면선박가격이올랐다는의미다신조선가는매년꾸준하게상승세를보이다지난해코로나여파로확연하게하락세로바뀌었다지난년까지만해도월기준포인트를기록하던신조선가지수는지난해월을기록했다다만새해들어소폭상승하며반등조짐을보이고있다이같은가격하락으로국내대형조선사한국조선해양대우조선해양삼성중공업는실제지난해월수주가몰렸음에도이기간을포함한년분기실적이크게개선되지않을것으로보인다목표수주량을채우기급급했던조선사들이막판에할일을해서라도수주량을늘리기분주했기때문이다금융정보업체에프앤가이드가전망한컨센서스실적예상치에따르면전년동기와비교해지난해분기한국조선해양의영업이익은가줄어들것으로보인다같은기간을비교해대우조선해양이영업이익도감소할전망이며삼성중공업은적자의폭은줄어들었지만여전히적자가축적될것으로예상된다지난해한국조선해양은연간목표수주량의대우조선해양은삼성중공업은를달성했다업계에선올해부턴서서히신조선가가바닥을치고일어나제값을받고선박을팔수있을것으로보고있다코로나로멈췄던발주물량이지난해하반기부터서서히풀리고환경규제에따른액화천연가스LNG선친환경선박에대한수요가늘고있기때문이다실제친환경선박에대한수요는전세계선박발주량을늘리는데한몫할것으로보인다클락슨은올해세계선박발주량이지난해보다증가할것으로예상했다'

In [21]:
import subprocess
import os
from gensim.summarization.summarizer import summarize

C:\anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [105]:
!pip install gensim

In [106]:
from gensim.summarization.summarizer import summarize

ERROR: Could not install packages due to an OSError: [WinError 5] 액세스가 거부되었습니다: 'C:\\anaconda3\\Lib\\site-packages\\~ensim\\corpora\\_mmreader.cp38-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for gensim: filename=gensim-3.4.0-cp38-cp38-win_amd64.whl size=22293577 sha256=a29f6df3becd5541642900ad0b08c246e8c4cf2bf70f453855811360489e23d2
  Stored in directory: c:\users\dmaqp\appdata\local\pip\cache\wheels\b4\a4\71\a301cdb2b7d5d31525936fcb8dcd9a5f144578d047407f7cf9
Successfully built gensim
  Attempting uninstall: gensim
    Found existing installation: gensim 4.1.2
    Uninstalling gensim-4.1.2:
      Successfully uninstalled gensim-4.1.2


In [ ]:
punctuation_text = text_stacked.replace("니다", "니다.") #습니다. 입니다.
punctuation_text

In [57]:
sent_rank_idx = get_rank(sent_graph)
sorted_sent_rank_idx = sorted(sent_rank_idx, key=lambda k : sent_rank_idx[k], reverse=True)

word_rank_idx = get_rank(word_graph)
sorted_word_rank_idx = sorted(word_rank_idx, key=lambda k : word_rank_idx[k], reverse=True)

In [63]:
summarize()

[ 이 데일리 김 다솔 인턴기자] 비트 코 인의 사상 최고 치 경신에 힘입어 암호 화폐 채굴업체 ‘ 스트롱 홀드 디지털 마이닝( 이하 스트롱 홀드)’ 이 상장 첫날 급등했다.
예상주가가 16~18 달러( 약 1만 8000~2 만 1000원) 였던 스트롱 홀드는 기업공개 (IPO) 때 19 달러( 약 2만 2000원) 로 책정됐다.
그 렉 비어드 스트롱 홀드 최고 경영자 (CEO) 는 “ 펜실 베니 아의 오랜 탄광 문제 개선을 위해 노력하고 있다 ”며 “ 비트 코인 채굴은 이 문제를 가장 경제적으로 해결하는 방법” 이라고 밝혔다.


In [122]:
from konlpy.tag import Mecab

In [123]:
mecab = Mecab()

Exception: The MeCab dictionary does not exist at "/usr/local/lib/mecab/dic/mecab-ko-dic". Is the dictionary correctly installed?
You can also try entering the dictionary path when initializing the Mecab class: "Mecab('/some/dic/path')"

In [24]:
stopwords = pd.read_csv('stopwords_kr.txt',header=None)
stopwords.columns = ['word']

In [42]:
stopwords[stopwords['word'] == '데일리']

,word
1,데일리


In [30]:
text_clean(test_df['contents'][357314])

'이데일리 김다솔 인턴기자 비트코인의 사상 최고치 경신에 힘입어 암호화폐 채굴업체 스트롱홀드 디지털마이닝이하 스트롱홀드이 상장 첫날 급등했다 일현지시간 스트롱홀드는 이날 나스닥에서 공모가대비  급등한 가격에 거래를 마치며 성공적으로 데뷔했다 스트롱홀드는 폐석탄을 재활용해 전력을 생산하는 친환경 가상화폐 채굴기업이다예상주가가 달러약 만만원였던 스트롱홀드는 기업공개IPO 때 달러약 만원로 책정됐다 첫날  오른 달러약 만원로 거래가 시작돼 달러약 만원까지 치솟은 뒤 소폭 하락한 달러약 만원에 장을 마감했다 시가총액은 약 억달러약 조억원다CNBC는 스트롱홀드의 적절한 상장 시기가 주가를 끌어올렸다고 평가했다 비트코인 신고가 기록일과 얼마 전 있었던 미 증권거래소SEC의 비트코인 선물 ETF 승인 등에 시기가 맞물려 투자자들의 주목을 받았다는 설명이다친환경 채굴 기업이라는 점도 스트롱홀드가 주목 받는 이유 중 하나로 꼽힌다 그렉 비어드 스트롱홀드 최고경영자CEO는 펜실베니아의 오랜 탄광 문제 개선을 위해 노력하고 있다며 비트코인 채굴은 이 문제를 가장 경제적으로 해결하는 방법이라고 밝혔다 년대 후반부터 석탄 채굴 사업에 집중한 펜실베니아는 탄광에서 나오는 부산물 문제로 골치를 썩고 있다한편 이날 비트코인은 한때 만달러약 만원에 근접하며 종전 최고가인 지난 월의 만달러약 만원를 경신했다 또한 비트코인 선물 ETF가 뉴욕증권거래소NYSE 거래되기 시작하면서 시장에는 향후 현물 ETF 등이 승인될 수 있다는 기대감이 높다'

In [7]:
test_df['contents'][0]

'㈜엘시티피에프브이(PFV)가 개별분양에 돌입한 상업시설 ‘엘시티 더몰(the MALL)’의 분양에 순풍이 불고 있다.지난 21일부터 24일까지 공개청약을 진행한 부산 해운대의 초대형 상업시설 ‘엘시티 더몰(the MALL)’의 경우 최고 청약경쟁률이 15대 1를 기록하는 등 분양에 선전하고 있어 주목을 받고 있다. 특히 해운대해수욕장을 조망할 수 있거나 출입구 주변에 위치한 인기 점포의 경우 대부분 분양이 완료됐다.상업시설은 엘시티 지상 1~3층에 8만3790㎡ 규모로 조성되며, 개별 분양으로 진행된다. 청약 전 일주일간 분양의향서를 접수한 결과 700여명이 몰리면서 조기 완판에 대한 기대감을 높이기도 했다.‘엘시티 더몰(the MALL)’은 부산광역시 해운대구 일원에 지상 1층~지상 3층, 연면적 8만 3790㎡ 규모로 조성되는 상업시설로, 국내에서 두번째로 높은 초고층 주상복합단지 엘시티에 들어서 일대 랜드마크로 자리잡을 것으로 보여진다.상업시설은 해운대 해수욕장과 인접해 있으며, 이러한 강점을 적극 활용한 매장 구성과 설계를 선보인다. 먼저, 해운대를 바라보는 상업시설 전면부에는 오션뷰가 가능한 통창이 적용돼 해운대 방문객들의 접근성을 높였다. 1층에는 오션뷰 데크를, 2~3층에 위치한 점포의 경우에는 오션뷰 테라스를 적용해 해운대 오션뷰를 극대화했다. 또한, 상업시설 앞에 위치한 광장 및 해수욕장과의 연결을 고려한 출입구 배치로 일대 유동인구를 흡수할 수 있을 것으로 보인다.상업시설 내에는 층별 색다른 경험을 위한 리딩테넌트 유치가 예정돼 있음은 물론 집객력이 큰 앵커테넌트들도 계획돼 있다. 현재 엘시티 98층~101층에는 부산 최고층 전망대 ‘부산 엑스 더 스카이(BUSAN X the SKY)’가 운영 중이며, 4층~6층에는 국내 최대 규모의 인피니티풀과 온천수를 활용해 사계절 내내 이용 가능한 도심형 워터파크가 내년 중 상업시설과 함께 개장 예정이다. 롯데호텔이 부산 최초로 오픈한 6성급 ‘시그니엘 부산’ 역시 호텔웨딩, 연회장 등이 마련돼, 엘시

In [24]:
# for text in test_df['contents'] : 
sentence = kkma.sentences(test_df['contents'][0])
len(sentence)

24

In [26]:
for idx in range(0,len(sentence)) :
    if len(sentence[idx]) <=10 :
        sentence[idx-1] += (' ' + sentence[idx])
        sentence[idx] = ''

In [28]:
len(sentence)

24

In [29]:
df_sampling = df.sample(n=1000, random_state = 1)
df_sampling

,date,title,contents,press,good,well,sad,mad,want
357314,2021.10.21. 오전 10:45,비트코인 급등에 신생 채굴업체 상장 첫날 52%↑,[이데일리 김다솔 인턴기자] 비트코인의 사상 최고치 경신에 힘입어 암호화폐 채굴업체...,http://www.edaily.co.kr/,0,0,0,0,0
53838,2021.03.04. 오후 3:01,"엔씨소프트, 블소2 사전예약 400만 달성",엔씨는 사전예약 400만 달성을 기념해 파사의 선물을 추가 보상으로 지급한다....,http://www.fnnews.com/,0,0,0,0,0
67355,2021.03.04. 오전 11:11,신한금투로 만능통장 계좌 옮기고 백화점 상품권 받자,신한금융투자는 만기가 돌아오는 개인종합자산관리계좌(ISA)를 개인퇴직연금계좌(IRP...,http://www.wowtv.co.kr/,0,0,0,0,0
294753,2021.08.26. 오후 5:22,"미래에셋증권, 계열사 투자조합에 90억원 출자",회사 측은 “계열사 미래에셋캐피탈이 결성하는 투자조합에 미래에셋증권이 유한책임조합원...,http://www.edaily.co.kr/,0,0,0,0,0
43803,2021.02.20. 오후 12:16,"""가습기살균제 가해기업 사과·배상하라""",(서울=연합뉴스) 임헌정 기자 = 가습기살균제 참사 피해자 등이 모인 '가습...,https://www.yna.co.kr/,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
155695,2021.05.10. 오후 3:07,"GS, 1Q 영업익 7064억…전년比 7366%↑",이민우 기자 letzwin@asiae.co.kr,https://www.asiae.co.kr/,0,0,0,0,0
115270,2021.04.08. 오후 6:01,"농협은행, 여자바둑리그 후원",농협은행이 2021 한국여자바둑리그를 후원한다. 장승현 농협은행 수석부행장(왼쪽)과...,http://www.hankyung.com/,0,0,0,0,0
180946,2021.05.28. 오전 11:35,"코퍼스코리아, 1분기 영업익 18억… 전년比 117% 증가",[서울=뉴시스]신항섭 기자 = 한류콘텐츠 전문기업 코퍼스코리아는 올 1분기 영업이익...,http://www.newsis.com,0,0,0,0,0
88371,2021.03.22. 오전 10:35,"""데이터 안심하고 거래해요""… 금융보안원, 원격 데이터분석센터 개점",변혜진 기자 hyejin8@mt.co.kr <저작권자 ⓒ '성공을 꿈꾸는 사람들의...,https://moneys.mt.co.kr/,0,0,0,0,0


In [24]:
twit = Twitter()

C:\anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [27]:
test = sentence[0] + ' ' + sentence[1]

In [11]:
test_df.contents[0]

'[데일리안 = 이충재 기자] 대신밸런스제11호스팩이 오는 2일 코스닥 시장에 상장한다.한국거래소 코스닥시장본부는 30일 "대신밸런스제11호기업인수목적㈜의 코스닥시장 신규상장을 승인했다"며 "매매거래는 2일부터 개시한다"고 밝혔다.'

In [29]:
kkma.nouns(test)

['데',
 '데일리',
 '일',
 '리',
 '안',
 '이',
 '충재',
 '기자',
 '대신',
 '밸런스',
 '11',
 '호스',
 '팩',
 '2',
 '2일',
 '일',
 '코스닥',
 '시장',
 '상장',
 '한국',
 '거래소',
 '코스닥시장본부',
 '본부',
 '30',
 '30일',
 '11호',
 '호',
 '기업',
 '인수',
 '인수목적',
 '목적',
 '의',
 '코스닥시장',
 '신규',
 '승인',
 '매매',
 '매매거래',
 '거래',
 '개시',
 '고']

In [4]:
page_select[-1].text

'이전'

In [18]:
page_select

[]

In [53]:
article_df

,date,title,contents,press,good,well,sad,mad,want
0,2021.01.14. 오후 10:51,"DGB대구은행, 지역 독거노인 고독사 예방 AI로봇 전달",DGB대구은행은 심각한 사회적 문제로 대두되고 있는 독거노인의 고독사를 예방하고 지...,http://www.wowtv.co.kr/,0,0,0,0,0
1,2021.01.14. 오후 8:06,"거래소 ""코스피 밸류에이션 G20 중 낮은 수준""",(서울=뉴스1) 전민 기자 = 한국거래소는 최근 전인미답의 3000선을 넘어선 코스...,http://news1.kr/,0,0,0,0,0
2,2021.01.14. 오후 8:03,개미들 돈보따리 들고 증시로… 올들어 개인자금 16조 증가,올해 들어 국내 주식시장에 들어간 개인 자금이 10여일 만에 16조원에 달한 것...,http://www.segye.com,0,0,0,0,0
3,2021.01.14. 오후 7:39,작년 가계대출 사상최대 112조 증가,"지난해 코로나19에 따른 생활자금 수요와 부동산대출, 주식투자를 위한 대출 등으로 ...",http://www.dt.co.kr/,0,0,0,0,0
4,2021.01.14. 오후 6:53,’빚투’가 유행?…주식계좌·신용대출 절반이 '2030',"청년층에 주식투자 광풍이 불고 있다. 신규 주식계좌와 신용대출의 절반가량이 20, ...",http://www.imaeil.com/,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
1560,2021.01.11. 오전 9:16,"롯데손보, 보험업 전반 디지털 전환 추진",롯데손해보험은 전통적 보험 방식을 뛰어넘는 새로운 디지털 모델로 나아가기 위해 보험...,http://www.dailian.co.kr/,0,0,0,0,0
1561,2021.01.11. 오전 9:15,"윤종규 KB금융 회장 ""완전한 금융플랫폼 기업으로 변모해야""","윤종규 KB금융그룹 회장은 2021년 그룹 경영전락회의에서 ""앞으로 KB는 금융회사...",http://www.hankyung.com/,0,0,0,0,0
1562,2021.01.11. 오전 9:12,"캐롯, '퍼마일자동차보험' 가입계약 수 10만건 돌파",[아시아경제 기하영 기자]캐롯손해보험은 주력상품인 '퍼마일자동차보험' 신규 가입 계...,https://www.asiae.co.kr/,0,0,0,0,0
1563,2021.01.11. 오전 9:10,"'탄 만큼만 낸다' 캐롯 퍼마일자동차보험, 가입자수 10만건 '훌쩍'",[기사제보 news@sportsseoul.com] Copyright ⓒ 스포츠서울&,http://www.sportsseoul.com/,0,0,0,0,0


In [55]:
article_df.to_csv('article_info0.csv', encoding='utf-8-sig')

In [57]:
df = pd.read_csv('금융2021news.csv', index_col=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 109759 entries, 0.0 to nan
Data columns (total 9 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   date      109759 non-null  object
 1   title     109759 non-null  object
 2   contents  106555 non-null  object
 3   press     109759 non-null  object
 4   good      109759 non-null  int64 
 5   well      109759 non-null  int64 
 6   sad       109759 non-null  int64 
 7   mad       109759 non-null  int64 
 8   want      109759 non-null  int64 
dtypes: int64(5), object(4)
memory usage: 8.4+ MB


In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92557 entries, 0 to 92556
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  29331 non-null  float64
 1   date        92557 non-null  object 
 2   title       92557 non-null  object 
 3   contents    89407 non-null  object 
 4   press       92557 non-null  object 
 5   good        92557 non-null  int64  
 6   well        92557 non-null  int64  
 7   sad         92557 non-null  int64  
 8   mad         92557 non-null  int64  
 9   want        92557 non-null  int64  
dtypes: float64(1), int64(5), object(4)
memory usage: 7.1+ MB


In [20]:
main_url = 'https://news.naver.com/main/list.naver?mode=LS2D&sid2=259&page2=&sid1=101&mid=shm&date={}&page={}'.format(search_day, page)
headers = {'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36'}
main_res = requests.get(main_url, headers = headers)
main_soup = BeautifulSoup(main_res.text, 'html.parser')
path = '#main_content > div.list_body.newsflash_body > ul > li'
page_path1 = '#main_content > div.paging > a'
page_path2 = '#main_content > div.paging > strong'
main_select = main_soup.select(path)
page_select = main_soup.select(page_path1)

In [23]:
main_url

'https://news.naver.com/main/list.naver?mode=LS2D&sid2=259&page2=&sid1=101&mid=shm&date=20210228&page=1'

In [199]:
scan_day = pd.datetime(2021,11,10)

<ipython-input-199-f9addf355d76>:1: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  scan_day = pd.datetime(2021,11,10)


In [200]:
str(scan_day.day).zfill(2)

'10'

In [187]:
article_df

,date,title,contents,press,good,well,sad,mad,want
0,2021.11.28. 오후 11:09,해외직구 했는데 연락두절?…카드사에 '차지백' 신청하세요[기하영의 생활 속 카드],"연말이 다가오면서 해외직구로 쇼핑하신 분들 많으시죠? 특히 11월에는 광군제, 블랙...",https://www.asiae.co.kr/,0,0,0,0,0
1,2021.11.28. 오후 10:11,"새마을금고, 가계대출 29일부터 중단…풍선효과 차단",[데일리안 = 김민석 기자] 새마을금고가 가계대출을 일시 중단한다고 29일 밝혔다....,http://www.dailian.co.kr/,0,0,0,0,0
2,2021.11.28. 오후 9:51,"새마을금고, 29일부터 가계대출 한시 중단",대출 상담사를 통한 주택담보대출도 중단한다. 이에 따라 생활안정자금 대출을 포함한 ...,http://www.edaily.co.kr/,0,0,0,0,0
3,2021.11.28. 오후 9:43,"새마을금고, 29일부터 가계대출 판매 한시 중단","새마을금고는 가계 주택 구입 자금 대출, 분양주택 입주 잔금 대출 등의 가계 대출 ...",http://www.wowtv.co.kr/,0,0,0,0,0
4,2021.11.28. 오후 7:38,[커지는 금리압박] 대출금리 급등에도 변동금리 비중 11%p 늘어,시장금리 상승으로 은행 대출금리가 1%포인트(p) 가까이 급등했지만 금리 상승 영향...,http://www.dt.co.kr/,0,0,0,0,0
5,2021.11.28. 오후 7:30,"신한카드·더존비즈온, 법인카드 회원 확대 MOU","신한카드는 더존비즈온과 법인카드 회원을 확대하고, 마케팅을 전개하기 위한 업무협약을...",http://www.dt.co.kr/,0,0,0,0,0
6,2021.11.28. 오후 6:42,껑충 뛴 물가에… 가계대출 총량 증가율 높여 숨통 터주나 [대출규제 소폭 완화 가능성],금융당국이 내년도 가계대출 총량 관리 목표치를 기존에 은행권에 제시한 4~5%보...,http://www.fnnews.com/,0,0,0,0,0
7,2021.11.28. 오후 6:29,"신흥국-선진국 주가 차별화, 언제까지",선진국과 신흥국 주가 차별화가 더 커지고 있다. 팬데믹 공포가 최고조에 달했던 20...,http://www.hani.co.kr/,0,0,0,0,0
8,2021.11.28. 오후 6:05,`기업·경남은행` 은행권 `기술금융` 실적 1위,올해 상반기 은행권 기술금융 실적 평가에서 기업은행과 경남은행이 1위에 올랐다고 금...,http://www.dt.co.kr/,0,0,0,0,0
9,2021.11.28. 오후 6:00,국민銀 후원한 100번째 '작은 도서관' 개관,[서울경제] 김수연(왼쪽 세 번째부터) ㈔작은도서관만드는사람들 대표와 윤도원 KB국...,http://www.sedaily.com,0,0,0,0,0


In [180]:
article__list_df = pd.DataFrame(columns=['link'])
scan_day = pd.datetime(2021,10,31)
page_count = True
page = 1

for a in range(0, 10) : 
    try :
        search_day = str(scan_day.year) + str(scan_day.month) + str(scan_day.day)
        while page_count :
            main_url = 'https://news.naver.com/main/list.naver?mode=LS2D&sid2=259&page2=&sid1=101&mid=shm&date={}&page={}'.format(search_day, page)
            headers = {'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36'}
            main_res = requests.get(main_url, headers = headers)
            main_soup = BeautifulSoup(main_res.text, 'html.parser')
            path = '#main_content > div.list_body.newsflash_body > ul > li'
            page_path1 = '#main_content > div.paging > a'
            page_path2 = '#main_content > div.paging > strong'
            main_select = main_soup.select(path)
            page_select = main_soup.select(page_path1)
            present = main_soup.select(page_path2)
            
            last = page_select[-1].text.isdigit()
            if 20 != len(main_select) :
                print(1)
                page_count = False
#             elif 20 == len(main_select) and prensent[0].text != page_select[-1].text:
#                 print(2)
#                 page_count = True
            elif int(present[0].text) == int(page_select[-1].text)+1  :
#                 print(2)
                page_count = False
            page += 1
            print(main_url)
    except :
        continue
    
    scan_day = scan_day - pd.Timedelta("1day")
    page_count = True
    page = 1
#     print(search_day)

# article_df.to_csv('article_info.csv', encoding='utf-8-sig')

<ipython-input-180-39e714ec8e33>:2: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  scan_day = pd.datetime(2021,10,31)


https://news.naver.com/main/list.naver?mode=LS2D&sid2=259&page2=&sid1=101&mid=shm&date=20211031&page=1
https://news.naver.com/main/list.naver?mode=LS2D&sid2=259&page2=&sid1=101&mid=shm&date=20211031&page=2
https://news.naver.com/main/list.naver?mode=LS2D&sid2=259&page2=&sid1=101&mid=shm&date=20211031&page=3
https://news.naver.com/main/list.naver?mode=LS2D&sid2=259&page2=&sid1=101&mid=shm&date=20211031&page=4
https://news.naver.com/main/list.naver?mode=LS2D&sid2=259&page2=&sid1=101&mid=shm&date=20211031&page=5
https://news.naver.com/main/list.naver?mode=LS2D&sid2=259&page2=&sid1=101&mid=shm&date=20211031&page=6
https://news.naver.com/main/list.naver?mode=LS2D&sid2=259&page2=&sid1=101&mid=shm&date=20211031&page=7
https://news.naver.com/main/list.naver?mode=LS2D&sid2=259&page2=&sid1=101&mid=shm&date=20211031&page=8
https://news.naver.com/main/list.naver?mode=LS2D&sid2=259&page2=&sid1=101&mid=shm&date=20211031&page=9
1
https://news.naver.com/main/list.naver?mode=LS2D&sid2=259&page2=&sid1=1

In [147]:
prensent[0].text == page_select[-1].text

False

In [161]:
int(prensent[0].text) == int(page_select[-1].text)+1

False

In [ ]:
elif 20 == len(main_select) and int(len(page_select))+1 == prensent[0].text:
    page_count = True

In [72]:
int(len(page_select))+1 == prensent[0].text

False

In [77]:
int(page_select[-1].text)+1 == int(prensent[0].text)

False

In [80]:
print(page_select[-1].text)
print(prensent[0].text)

8
1


In [164]:
scan_day = pd.datetime(2021,10,25)
page_count = True
page = 1

search_day = str(scan_day.year) + str(scan_day.month) + str(scan_day.day)
main_url = 'https://news.naver.com/main/list.naver?mode=LS2D&sid2=259&sid1=101&mid=shm&date=20211024&page=8'
headers = {'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36'}
main_res = requests.get(main_url, headers = headers)
main_soup = BeautifulSoup(main_res.text, 'html.parser')
path = '#main_content > div.list_body.newsflash_body > ul > li'
page_path1 = '#main_content > div.paging > a'
page_path2 = '#main_content > div.paging > strong'
main_select = main_soup.select(path)
page_select = main_soup.select(page_path1)
present = main_soup.select(page_path2)
# int(page_select[-1].text)+1 == int(prensent[0].text)


<ipython-input-164-263416e553d7>:1: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  scan_day = pd.datetime(2021,10,25)


In [169]:
int(present[0].text) == int(page_select[-1].text)+1

True

In [167]:
int(page_select[-1].text)+1

8

In [17]:
page_select[-1].text == '다음'

True

In [19]:
page_select[-1].text

[<a class="nclicks(fls.page)" href="?mode=LS2D&amp;sid2=259&amp;page2=&amp;sid1=101&amp;mid=shm&amp;date=20211024&amp;page=2">2</a>,
 <a class="nclicks(fls.page)" href="?mode=LS2D&amp;sid2=259&amp;page2=&amp;sid1=101&amp;mid=shm&amp;date=20211024&amp;page=3">3</a>,
 <a class="nclicks(fls.page)" href="?mode=LS2D&amp;sid2=259&amp;page2=&amp;sid1=101&amp;mid=shm&amp;date=20211024&amp;page=4">4</a>,
 <a class="nclicks(fls.page)" href="?mode=LS2D&amp;sid2=259&amp;page2=&amp;sid1=101&amp;mid=shm&amp;date=20211024&amp;page=5">5</a>,
 <a class="nclicks(fls.page)" href="?mode=LS2D&amp;sid2=259&amp;page2=&amp;sid1=101&amp;mid=shm&amp;date=20211024&amp;page=6">6</a>,
 <a class="nclicks(fls.page)" href="?mode=LS2D&amp;sid2=259&amp;page2=&amp;sid1=101&amp;mid=shm&amp;date=20211024&amp;page=7">7</a>,
 <a class="nclicks(fls.page)" href="?mode=LS2D&amp;sid2=259&amp;page2=&amp;sid1=101&amp;mid=shm&amp;date=20211024&amp;page=8">8</a>]

In [336]:
len(page_select)

7

In [307]:
scan_day = pd.datetime(2021,11,30)
for a in range(3) :
    search_day = str(scan_day.year) + str(scan_day.month) + str(scan_day.day)
    print(search_day)

    scan_day = scan_day - pd.Timedelta("1day")

20211130
20211129
20211128


<ipython-input-307-d63ba9e4a2e7>:1: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  scan_day = pd.datetime(2021,11,30)


In [309]:
article_df

,date,title,contents,press,good,well,sad,mad,want
0,2021.11.30. 오후 9:22,"대신밸런스스팩11호, 2일 신규 상장",[데일리안 = 이충재 기자] 대신밸런스제11호스팩이 오는 2일 코스닥 시장에 상장한...,http://www.dailian.co.kr/,0,0,0,0,0
1,2021.11.30. 오후 8:22,"LG에너지솔루션, 거래소 상장 예비심사 통과",[데일리안 = 이충재 기자] LG에너지솔루션이 한국거래소의 상장예비심사를 통과했다....,http://www.dailian.co.kr/,0,0,0,0,0
2,2021.11.30. 오후 7:57,"'연중 최저치'까지 밀린 코스피, 11월 일평균 거래액도 '바닥'",(서울=뉴스1) 강은성 기자 = 코스피 지수가 2900선 안팎에 머물며 조정이 길어...,http://news1.kr/,0,0,0,0,0
3,2021.11.30. 오후 7:46,국민은행 `Liiv M` 출시 2년만에 20만명,국민은행은 금융권 최초 이동통신서비스 Liiv M(이하 리브모바일)이 출시 2년 만...,http://www.dt.co.kr/,0,0,0,0,0
4,2021.11.30. 오후 7:46,"SBI저축銀, 2년 연속 위탁보호 청소년 보금자리 지원",SBI저축은행이 한국구세군과 함께 위탁보호종료 청소년 주거환경 개선과 자립을 지원하...,http://www.dt.co.kr/,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
432,2021.11.30. 오전 3:09,"[머니 컨설팅]ISA 만기자금, 연금계좌 넣으면 혜택 톡톡",ISA는 2016년 국내에 처음 도입됐다. 당시 계좌 하나로 예금부터 펀드까지 다양...,http://www.donga.com/,0,0,0,0,0
433,2021.11.30. 오전 3:09,"[단신]KB국민은행, Z세대 금융플랫폼 ‘리브 Next’ 선보여 外",KB국민은행이 Z세대를 위한 금융 플랫폼 ‘리브 Next’를 선보였다. 리브 넥스트...,http://www.donga.com/,0,0,0,0,0
434,2021.11.30. 오전 3:06,새마을금고 이어 신협도 가계대출 중단… “서민 돈줄 끊긴다”,새마을금고에 이어 신협도 주택담보대출 등 신규 가계대출을 중단한다. 은행권 대출 규...,http://www.donga.com/,0,0,0,0,0
435,2021.11.30. 오전 3:05,"농어업인안전보험금, 연금으로도 받는다",앞으로 농어업인안전보험에 가입한 농어업인과 그 유족은 보험금을 일시금이 아닌 연금으...,http://www.donga.com/,0,0,0,0,0


In [293]:
scan_day = pd.datetime(2021,11,30)
search_day = str(scan_day.year) + str(scan_day.month) + str(scan_day.day)
print(search_day)

scan_day = scan_day - pd.Timedelta("1day")
search_day = str(scan_day.year) + str(scan_day.month) + str(scan_day.day)
print(search_day)

20211130
20211129


<ipython-input-293-c62bcb8e1e32>:1: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  scan_day = pd.datetime(2021,11,30)


In [288]:
article_df

,date,title,contents,press,good,well,sad,mad,want
0,2021.11.30. 오후 9:22,"대신밸런스스팩11호, 2일 신규 상장",[데일리안 = 이충재 기자] 대신밸런스제11호스팩이 오는 2일 코스닥 시장에 상장한...,http://www.dailian.co.kr/,0,0,0,0,0
1,2021.11.30. 오후 8:22,"LG에너지솔루션, 거래소 상장 예비심사 통과",[데일리안 = 이충재 기자] LG에너지솔루션이 한국거래소의 상장예비심사를 통과했다....,http://www.dailian.co.kr/,0,0,0,0,0
2,2021.11.30. 오후 7:57,"'연중 최저치'까지 밀린 코스피, 11월 일평균 거래액도 '바닥'",(서울=뉴스1) 강은성 기자 = 코스피 지수가 2900선 안팎에 머물며 조정이 길어...,http://news1.kr/,0,0,0,0,0
3,2021.11.30. 오후 7:46,국민은행 `Liiv M` 출시 2년만에 20만명,국민은행은 금융권 최초 이동통신서비스 Liiv M(이하 리브모바일)이 출시 2년 만...,http://www.dt.co.kr/,0,0,0,0,0
4,2021.11.30. 오후 7:46,"SBI저축銀, 2년 연속 위탁보호 청소년 보금자리 지원",SBI저축은행이 한국구세군과 함께 위탁보호종료 청소년 주거환경 개선과 자립을 지원하...,http://www.dt.co.kr/,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
432,2021.11.30. 오전 3:09,"[머니 컨설팅]ISA 만기자금, 연금계좌 넣으면 혜택 톡톡",ISA는 2016년 국내에 처음 도입됐다. 당시 계좌 하나로 예금부터 펀드까지 다양...,http://www.donga.com/,0,0,0,0,0
433,2021.11.30. 오전 3:09,"[단신]KB국민은행, Z세대 금융플랫폼 ‘리브 Next’ 선보여 外",KB국민은행이 Z세대를 위한 금융 플랫폼 ‘리브 Next’를 선보였다. 리브 넥스트...,http://www.donga.com/,0,0,0,0,0
434,2021.11.30. 오전 3:06,새마을금고 이어 신협도 가계대출 중단… “서민 돈줄 끊긴다”,새마을금고에 이어 신협도 주택담보대출 등 신규 가계대출을 중단한다. 은행권 대출 규...,http://www.donga.com/,0,0,0,0,0
435,2021.11.30. 오전 3:05,"농어업인안전보험금, 연금으로도 받는다",앞으로 농어업인안전보험에 가입한 농어업인과 그 유족은 보험금을 일시금이 아닌 연금으...,http://www.donga.com/,0,0,0,0,0


In [269]:
len(main_select)

1

In [264]:
scan_day = pd.datetime(2021,11,30)
search-day = str(scan_day.year)+str(scan_day.month)+str(scan_day.day)
scan_day = scan_day - pd.Timedelta("1day")

<ipython-input-264-aa2abdddbb02>:1: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  scan_day = pd.datetime(2021,11,30)


'20211129'

In [259]:
scan_day - pd.Timedelta('1day')

datetime.datetime(2021, 11, 29, 0, 0)

In [232]:
page = 1

try :
    while page_count :
        main_url = 'https://news.naver.com/main/list.naver?mode=LS2D&sid2=259&page2=&sid1=101&mid=shm&date=20211130&page={}'.format(page)
        headers = {'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36'}
        main_res = requests.get(main_url, headers = headers)
        main_soup = BeautifulSoup(main_res.text, 'html.parser')
        path = '#main_content > div.list_body.newsflash_body > ul > li'
        page_path = '#main_content > div.paging > a'
        main_select = main_soup.select(path)
        page_select = main_soup.select(page_path)
        
        if 20 != len(main_select) and 10 > len(page_select):
            page_count = False
        page += 1
except :
    pass

In [224]:


for i in range(0,len(main_select)) :
    news_cont_url = main_select[i].select('a')[0]['href']

    cont_res = requests.get(news_cont_url, headers = headers)
    cont_soup = BeautifulSoup(cont_res.text, 'html.parser')
    cont_select_head = cont_soup.select('#main_content > div.article_header')
    cont_select_body = cont_soup.select('#articleBodyContents')

    target_text_without_child_tags = [bs_object for bs_object in cont_select_body[0]
        if isinstance(bs_object, NavigableString)]
    text = "".join(target_text_without_child_tags[8:-3])

    date = cont_select_head[0].select('span.t11')[0].text
    title = cont_select_head[0].select('div.article_info #articleTitle')[0].text
    contents = text
    press = cont_select_head[0].select('a')[0]['href']
    good = cont_select_head[0].select('span.u_likeit_list_count._count')[0].text
    well = cont_select_head[0].select('span.u_likeit_list_count._count')[1].text
    sad = cont_select_head[0].select('span.u_likeit_list_count._count')[2].text
    mad = cont_select_head[0].select('span.u_likeit_list_count._count')[3].text
    want = cont_select_head[0].select('span.u_likeit_list_count._count')[4].text

    data = {'date':date, 'title':title, 'contents':contents, 'press':press, 'good':good,
           'well':well, 'sad':sad, 'mad':mad, 'want':want}
    article_df = article_df.append(data, ignore_index=True)

IndexError: list index out of range

In [200]:
article_df

,date,title,contents,press,good,well,sad,mad,want
0,2021.11.30. 오후 9:22,"대신밸런스스팩11호, 2일 신규 상장",[데일리안 = 이충재 기자] 대신밸런스제11호스팩이 오는 2일 코스닥 시장에 상장한...,http://www.dailian.co.kr/,0,0,0,0,0
1,2021.11.30. 오후 8:22,"LG에너지솔루션, 거래소 상장 예비심사 통과",[데일리안 = 이충재 기자] LG에너지솔루션이 한국거래소의 상장예비심사를 통과했다....,http://www.dailian.co.kr/,0,0,0,0,0
2,2021.11.30. 오후 7:57,"'연중 최저치'까지 밀린 코스피, 11월 일평균 거래액도 '바닥'",(서울=뉴스1) 강은성 기자 = 코스피 지수가 2900선 안팎에 머물며 조정이 길어...,http://news1.kr/,0,0,0,0,0
3,2021.11.30. 오후 7:46,국민은행 `Liiv M` 출시 2년만에 20만명,국민은행은 금융권 최초 이동통신서비스 Liiv M(이하 리브모바일)이 출시 2년 만...,http://www.dt.co.kr/,0,0,0,0,0
4,2021.11.30. 오후 7:46,"SBI저축銀, 2년 연속 위탁보호 청소년 보금자리 지원",SBI저축은행이 한국구세군과 함께 위탁보호종료 청소년 주거환경 개선과 자립을 지원하...,http://www.dt.co.kr/,0,0,0,0,0
5,2021.11.30. 오후 7:46,경상수지 확대 배경은 `고령화`,최근의 경상수지 확대 증가 배경에는 급속한 고령화에 따라 저축인구 비중 상승이 크게...,http://www.dt.co.kr/,0,0,0,0,0
6,2021.11.30. 오후 7:46,"""코로나 불확실성 지속, 포용금융 확대 필요""",오미크론 코로나19 새 변이 바이러스 확산으로 금융시장 불확실성이 계속되는 가운데 ...,http://www.dt.co.kr/,0,0,0,0,0
7,2021.11.30. 오후 7:34,"[속도내는 데이터경제] 대출 잔액은 기본, 채용정보까지...은행·카드·보험사 신사업...",API(Application Programming Interface)방식을 통한 금...,http://www.dt.co.kr/,0,0,0,0,0
8,2021.11.30. 오후 7:29,(영상)'내 손안의 금융비서' 마이데이터 시장 열린다,<앵커>내일부터 `내 손안의 금융비서`로 불리는 마이데이터 시범서비스가 시작됩니다....,http://www.edaily.co.kr/,0,0,0,0,0
9,2021.11.30. 오후 7:19,"BNK경남은행, 상반기 은행권 기술금융 실적평가 '1위'",[창원=뉴시스] 홍정명 기자 = BNK경남은행은 금융위원회의 '2021년 상반기 은...,http://www.newsis.com,0,0,0,0,0


In [225]:
news_cont_url = 'https://sports.news.naver.com/news?oid=003&aid=0010861727'

cont_res = requests.get(news_cont_url, headers = headers)
cont_soup = BeautifulSoup(cont_res.text, 'html.parser')
cont_select_head = cont_soup.select('#main_content > div.article_header')
cont_select_body = cont_soup.select('#articleBodyContents')

target_text_without_child_tags = [bs_object for bs_object in cont_select_body[0]
    if isinstance(bs_object, NavigableString)]
text = "".join(target_text_without_child_tags[8:-3])

date = cont_select_head[0].select('span.t11')[0].text
title = cont_select_head[0].select('div.article_info #articleTitle')[0].text
contents = text
press = cont_select_head[0].select('a')[0]['href']
good = cont_select_head[0].select('span.u_likeit_list_count._count')[0].text
well = cont_select_head[0].select('span.u_likeit_list_count._count')[1].text
sad = cont_select_head[0].select('span.u_likeit_list_count._count')[2].text
mad = cont_select_head[0].select('span.u_likeit_list_count._count')[3].text
want = cont_select_head[0].select('span.u_likeit_list_count._count')[4].text

data = {'date':date, 'title':title, 'contents':contents, 'press':press, 'good':good,
       'well':well, 'sad':sad, 'mad':mad, 'want':want}
article_df = article_df.append(data, ignore_index=True)

IndexError: list index out of range

In [226]:
cont_select_body

[]

In [197]:
cont_select_head[0].select('span.t11')[0].text

'2021.11.30. 오후 7:34'

In [ ]:
'https://news.naver.com/main/list.naver?mode=LS2D&sid2=259&page2=&sid1=101&mid=shm&date=20211130&page={}'

In [228]:
article_df = pd.DataFrame(columns=['date','title','contents','press','good','well','sad','mad','want'])
page_count = True
page = 1
count = 1
try :
    while page_count :
        main_url = 'https://news.naver.com/main/list.naver?mode=LS2D&sid2=259&page2=&sid1=101&mid=shm&date=20211130&page={}'.format(page)
        headers = {'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36'}
        main_res = requests.get(main_url, headers = headers)
        main_soup = BeautifulSoup(main_res.text, 'html.parser')
        path = '#main_content > div.list_body.newsflash_body > ul > li'
        page_path = '#main_content > div.paging > a'
        main_select = main_soup.select(path)
        page_select = main_soup.select(page_path)

        if 20 != len(main_select) and 10 > len(page_select):
            page_count = False
        page += 1

        for i in range(0,len(main_select)) :
            news_cont_url = main_select[i].select('a')[0]['href']

            cont_res = requests.get(news_cont_url, headers = headers)
            cont_soup = BeautifulSoup(cont_res.text, 'html.parser')
            cont_select_head = cont_soup.select('#main_content > div.article_header')
            cont_select_body = cont_soup.select('#articleBodyContents')

            target_text_without_child_tags = [bs_object for bs_object in cont_select_body[0]
                if isinstance(bs_object, NavigableString)]
            text = "".join(target_text_without_child_tags[8:-3])

            date = cont_select_head[0].select('span.t11')[0].text
            title = cont_select_head[0].select('div.article_info #articleTitle')[0].text
            contents = text
            press = cont_select_head[0].select('a')[0]['href']
            good = cont_select_head[0].select('span.u_likeit_list_count._count')[0].text
            well = cont_select_head[0].select('span.u_likeit_list_count._count')[1].text
            sad = cont_select_head[0].select('span.u_likeit_list_count._count')[2].text
            mad = cont_select_head[0].select('span.u_likeit_list_count._count')[3].text
            want = cont_select_head[0].select('span.u_likeit_list_count._count')[4].text

            data = {'date':date, 'title':title, 'contents':contents, 'press':press, 'good':good,
                   'well':well, 'sad':sad, 'mad':mad, 'want':want}
            article_df = article_df.append(data, ignore_index=True)
except :
    pass

In [229]:
article_df

,date,title,contents,press,good,well,sad,mad,want
0,2021.11.30. 오후 9:22,"대신밸런스스팩11호, 2일 신규 상장",[데일리안 = 이충재 기자] 대신밸런스제11호스팩이 오는 2일 코스닥 시장에 상장한...,http://www.dailian.co.kr/,0,0,0,0,0
1,2021.11.30. 오후 8:22,"LG에너지솔루션, 거래소 상장 예비심사 통과",[데일리안 = 이충재 기자] LG에너지솔루션이 한국거래소의 상장예비심사를 통과했다....,http://www.dailian.co.kr/,0,0,0,0,0
2,2021.11.30. 오후 7:57,"'연중 최저치'까지 밀린 코스피, 11월 일평균 거래액도 '바닥'",(서울=뉴스1) 강은성 기자 = 코스피 지수가 2900선 안팎에 머물며 조정이 길어...,http://news1.kr/,0,0,0,0,0
3,2021.11.30. 오후 7:46,국민은행 `Liiv M` 출시 2년만에 20만명,국민은행은 금융권 최초 이동통신서비스 Liiv M(이하 리브모바일)이 출시 2년 만...,http://www.dt.co.kr/,0,0,0,0,0
4,2021.11.30. 오후 7:46,"SBI저축銀, 2년 연속 위탁보호 청소년 보금자리 지원",SBI저축은행이 한국구세군과 함께 위탁보호종료 청소년 주거환경 개선과 자립을 지원하...,http://www.dt.co.kr/,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
230,2021.11.30. 오후 12:00,"""고령화로 인구요인의 경상수지 흑자 기여↓…2038년 GDP 대비 0%""",(서울=뉴스1) 김성은 기자 = 인구요인의 경상수지 흑자 기여도가 올해를 정점(국내...,http://news1.kr/,0,0,0,0,0
231,2021.11.30. 오전 11:59,"금결원 ""마이데이터 금융인증서로 이용하세요""",[서울=뉴시스] 정옥주 기자 = 금융결제원(금결원)은 다음달 1일 시범실시되는 금융...,http://www.newsis.com,0,0,0,0,0
232,2021.11.30. 오전 11:58,"""고객님 어서 오십시오"" 프리랜서 시장 선점하는 글로벌 은행…국내 은행 걸음마 수준","JP모간 체이스, 리리, N26 등 글로벌 은행들이 코로나19 사태 이후 더 커지는...",http://www.mk.co.kr/,0,0,0,0,0
233,2021.11.30. 오전 11:58,"은행들, 예·적금 금리 인상 행렬…농협·부산은행도 최대 0.4%P",30일 은행권에 따르면 NH농협은행과 BNK부산은행 등이 이날부터 주요 예·적금 상...,http://biz.chosun.com/,0,0,0,0,0


In [244]:
article_df

,date,title,contents,press,good,well,sad,mad,want,summary
0,2021.01.01. 오전 10:00,해운대 랜드마크 상업시설 ‘엘시티 더몰(the MALL)’ 분양돌입… 조기 완판 기대,㈜엘시티피에프브이(PFV)가 개별분양에 돌입한 상업시설 ‘엘시티 더몰(the MAL...,https://www.asiae.co.kr/,0,0,0,0,0,‘엘시티 더몰(the MALL)’은 해운대 방문객 수요와 함께 엘시티 내 마련되는 ...
1,2021.01.01. 오전 10:00,"합리적 가격 갖춘, 시티건설 `화성 남양 시티프라디움 4차`, 1월 4일부터 정당계...",시티건설은 경기도 화성시 남양뉴타운 B-14BL에 공급하는 '화성 남양 시티프라디움...,http://www.dt.co.kr/,0,0,0,0,0,NaN
2,2021.01.01. 오전 10:00,재산세 일부 임차인이 부담…넷리스 리츠 사볼까,[이데일리 권소현 기자] 올해 글로벌 리츠 변동성이 확대될 것으로 예상되는 가운데 ...,http://www.edaily.co.kr/,0,0,0,0,0,[이데일리 권소현 기자] 올해 글로벌 리츠 변동성이 확대될 것으로 예상되는 가운데 ...
3,2021.01.01. 오전 10:01,조정대상지역 제외된 포항 ‘흥해 서희스타힐스 더 캐슬’ 오는 6일(수) 1순위 청...,"지난 17일 국토교통부가 부산 9곳, 대구 7곳, 광주 5곳, 울산 2곳, 파...",http://www.fnnews.com/,0,0,0,0,0,‘흥해 서희스타힐스 더 캐슬’은 포항시 북구 흥해읍 남성리 남옥지구 B1블록에 지하...
4,2021.01.01. 오전 10:01,지식산업센터 임대 관리 편해 투자자 사이에서 인기 높아… 장기 임대에 임대료 안 밀린다,임대인 관리와 안정적이고 꾸준한 수익을 기대할 수 있는 지식산업센터 투자에 투...,http://www.fnnews.com/,0,0,0,0,0,임대인 관리와 안정적이고 꾸준한 수익을 기대할 수 있는 지식산업센터 투자에 투자자가...
...,...,...,...,...,...,...,...,...,...,...
323112,2021.11.30. 오후 9:31,내년부터 문화재·미술품 상속세 물납 허용된다…최대 10년 분할 납부,내년부터 상속세를 미술품이나 문화재로 대신 납부할 수 있는 길이 열린다. 또 상속세...,http://www.hankookilbo.com/,0,0,0,0,0,또 상속세를 최대 10년에 걸쳐 나눠 낼 수 있게 된다.이에 따라 역사적·학술적·예...
323113,2021.11.30. 오후 9:32,"난소암 위치 정확히 포착…새 조영치료제, FDA 승인",난소암 수술 도중 암 병변을 정확하게 포착하는 데 도움을 주는 새로운 조영치료제가 ...,https://www.kormedi.com/,0,0,0,0,0,난소암 수술 도중 암 병변을 정확하게 포착하는 데 도움을 주는 새로운 조영치료제가 ...
323114,2021.11.30. 오후 9:33,"한국타이어, 캐나다 프리사이슬리 인수한다",[파이낸셜뉴스] 한국타이어가 미래 모빌리티 핵심 분야인 자율주행 부품 기술을 보...,http://www.fnnews.com/,0,0,0,0,0,NaN
323115,2021.11.30. 오후 9:33,"비츄인, 2021 대한민국 소비자만족지수1위 손소독제 부문 2년 연속 수상",'대한민국 소비자만족지수 1위'는 국내 소비자들의 만족도가 높고 미래성장 가능성을 ...,http://www.mt.co.kr/,0,0,0,0,0,NaN


In [245]:
article_df2 = pd.read_csv('afterLDA.csv',index_col=0)

In [246]:
article_df2

,title,LDA_topic_value,cluster_LDA,LDA_topic_score
0,해운대 랜드마크 상업시설 ‘엘시티 더몰(the MALL)’ 분양돌입… 조기 완판 기대,"[(0, 0.6916478), (1, 0.015565783), (6, 0.06934...",0.0,0.6916
1,"합리적 가격 갖춘, 시티건설 `화성 남양 시티프라디움 4차`, 1월 4일부터 정당계...","[(0, 0.9159487), (1, 0.01692329), (4, 0.022365...",0.0,0.9160
2,재산세 일부 임차인이 부담…넷리스 리츠 사볼까,"[(0, 0.021519404), (1, 0.20194383), (2, 0.0701...",7.0,0.4127
3,조정대상지역 제외된 포항 ‘흥해 서희스타힐스 더 캐슬’ 오는 6일(수) 1순위 청...,"[(0, 0.7659112), (1, 0.08600859), (2, 0.018735...",0.0,0.7659
4,지식산업센터 임대 관리 편해 투자자 사이에서 인기 높아… 장기 임대에 임대료 안 밀린다,"[(0, 0.57049745), (1, 0.13001157), (2, 0.04496...",0.0,0.5705
...,...,...,...,...
323112,내년부터 문화재·미술품 상속세 물납 허용된다…최대 10년 분할 납부,"[(0, 0.06022055), (1, 0.114306375), (2, 0.1795...",0.0,0.5336
323113,"난소암 위치 정확히 포착…새 조영치료제, FDA 승인","[(2, 0.3843323), (3, 0.03459393), (4, 0.418801...",0.0,0.6066
323114,"한국타이어, 캐나다 프리사이슬리 인수한다","[(3, 0.025331328), (4, 0.4659004), (6, 0.15833...",4.0,0.4294
323115,"비츄인, 2021 대한민국 소비자만족지수1위 손소독제 부문 2년 연속 수상","[(1, 0.029981995), (2, 0.022726363), (3, 0.010...",9.0,0.4040


In [247]:
article_df['LDA_topic_value'] = article_df2['LDA_topic_value']
article_df['cluster_LDA'] = article_df2['cluster_LDA']
article_df['LDA_topic_score'] = article_df2['LDA_topic_score']

In [250]:
article_df.to_csv('data_last.csv', encoding='utf-8-sig')

In [259]:
article_df

,date,title,contents,press,good,well,sad,mad,want,summary,LDA_topic_value,cluster_LDA,LDA_topic_score
0,2021.01.01. 오전 10:00,해운대 랜드마크 상업시설 ‘엘시티 더몰(the MALL)’ 분양돌입… 조기 완판 기대,㈜엘시티피에프브이(PFV)가 개별분양에 돌입한 상업시설 ‘엘시티 더몰(the MAL...,https://www.asiae.co.kr/,0,0,0,0,0,‘엘시티 더몰(the MALL)’은 해운대 방문객 수요와 함께 엘시티 내 마련되는 ...,"[(0, 0.6916478), (1, 0.015565783), (6, 0.06934...",0.0,0.6916
1,2021.01.01. 오전 10:00,"합리적 가격 갖춘, 시티건설 `화성 남양 시티프라디움 4차`, 1월 4일부터 정당계...",시티건설은 경기도 화성시 남양뉴타운 B-14BL에 공급하는 '화성 남양 시티프라디움...,http://www.dt.co.kr/,0,0,0,0,0,NaN,"[(0, 0.9159487), (1, 0.01692329), (4, 0.022365...",0.0,0.9160
2,2021.01.01. 오전 10:00,재산세 일부 임차인이 부담…넷리스 리츠 사볼까,[이데일리 권소현 기자] 올해 글로벌 리츠 변동성이 확대될 것으로 예상되는 가운데 ...,http://www.edaily.co.kr/,0,0,0,0,0,[이데일리 권소현 기자] 올해 글로벌 리츠 변동성이 확대될 것으로 예상되는 가운데 ...,"[(0, 0.021519404), (1, 0.20194383), (2, 0.0701...",7.0,0.4127
3,2021.01.01. 오전 10:01,조정대상지역 제외된 포항 ‘흥해 서희스타힐스 더 캐슬’ 오는 6일(수) 1순위 청...,"지난 17일 국토교통부가 부산 9곳, 대구 7곳, 광주 5곳, 울산 2곳, 파...",http://www.fnnews.com/,0,0,0,0,0,‘흥해 서희스타힐스 더 캐슬’은 포항시 북구 흥해읍 남성리 남옥지구 B1블록에 지하...,"[(0, 0.7659112), (1, 0.08600859), (2, 0.018735...",0.0,0.7659
4,2021.01.01. 오전 10:01,지식산업센터 임대 관리 편해 투자자 사이에서 인기 높아… 장기 임대에 임대료 안 밀린다,임대인 관리와 안정적이고 꾸준한 수익을 기대할 수 있는 지식산업센터 투자에 투...,http://www.fnnews.com/,0,0,0,0,0,임대인 관리와 안정적이고 꾸준한 수익을 기대할 수 있는 지식산업센터 투자에 투자자가...,"[(0, 0.57049745), (1, 0.13001157), (2, 0.04496...",0.0,0.5705
...,...,...,...,...,...,...,...,...,...,...,...,...,...
323112,2021.11.30. 오후 9:31,내년부터 문화재·미술품 상속세 물납 허용된다…최대 10년 분할 납부,내년부터 상속세를 미술품이나 문화재로 대신 납부할 수 있는 길이 열린다. 또 상속세...,http://www.hankookilbo.com/,0,0,0,0,0,또 상속세를 최대 10년에 걸쳐 나눠 낼 수 있게 된다.이에 따라 역사적·학술적·예...,"[(0, 0.06022055), (1, 0.114306375), (2, 0.1795...",0.0,0.5336
323113,2021.11.30. 오후 9:32,"난소암 위치 정확히 포착…새 조영치료제, FDA 승인",난소암 수술 도중 암 병변을 정확하게 포착하는 데 도움을 주는 새로운 조영치료제가 ...,https://www.kormedi.com/,0,0,0,0,0,난소암 수술 도중 암 병변을 정확하게 포착하는 데 도움을 주는 새로운 조영치료제가 ...,"[(2, 0.3843323), (3, 0.03459393), (4, 0.418801...",0.0,0.6066
323114,2021.11.30. 오후 9:33,"한국타이어, 캐나다 프리사이슬리 인수한다",[파이낸셜뉴스] 한국타이어가 미래 모빌리티 핵심 분야인 자율주행 부품 기술을 보...,http://www.fnnews.com/,0,0,0,0,0,NaN,"[(3, 0.025331328), (4, 0.4659004), (6, 0.15833...",4.0,0.4294
323115,2021.11.30. 오후 9:33,"비츄인, 2021 대한민국 소비자만족지수1위 손소독제 부문 2년 연속 수상",'대한민국 소비자만족지수 1위'는 국내 소비자들의 만족도가 높고 미래성장 가능성을 ...,http://www.mt.co.kr/,0,0,0,0,0,NaN,"[(1, 0.029981995), (2, 0.022726363), (3, 0.010...",9.0,0.4040


In [17]:
import pandas as pd

In [45]:
dic = {'test1' : 7, 'test2': 5, 'test3' : 1 }
df = pd.DataFrame(dic.items(), columns=['value','key'])
df[df['key'] ==max(dic.values())]['value']

0    test1
Name: value, dtype: object

In [58]:
a[a.index("[")+1:a.index("]")]

"'금융정책'"

In [14]:
dic.values()

dict_values

In [46]:
a = "Index(['금융정책'], dtype='object')"

In [47]:
a.index('[')

6

In [ ]:
# 패딩된 array에서 카테고리화 하는 함수
def categorizing_func(x) :
    # 모델을 불러오기
    from tensorflow.keras.models import load_model
    model = load_model('best_model_220103_R3.h5')
    
    # 모델 적용
    yhat = model.predict(x)
    
    # 기사가 1개이기 때문에 1행만 출력
    y = pd.DataFrame(yhat) #.iloc[0,:]
    
    # 카테고리 정의
    cate_dic = {0 : '부동산분양정보', 1 : '금융', 2 : '기업일반정보',3 :'증권,주식',4:'기업기술정보',5:'기업실적등', 
           6: '금융정책', 7:'부동산정책', 8:'가상화폐', 9: '주식시장'}
    y.columns = cate_dic.values()
    
    
    # 최대값인 칼럼 출력
 
    li = []

    for i in y:
        li.append(y[i].max())    
    
    dd = {}
    j = 0
    while j < len(y.columns):
        dd[y.columns[j]] = li[j]
        j += 1
    
    aaa = pd.DataFrame(columns = ['categ','score'])
    aaa['categ'] = dd.keys()
    aaa['score'] = dd.values()
    aaaa = str(aaa[aaa['score'] == aaa['score'].max()]['categ'])
    
    return re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", aaaa).strip()